##### Copyright 2024 Google LLC.

In [3]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tuning Gemma using Unsloth

Welcome to this step-by-step guide on fine-tuning the [Gemma](https://huggingface.co/google/gemma-2b) using [Unsloth](https://unsloth.ai/).


[**Gemma**](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

[**Unsloth**](https://unsloth.ai/) is a Python framework developed for finetuning large language models like Gemma 2.4x faster, using 58% less VRAM, and with no degradation in accuracy.


In this notebook, you will learn how to finetune Gemma 2 models using **Unsloth** in a Google Colab environment. You'll install the necessary packages, finetune the model, and run a sample inference.

<table align="left">
<td>
 <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Finetune_with_Unsloth.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Hugging Face setup

**Before you dive into the tutorial, let's get you set up with Hugging Face. This is needed to upload the finetuned model into Hugging Face Hub.**

1. **Hugging Face Account:**  If you don't already have one, you can create a free Hugging Face account by clicking [here](https://huggingface.co/join).
2. **Hugging Face Token:**  Generate a Hugging Face access (with `write` permission) token by clicking [here](https://huggingface.co/settings/tokens). You'll need this token later in the tutorial.

**Once you've completed these steps, you're ready to move on to the next section where you'll set up environment variables in your Colab environment.**


### Configure your HF token

Add your Hugging Face token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

In [4]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

You'll need to install a few Python packages and dependencies for Unsloth.

Run the following cell to install or upgrade the necessary packages:

In [2]:
#   Install Unsloth library
! pip install unsloth --upgrade --no-cache-dir

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 216.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 251.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Finetuning Gemma 2 using Unsloth library

### Initializing Gemma 2 model

Unsloth library supports a variety of open-source LLMs including Gemma. For this notebook, you will use Gemma 2's 2b model. You can load the Gemma 2 model in Unsloth using the `FastLanguageModel` class. To know more about the other variants of Gemma 2 model provided by Unsloth, visit the [Unsloth's supported models documentation](https://docs.unsloth.ai/get-started/all-our-models).

In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = None,        # None for auto detection.
    load_in_4bit = True, # Use 4bit quantization to reduce memory usage.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Gemma2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

### Load a dataset

For this guide, you'll use an existing dataset from Hugging Face. You can replace it with your own dataset if you prefer.

The dataset chosen for this guide is [**yahma/alpaca-cleaned**](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a clean version of the original Alpaca dataset by Stanford. The Alpaca dataset is a collection of over 50,000 instructions and demonstrations that can be used to fine-tune language models to better understand and follow instructions.

**Credits:** **https://huggingface.co/yahma**

In [6]:
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Let's look at a few samples to understand the data.

In [7]:
dataset[15]

{'output': "The motherboard, also known as the mainboard or system board, is the central printed circuit board in a computer. It serves as the backbone or foundation for a computer, connecting all the different components such as the CPU, RAM, storage drives, expansion cards, and peripherals. The motherboard manages communication and data transfer between these components, allowing them to work together and perform their designated tasks.\n\nThe motherboard also includes important circuitry such as the power regulation circuit that provides power to the different components, and the clock generator which synchronizes the operation of these components. It also contains the BIOS (basic input/output system), which is a firmware that controls the boot process and provides an interface for configuring and managing the computer's hardware. Other features on a motherboard may include built-in networking, audio, and video capabilities.\n\nOverall, the function of a computer motherboard is to p

In [8]:
from google.colab import data_table
import pandas as pd

# Enable interactive DataFrame display
data_table.enable_dataframe_formatter()

# Convert the 'train' split to a Pandas DataFrame
df_train = pd.DataFrame(dataset)

# Select the first 5 rows
df_train.head(5)

,output,input,instruction
0,1. Eat a balanced and nutritious diet: Make su...,,Give three tips for staying healthy.
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?
2,An atom is the basic building block of all mat...,,Describe the structure of an atom.
3,There are several ways to reduce air pollution...,,How can we reduce air pollution?
4,I had to make a difficult decision when I was ...,,Pretend you are a project manager of a constru...


### Set the prompt template

Here you will define the Alpaca prompt template. This template has 3 sections:
- Instruction
- Input
- Response

In [9]:
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an
input that provides further context. Write a response that appropriately
completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # EOS_TOKEN is necessary.

### Define the formatting function

The formatting function applies the template created above to each row in the dataset and converts it into a format suited for training.

In [10]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    # EOS_TOKEN is necessary, otherwise your generation will go on forever!
    texts = [alpaca_prompt_template.format(instruction, input, output) + EOS_TOKEN
                                  for instruction, input, output in
                                  zip(instructions, inputs, outputs)]
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

### Set LoRA configuration

LoRA (Low-Rank Adaptation) allows for efficient fine-tuning by adapting only a subset of model parameters.

Here, you set the following parameters:
- `r` to 16, which controls the rank of the adaptation matrices.
- `lora_alpha` to 16 for scaling.
- `lora_dropout` to 0 since it is optimized.

To know more about LoRA parameters and their effects, check out the [LoRA parameters encyclopedia](https://github.com/unslothai/unsloth/wiki#lora-parameters-encyclopedia).

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA attention dimension
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # Alpha parameter for LoRA scaling
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # Rank stabilized LoRA
    loftq_config = None, # LoRA-Fine-Tuning-Aware Quantization
)

Unsloth 2025.3.19 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


### Set training configuration

Set up the training arguments that define how the model will be trained.

Here, you'll define the following parameters:

- For training and evaluation:
  - `output directory`
  - `max steps`
  - `batch sizes`

- To optimize the training process:
  - `learning rate`
  - `optimizer`
  - `learning rate scheduler`

**Note:** `max_steps` is set as 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run.

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    # num_train_epochs = 1, # Set this for 1 full training run.
    max_steps = 60,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 42,
    output_dir = "outputs",
    report_to = "none",
)

<a name="Train"></a>
### Train the model

[Huggingface's TRL](https://huggingface.co/docs/trl/index) offers a user-friendly API for building SFT models and training them on your dataset with just a few lines of code. Here you will use Huggingface TRL's `SFTTrainer` class to train the model. This class inherits from the `Trainer` class available in the Transformers library, but is specifically optimized for supervised fine-tuning (instruction tuning). Read more about SFFTrainer from the [official TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # Setting packing as False can speed up training five times
    # for short sequences.
    packing = False,
    args = training_args
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

Now, let's start the fine-tuning process by calling `trainer.train()`, which uses `SFTTrainer` to handle the training loop, including data loading, forward and backward passes, and optimizer steps, all configured according to the settings you've provided.

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,766,720/2,000,000,000 (1.04% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.875100
2,2.087000
3,2.079900
4,2.016000
5,1.536000
6,1.430400
7,1.328500
8,1.366200
9,1.179400
10,1.110200


### Save the model locally

After training is complete, save the fine-tuned model by calling `save_pretrained(new_model)`. This saves the model weights and configuration files to the directory specified by `new_model` (**gemma_ft_unsloth**). You can reload and use the fine-tuned model later for inference or further training.

In [15]:
new_model = "gemma_ft_unsloth"
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('gemma_ft_unsloth/tokenizer_config.json',
 'gemma_ft_unsloth/special_tokens_map.json',
 'gemma_ft_unsloth/tokenizer.model',
 'gemma_ft_unsloth/added_tokens.json',
 'gemma_ft_unsloth/tokenizer.json')

### Push to Hugging Face Hub

You can use the model's `push_to_hub` method to upload the model to Hugging Face Hub.

**Note**: In the following code snippets, replace "your_hf_username" to your Hugging Face username.

In [16]:
# Push the trained model to Hub
model.push_to_hub("Fabbernat/gemma_ft_unsloth")

HfHubHTTPError: (Request ID: Root=1-68087302-6370e65a25ad793878383a22;1bdd74e6-b32f-471a-9597-e23da2becc57)

403 Forbidden: You don't have the rights to create a model under the namespace "your_hf_username".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

If you only want to save the Lora adapters, uncomment and run the code below.

In [ ]:
#model.push_to_hub_merged("your_hf_username/gemma_ft_lora", tokenizer,
#                         save_method = "lora", token = os.environ["HF_TOKEN"])

If you want to save the model as GGUF, uncomment and run the code below. To check the different quantization methods supported by Unsloth, visit
[Unsloth wiki's Save as GGUF section](https://github.com/unslothai/unsloth/wiki#saving-to-gguf).

In [ ]:
#model.push_to_hub_gguf("your_hf_username/gemma_ft_q4_k_m",
#                       tokenizer, quantization_method = "q4_k_m")

To know more about the different formats available in Unsloth, check out [Unsloth wiki's saving models section](https://github.com/unslothai/unsloth/wiki#saving-models-to-16bit-for-vllm).

## Inference

### Prompt using the newly fine-tuned model


Now that you've finally fine-tuned your custom Gemma model, let's reload the LoRA adapter weights and tokenizer to finally prompt using it and also verify if it's really working as intended.

In [1]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model, # Your finetuned model name
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

NameError: name 'FastLanguageModel' is not defined

Now, test the fine-tuned model with a sample prompt by first using the tokenizer to generate the input ids, and then relying on the reloaded fine-tuned model to generate a response using `model.generate()`.

Instead of waiting the entire time, you can view the generation token by token by using a `TextStreamer` for continuous inference.

In [2]:
from transformers import TextStreamer

inputs = tokenizer(
[
    alpaca_prompt_template.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

NameError: name 'tokenizer' is not defined

Congratulations! You've successfully fine-tuned Gemma using Unsloth. You've covered the entire process, from setting up the environment to training and testing the model.

## What's next?
Your next steps could include the following:

- **Experiment with Different Datasets**: Try fine-tuning on other datasets in [Hugging Face](https://huggingface.co/docs/datasets/en/index) or your own data to adapt the model to various tasks or domains.

- **Tune Hyperparameters**: Adjust training parameters (e.g., learning rate, batch size, epochs, LoRA settings) to optimize performance and improve training efficiency.

- **Save model in different formats**: Check the different saving formats provided by Unsloth and use it in `llama.cpp` or a UI based system like `GPT4All`.

By exploring these activities, you'll deepen your understanding and further enhance your fine-tuned Gemma model. Happy experimenting!

In [3]:
human_readable_questions_full = {
'Does the word "defeat" mean the same thing in sentences "It was a narrow defeat." and "The army\'s only defeat."?': 'Yes',
'Does the word "groom" mean the same thing in sentences "Groom the dogs." and "Sheila groomed the horse."?': 'Yes',
'Does the word "penetration" mean the same thing in sentences "The penetration of upper management by women." and "Any penetration, however slight, is sufficient to complete the offense."?': 'Yes',
'Does the word "hit" mean the same thing in sentences "We hit Detroit at one in the morning but kept driving through the night." and "An interesting idea hit her."?': 'No',
'Does the word "deliberation" mean the same thing in sentences "He was a man of judicial deliberation." and "A little deliberation would have deterred them."?': 'No',
'Does the word "navel" mean the same thing in sentences "They argued whether or not Adam had a navel." and "You were not supposed to show your navel on television."?': 'Yes',
'Does the word "afforest" mean the same thing in sentences "After we leave the quarry, we intend to afforest the land and turn it into a nature reserve." and "Afforest the mountains."?': 'Yes',
'Does the word "solve" mean the same thing in sentences "Solve an old debt." and "Did you solve the problem ?"?': 'No',
'Does the word "purchase" mean the same thing in sentences "They offer a free hamburger with the purchase of a drink." and "They closed the purchase with a handshake."?': 'Yes',
'Does the word "software" mean the same thing in sentences "Did you test the software package to ensure completeness ?" and "The market for software is expected to expand."?': 'Yes',
'Does the word "push" mean the same thing in sentences "Some details got lost in the push to get the project done." and "The army made a push toward the sea."?': 'Yes',
'Does the word "bake" mean the same thing in sentences "Idaho potatoes bake beautifully." and "This oven bakes potatoes."?': 'Yes',
'Does the word "relieve" mean the same thing in sentences "Relieve the pressure and the stress." and "This pill will relieve your headaches."?': 'No',
'Does the word "style" mean the same thing in sentences "In the characteristic New York style." and "This style of shoe is in demand."?': 'No',
'Does the word "crumb" mean the same thing in sentences "Crumb the table." and "Crumb a cutlet."?': 'No',
'Does the word "include" mean the same thing in sentences "I include you in the list of culprits." and "The list includes the names of many famous writers."?': 'No',
'Does the word "companion" mean the same thing in sentences "His dog has been his trusted companion for the last five years." and "Drinking companions."?': 'Yes',
'Does the word "reveal" mean the same thing in sentences "The actress wo n\'t reveal how old she is." and "He revealed the children found."?': 'Yes',
'Does the word "presence" mean the same thing in sentences "I\'m convinced that there was a presence in that building that I ca n\'t explain, which led to my heroic actions." and "She blushed in his presence."?': 'No',
'Does the word "relax" mean the same thing in sentences "Do n\'t relax your efforts now." and "The rules relaxed after the new director arrived."?': 'No',
'Does the word "parity" mean the same thing in sentences "Parity is often used to check the integrity of transmitted data." and "The parity of the mother must be considered."?': 'No',
'Does the word "raise" mean the same thing in sentences "To raise a wall, or a heap of stones." and "Raise a barn."?': 'Yes',
'Does the word "suspend" mean the same thing in sentences "Suspend the particles." and "The prison sentence was suspended."?': 'No',
'Does the word "amass" mean the same thing in sentences "To amass a treasure or a fortune." and "She is amassing a lot of data for her thesis."?': 'Yes',
'Does the word "term" mean the same thing in sentences "A healthy baby born at full term." and "He learned many medical terms."?': 'No',
'Does the word "leash" mean the same thing in sentences "He\'s always gotten a long leash." and "Kept a tight leash on his emotions."?': 'Yes',
'Does the word "conversion" mean the same thing in sentences "The conversion of equations." and "His conversion to the Catholic faith."?': 'No',
'Does the word "making" mean the same thing in sentences "The making of measurements." and "It was already in the making."?': 'Yes',
'Does the word "set" mean the same thing in sentences "Before the set of sun." and "They played two sets of tennis after dinner."?': 'No',
'Does the word "mate" mean the same thing in sentences "He lost the mate to his shoe." and "Camels hate leaving their mates."?': 'No',
'Does the word "expression" mean the same thing in sentences "They stared at the newcomer with a puzzled expression." and "His manner of expression showed how much he cared."?': 'No',
'Does the word "rim" mean the same thing in sentences "Rim a hat." and "Sugar rimmed the dessert plate."?': 'No',
'Does the word "cure" mean the same thing in sentences "Cure meats." and "Cure hay."?': 'Yes',
'Does the word "rift" mean the same thing in sentences "My marriage is in trouble, the fight created a rift between us and we ca n\'t reconnect." and "The Grand Canyon is a rift in the Earth\'s surface, but is smaller than some of the undersea ones."?': 'Yes',
'Does the word "swim" mean the same thing in sentences "We had to swim for 20 minutes to reach the shore." and "A big fish was swimming in the tank."?': 'Yes',
'Does the word "quiet" mean the same thing in sentences "The teacher did her best to quiet the children down." and "When you quiet, we can start talking."?': 'No',
'Does the word "top" mean the same thing in sentences "At the top of his profession." and "To be at the top of one\'s class, or at the top of the school."?': 'Yes',
'Does the word "consultation" mean the same thing in sentences "A consultation of several medical specialists." and "Frequent consultations with his lawyer."?': 'Yes',
'Does the word "chiromance" mean the same thing in sentences "She refused to chiromance my fate." and "The Gypsies chiromanced."?': 'Yes',
'Does the word "bank" mean the same thing in sentences "Bank on your good education." and "The pilot had to bank the aircraft."?': 'No',
'Does the word "rag" mean the same thing in sentences "Rag that old tune." and "Rag ore."?': 'No',
'Does the word "work" mean the same thing in sentences "Work equals force times distance." and "Work is done against friction to drag a bag along the ground."?': 'Yes',
'Does the word "allowance" mean the same thing in sentences "He objected to the allowance of smoking in the dining room." and "A child\'s allowance should not be too generous."?': 'No',
'Does the word "contact" mean the same thing in sentences "Litmus paper turns red on contact with an acid." and "He used his business contacts to get an introduction to the governor."?': 'No',
'Does the word "virus" mean the same thing in sentences "The virus of jealousy is latent in everyone." and "He caught a virus and had to stay home from school."?': 'No',
'Does the word "humour" mean the same thing in sentences "The sensitive subject was treated with humour, but in such way that no one was offended." and "She has a great sense of humour, and I always laugh a lot whenever we get together."?': 'Yes',
'Does the word "neighbor" mean the same thing in sentences "What is the closest neighbor to the Earth ?" and "Fort Worth is a neighbor of Dallas."?': 'Yes',
'Does the word "sinking" mean the same thing in sentences "He could not control the sinking of his legs." and "After several hours of sinking an unexpected rally rescued the market."?': 'Yes',
'Does the word "sneak" mean the same thing in sentences "Sneak a look." and "Sneak a cigarette."?': 'Yes',
'Does the word "fix" mean the same thing in sentences "Fix your eyes on this spot." and "Fix a race."?': 'No',
'Does the word "impulse" mean the same thing in sentences "The impulse knocked him over." and "The total impulse from the impact will depend on the kinetic energy of the bullet."?': 'No',
'Does the word "fetish" mean the same thing in sentences "I know a guy who has a foot fetish." and "Common male fetishes are breasts, legs, hair, shoes, and underwear."?': 'Yes',
'Does the word "seizure" mean the same thing in sentences "The seizure of a thief, a property, a throne, etc." and "The search warrant permitted the seizure of evidence."?': 'Yes',
'Does the word "pattern" mean the same thing in sentences "The American constitution has provided a pattern for many republics." and "They changed their dietary pattern."?': 'No',
'Does the word "conscience" mean the same thing in sentences "A person of unflagging conscience." and "He has no conscience about his cruelty."?': 'No',
'Does the word "demystify" mean the same thing in sentences "The article was written to demystify the mechanics of the internal combustion engine." and "Let\'s demystify the event by explaining what it is all about."?': 'Yes',
'Does the word "team" mean the same thing in sentences "We need more volunteers for the netball team." and "The IT manager leads a team of three software developers."?': 'Yes',
'Does the word "conduct" mean the same thing in sentences "You can not conduct business like this." and "To conduct the affairs of a kingdom."?': 'Yes',
'Does the word "administer" mean the same thing in sentences "Administer critical remarks to everyone present." and "She administers the funds."?': 'No',
'Does the word "abort" mean the same thing in sentences "I wasted a year of my life working on an abort." and "He sent a short message requesting an abort due to extreme winds in the area."?': 'Yes',
'Does the word "glass" mean the same thing in sentences "She collected old glass." and "We collected art glass."?': 'Yes',
'Does the word "oppose" mean the same thing in sentences "The senator said he would oppose the bill." and "The board opposed his motion."?': 'No',
'Does the word "married" mean the same thing in sentences "My plan is to wait to have sex until i\'m married." and "We invited several young marrieds."?': 'Yes',
'Does the word "utility" mean the same thing in sentences "A computer system provides utility programs to perform the tasks needed by most users." and "I\'ve bought a new disk utility that can recover deleted files."?': 'Yes',
'Does the word "passive" mean the same thing in sentences "` The ball was thrown by the boy\' uses the passive voice." and "` The ball was thrown\' is an abbreviated passive."?': 'Yes',
'Does the word "hold" mean the same thing in sentences "Hold in place." and "She always held herself as a lady."?': 'Yes',
'Does the word "bit" mean the same thing in sentences "A bit of paper." and "A bit of rock caught him in the eye."?': 'No',
'Does the word "induce" mean the same thing in sentences "Induce a crisis." and "The ads induced me to buy a VCR."?': 'No',
'Does the word "center" mean the same thing in sentences "Convention center." and "They received messages from several centers."?': 'Yes',
'Does the word "morale" mean the same thing in sentences "Morale is an important quality in soldiers. With good morale they\'ll charge into a hail of bullets ; without it they wo n\'t even cross a street." and "After the layoffs morale was at an all time low, they were so dispirited nothing was getting done."?': 'Yes',
'Does the word "security" mean the same thing in sentences "Military security has been stepped up since the recent uprising." and "The head of security was a former policeman."?': 'No',
'Does the word "picture" mean the same thing in sentences "The author gives a depressing picture of life in Poland." and "Too often the narrative was interrupted by long word pictures."?': 'Yes',
'Does the word "judge" mean the same thing in sentences "I judge a man\'s character by the cut of his suit." and "I can not judge some works of modern art."?': 'Yes',
'Does the word "cast" mean the same thing in sentences "The cast would need a great deal of machining to become a recognizable finished part." and "The cast was praised for a fine performance."?': 'No',
'Does the word "fire" mean the same thing in sentences "Do n\'t fire until you see the whites of their eyes." and "The gun fired."?': 'Yes',
'Does the word "grab" mean the same thing in sentences "This story will grab you." and "Grab power."?': 'No',
'Does the word "passion" mean the same thing in sentences "We share a passion for books." and "He has a passion for cock fighting."?': 'No',
'Does the word "read" mean the same thing in sentences "Can you read Greek ?" and "The gauge read ` empty\'."?': 'No',
'Does the word "roast" mean the same thing in sentences "Roast the turkey." and "To roast meat on a spit."?': 'Yes',
'Does the word "dress" mean the same thing in sentences "He came to the party in formal dress." and "Fastidious about his dress."?': 'Yes',
'Does the word "miss" mean the same thing in sentences "How could I miss that typo ?" and "The workers on the conveyor belt miss one out of ten."?': 'Yes',
'Does the word "rerun" mean the same thing in sentences "We have to rerun the subjects -- they misunderstood the instructions." and "Bush wants to rerun in 1996."?': 'No',
'Does the word "chip" mean the same thing in sentences "Be careful not to chip the paint." and "Chip a tooth."?': 'Yes',
'Does the word "share" mean the same thing in sentences "They share a language." and "To share a shelter with another."?': 'Yes',
'Does the word "stone" mean the same thing in sentences "A heavy chap who must have weighed more than twenty stone." and "She had jewels made of all the rarest stones."?': 'No',
'Does the word "approach" mean the same thing in sentences "Would counsel please approach the bench ? asked the judge." and "He approached the age of manhood."?': 'Yes',
'Does the word "restore" mean the same thing in sentences "Restore the emperor to the throne." and "To restore harmony among those who are at variance."?': 'Yes',
'Does the word "election" mean the same thing in sentences "The results of the election will be announced tonight." and "They celebrated his election."?': 'No',
'Does the word "bell" mean the same thing in sentences "Saved by the bell." and "She heard the distant toll of church bells."?': 'Yes',
'Does the word "logic" mean the same thing in sentences "Economic logic requires it." and "By the logic of war."?': 'Yes',
'Does the word "drip" mean the same thing in sentences "After putting oil on the side of the salad, the chef should drip a little vinegar in the oil." and "My broken pen dripped ink onto the table."?': 'Yes',
'Does the word "event" mean the same thing in sentences "He acted very wise after the event." and "In that event, the first possibility is excluded."?': 'No',
'Does the word "approval" mean the same thing in sentences "He bought it on approval." and "Although she fussed at them, she secretly viewed all her children with approval."?': 'No',
'Does the word "marmalade" mean the same thing in sentences "Thick cut marmalade." and "Lime marmalade."?': 'Yes',
'Does the word "dress" mean the same thing in sentences "Parents must feed and dress their child." and "I rose and dressed before daybreak."?': 'Yes',
'Does the word "square" mean the same thing in sentences "You can compute the area of a square if you know the length of its sides." and "A checkerboard has 64 squares."?': 'No',
'Does the word "trip" mean the same thing in sentences "He took a trip to the shopping center." and "We made a trip to the beach."?': 'Yes',
'Does the word "restraint" mean the same thing in sentences "He was a model of polite restraint." and "The unlawful restraint of trade."?': 'No',
'Does the word "seating" mean the same thing in sentences "There is plenty of comfortable seating." and "There is seating for 40 students in this classroom."?': 'Yes',
'Does the word "irony" mean the same thing in sentences "The irony of Ireland\'s copying the nation she most hated." and "Irony is wasted on the stupid."?': 'No',
'Does the word "flare" mean the same thing in sentences "The skirt had a wide flare." and "She could not control her flare of rage."?': 'No',
'Does the word "internationalize" mean the same thing in sentences "Internationalize trade of certain drugs." and "We internationalized the committee."?': 'No',
'Does the word "favor" mean the same thing in sentences "She enjoyed the queen\'s favor." and "To fall out of favor."?': 'Yes',
'Does the word "sit" mean the same thing in sentences "I asked him to sit." and "She sat on the jury."?': 'No',
'Does the word "shore" mean the same thing in sentences "Shore and buttress an old building." and "The river was shored by trees."?': 'No',
'Does the word "snorter" mean the same thing in sentences "The storm was n\'t long but it was a snorter." and "A snorter of a sermon."?': 'Yes',
'Does the word "government" mean the same thing in sentences "Tyrannical government." and "He had considerable experience of government."?': 'No',
'Does the word "producer" mean the same thing in sentences "Maine is a leading producer of potatoes." and "This microorganism is a producer of disease."?': 'Yes',
'Does the word "step" mean the same thing in sentences "The architect wants to step the terrace." and "She is always stepping on others to get ahead."?': 'No',
'Does the word "lock" mean the same thing in sentences "If you put the brakes on too hard, the wheels will lock." and "She locked her jewels in the safe."?': 'Yes',
'Does the word "interview" mean the same thing in sentences "The reporter gave the witness an interview." and "My interviews with teenagers revealed a weakening of religious bonds."?': 'Yes',
'Does the word "estimation" mean the same thing in sentences "In my estimation the boy is innocent." and "They had a high estimation of his ability."?': 'No',
'Does the word "backup" mean the same thing in sentences "A traffic backup on the main street." and "He discovered a backup in the toilet."?': 'Yes',
'Does the word "lubricate" mean the same thing in sentences "Lubricate my car." and "The liquid in this can lubricates well."?': 'No',
'Does the word "wash" mean the same thing in sentences "Dishwashers wash dishes much more efficiently than most humans." and "Wash the towels, please !"?': 'No',
'Does the word "nose" mean the same thing in sentences "The hound has a good nose." and "The nose of the rocket heated up on reentry."?': 'No',
'Does the word "presence" mean the same thing in sentences "He felt the presence of an evil force." and "He sensed the presence of danger."?': 'No',
'Does the word "ceiling" mean the same thing in sentences "The dining room had an ornate ceiling." and "He put a ceiling on the number of women who worked for him."?': 'No',
'Does the word "ruin" mean the same thing in sentences "You have brought ruin on this entire family." and "They explored several Roman ruins."?': 'No',
'Does the word "bandage" mean the same thing in sentences "Bandage an incision." and "The nurse bandaged a sprained ankle."?': 'Yes',
'Does the word "move" mean the same thing in sentences "I was sitting on the sofa for a long time, I was too lazy to move." and "We moved from our previous position."?': 'Yes',
'Does the word "conflict" mean the same thing in sentences "He noticed a conflict in the dates of the two meetings." and "This form of conflict is essential to Mann\'s writing."?': 'No',
'Does the word "cookery" mean the same thing in sentences "People are needed who have experience in cookery." and "Henry was not very good at cookery and most of his meals ended up burned."?': 'Yes',
'Does the word "image" mean the same thing in sentences "A public image is as fragile as Humpty Dumpty." and "A movie is a series of images projected so rapidly that the eye integrates them."?': 'No',
'Does the word "beat" mean the same thing in sentences "He heard the beat of a drum." and "The conductor set the beat."?': 'No',
'Does the word "pitch" mean the same thing in sentences "Pitch a tent." and "Pitch the tent over there."?': 'Yes',
'Does the word "marginalization" mean the same thing in sentences "The marginalization of the underclass." and "The marginalization of literature."?': 'Yes',
'Does the word "plus" mean the same thing in sentences "Two plus two equals four." and "Four plus three equals seven."?': 'Yes',
'Does the word "god" mean the same thing in sentences "He was a god among men." and "Money was his god."?': 'No',
'Does the word "defang" mean the same thing in sentences "Defang the poisonous snake." and "The snake was defanged."?': 'Yes',
'Does the word "school" mean the same thing in sentences "The school was founded in 1900." and "When the school day was done we would walk home together."?': 'No',
'Does the word "enrich" mean the same thing in sentences "Hobbies enrich lives." and "The oil boom enriched a lot of local people."?': 'No',
'Does the word "visibility" mean the same thing in sentences "A windshield with good visibility." and "That candidate does not have sufficient visibility to win an election."?': 'No',
'Does the word "post" mean the same thing in sentences "Post a sign." and "I\'ll post the news on the bulletin board."?': 'No',
'Does the word "adhesion" mean the same thing in sentences "The adhesion of Seville was decisive." and "A heated hydraulic press was required for adhesion."?': 'No',
'Does the word "sister" mean the same thing in sentences "My sister is always driving me crazy." and "My sister married a musician."?': 'Yes',
'Does the word "expose" mean the same thing in sentences "Expose the blanket to sunshine." and "Expose your students to art."?': 'Yes',
'Does the word "sound" mean the same thing in sentences "The sound of rain on the roof." and "The beautiful sound of music."?': 'Yes',
'Does the word "jolt" mean the same thing in sentences "The door closed with a jolt." and "All the jars and jolts were smoothed out by the shock absorbers."?': 'Yes',
'Does the word "sketch" mean the same thing in sentences "Sketch the building." and "Sketch the outline of the book."?': 'No',
'Does the word "fill" mean the same thing in sentences "There was not enough fill for the trench." and "The mixer returned to the plant for another fill."?': 'Yes',
'Does the word "depth" mean the same thing in sentences "The depth of the water." and "Depth of a closet."?': 'Yes',
'Does the word "organize" mean the same thing in sentences "Can you help me organize my files ?" and "Organize a strike."?': 'No',
'Does the word "gas" mean the same thing in sentences "He stepped on the gas." and "The atmosphere is made up of a number of different gases."?': 'No',
'Does the word "nonproliferation" mean the same thing in sentences "Nuclear disarmament and nonproliferation are closely related goals." and "They protested that the nonproliferation treaty was just a plot to maintain the hegemony of those who already had nuclear weapons."?': 'Yes',
'Does the word "rig" mean the same thing in sentences "Rig a ship." and "They rigged the bomb to the ignition."?': 'No',
'Does the word "peace" mean the same thing in sentences "The roommates lived in peace together." and "Peace came on November 11th."?': 'No',
'Does the word "drag" mean the same thing in sentences "Too many laws are a drag on the use of new land." and "Taxation is a drag on the economy."?': 'Yes',
'Does the word "introduce" mean the same thing in sentences "Introduce the new neighbors to the community." and "A new word processor was introduced."?': 'No',
'Does the word "shtik" mean the same thing in sentences "Give him a shtik cake." and "His shtik made us laugh."?': 'No',
'Does the word "history" mean the same thing in sentences "All of human history." and "History takes the long view."?': 'No',
'Does the word "harness" mean the same thing in sentences "Harness natural forces and resources." and "Harness the horse."?': 'No',
'Does the word "clatter" mean the same thing in sentences "The clatter of iron wheels on cobblestones." and "Clatter of shutters."?': 'Yes',
'Does the word "religion" mean the same thing in sentences "Islam is a major religion in parts of Asia and Africa." and "Eckankar is a new religion but Zoroastrianism is an old religion."?': 'Yes',
'Does the word "squash" mean the same thing in sentences "We all managed to squash into Mum\'s tiny car." and "Somehow, she squashed all her books into her backpack, which was now too heavy to carry."?': 'Yes',
'Does the word "sex" mean the same thing in sentences "She did n\'t want to know the sex of the foetus." and "Slime molds are sometimes erroneously said to have thirteen sexes."?': 'Yes',
'Does the word "estimate" mean the same thing in sentences "I estimate this chicken to weigh three pounds." and "He estimated what would be likely."?': 'Yes',
'Does the word "burst" mean the same thing in sentences "The bubble burst." and "I blew the balloon up too much, and it burst."?': 'Yes',
'Does the word "relish" mean the same thing in sentences "I do n\'t relish the idea of going out tonight." and "He relishes their time together."?': 'Yes',
'Does the word "twist" mean the same thing in sentences "Twist one\'s head." and "Twist the strips of dough."?': 'No',
'Does the word "yacht" mean the same thing in sentences "Would you like to go sailing on my uncle\'s yacht ?" and "You are a true yachtsman ! Are you a member of the local yacht club ?"?': 'Yes',
'Does the word "poison" mean the same thing in sentences "This mushrooms can poison." and "Her husband poisoned her drink in order to kill her."?': 'No',
'Does the word "guard" mean the same thing in sentences "The left guard was injured on the play." and "Guards must be good blockers."?': 'No',
'Does the word "unleash" mean the same thing in sentences "Unleash the dogs in the park." and "He unleashed his dog in the park."?': 'Yes',
'Does the word "glorify" mean the same thing in sentences "Glorify one\'s spouse\'s cooking." and "You are glorifying a rather mediocre building."?': 'No',
'Does the word "storage" mean the same thing in sentences "I′d recommend backing up these files to storage before reinstalling the operating system." and "There\'s a lot of storage space in the loft."?': 'No',
'Does the word "region" mean the same thing in sentences "Here we enter the region of opinion." and "The equatorial regions."?': 'No',
'Does the word "rock" mean the same thing in sentences "There were rocks all over the garden." and "Thou art Peter, and upon this rock I will build my church -- Gospel According to Matthew."?': 'Yes',
'Does the word "patch" mean the same thing in sentences "A patch of bad weather." and "When ice skating, be sure to stay away from reeds, there\'s always thin patches of ice there and you could fall through."?': 'Yes',
'Does the word "zone" mean the same thing in sentences "The white zone is for loading and unloading only." and "There is a no-smoking zone that extends 25 feet outside of each entrance."?': 'Yes',
'Does the word "score" mean the same thing in sentences "Score the SAT essays." and "She scored high on the SAT."?': 'No',
'Does the word "scopolamine" mean the same thing in sentences "Transdermal scopolamine is used to treat motion sickness." and "Someone sedated with scopolamine has difficulty lying."?': 'Yes',
'Does the word "rake" mean the same thing in sentences "Rake leaves." and "The enemy machine guns raked the roadway."?': 'No',
'Does the word "emphasis" mean the same thing in sentences "His emphasis on civil rights." and "He used a yellow highlighter to indicate where to give emphasis in his speech."?': 'No',
'Does the word "explosion" mean the same thing in sentences "The information explosion." and "An explosion of laughter."?': 'No',
'Does the word "assume" mean the same thing in sentences "Mr. Jones will assume the position of a lifeguard until a proper replacement is found." and "She assumed strange manners."?': 'Yes',
'Does the word "nightclub" mean the same thing in sentences "There are both gays and heteros in this nightclub." and "The gossip columnist got his information by visiting nightclubs every night."?': 'Yes',
'Does the word "flex" mean the same thing in sentences "Flex your wrists." and "Flex a muscle."?': 'No',
'Does the word "nail" mean the same thing in sentences "Nail the board onto the wall." and "The chemists could not nail the identity of the chromosome."?': 'No',
'Does the word "form" mean the same thing in sentences "Social groups form everywhere." and "They formed a company."?': 'Yes',
'Does the word "hoist" mean the same thing in sentences "Hoist the flags." and "Hoist a sail."?': 'Yes',
'Does the word "sacrifice" mean the same thing in sentences "The general had to sacrifice several soldiers to save the regiment." and "The animals were sacrificed after the experiment."?': 'Yes',
'Does the word "summon" mean the same thing in sentences "Summon a lawyer." and "Summon all your courage."?': 'No',
'Does the word "bond" mean the same thing in sentences "A $ 10,000 bond was furnished by an alderman." and "Organic chemistry primarily consists of the study of carbon bonds, in their many variations."?': 'No',
'Does the word "hem" mean the same thing in sentences "It seeped along the hem of his jacket." and "Let down the hem."?': 'Yes',
'Does the word "right" mean the same thing in sentences "We\'re on the side of right in this contest." and "Mineral rights."?': 'Yes',
'Does the word "feel" mean the same thing in sentences "I can feel the sadness in his poems." and "The runner felt her pulse."?': 'No',
'Does the word "understand" mean the same thing in sentences "She did not understand her husband." and "I understand him to be a good doctor."?': 'Yes',
'Does the word "have" mean the same thing in sentences "I have a house and a car." and "I\'ve got a house in the country."?': 'Yes',
'Does the word "chop" mean the same thing in sentences "Chop a hole in the ground." and "Chop meat."?': 'No',
'Does the word "massacre" mean the same thing in sentences "St. Valentine\'s Day massacre." and "The massacre on St. Bartholomew\'s Day."?': 'Yes',
'Does the word "neck" mean the same thing in sentences "The horse won by a neck." and "He admired her long graceful neck."?': 'Yes',
'Does the word "belt" mean the same thing in sentences "Belt your trousers." and "The rotund man had difficulty belting his pants, and generally wore suspenders to avoid the issue."?': 'Yes',
'Does the word "nature" mean the same thing in sentences "Matters of a personal nature." and "The true nature of jealousy."?': 'No',
'Does the word "pedal" mean the same thing in sentences "The pedal of a loom." and "There are three pedals on manual cars, two on automatics."?': 'Yes',
'Does the word "cinema" mean the same thing in sentences "The cinema is right across the street from the restaurant." and "This story would be good cinema."?': 'No',
'Does the word "stampede" mean the same thing in sentences "She and her husband would join in the general stampede. -W. Black." and "When he shouted ` fire\' there was a stampede to the exits."?': 'No',
'Does the word "blind" mean the same thing in sentences "He spent hours reading to the blind." and "They had just moved in and had not put up blinds yet."?': 'No',
'Does the word "totality" mean the same thing in sentences "Appalled by the totality of the destruction." and "The totality of war and its consequences."?': 'No',
'Does the word "course" mean the same thing in sentences "The course had only nine holes." and "The course was less than a mile."?': 'Yes',
'Does the word "offing" mean the same thing in sentences "There was a wedding in the offing." and "There was a ship in the offing."?': 'No',
'Does the word "awareness" mean the same thing in sentences "The crash intruded on his awareness." and "I gradually passed from sleep to full awareness."?': 'Yes',
'Does the word "remove" mean the same thing in sentences "Remove a case to another court." and "He removed his children to the countryside."?': 'Yes',
'Does the word "snap" mean the same thing in sentences "If you bend it too much, it will snap." and "He snapped his stick in anger."?': 'Yes',
'Does the word "style" mean the same thing in sentences "Style the dress." and "Style my hair."?': 'Yes',
'Does the word "dignity" mean the same thing in sentences "It was beneath his dignity to cheat." and "Showed his true dignity when under pressure."?': 'Yes',
'Does the word "constellate" mean the same thing in sentences "The poets constellate in this town every summer." and "Hills constellated with lights."?': 'No',
'Does the word "roll" mean the same thing in sentences "To roll a wheel, a ball, or a barrel." and "The ball rolled."?': 'Yes',
'Does the word "footing" mean the same thing in sentences "On a friendly footing." and "They were on a friendly footing."?': 'No',
'Does the word "conjecture" mean the same thing in sentences "The physicist used his conjecture about subatomic particles to design an experiment." and "I explained it, but it is pure conjecture whether he understood, or not."?': 'No',
'Does the word "mark" mean the same thing in sentences "I filled the bottle up to the 500ml mark." and "His arrow hit the mark."?': 'No',
'Does the word "acknowledge" mean the same thing in sentences "It is important to acknowledge the work of others in one\'s own writing." and "The Crown Prince was acknowledged as the true heir to the throne."?': 'No',
'Does the word "fall" mean the same thing in sentences "When that became known the price of their stock went into free fall." and "In the fall of 1973."?': 'No',
'Does the word "pressure" mean the same thing in sentences "She has felt pressure lately because her boss expects her to get the job done by the first." and "The public brought pressure to bear on the government."?': 'No',
'Does the word "identity" mean the same thing in sentences "The identity under numerical multiplication is 1." and "She guessed the identity of his lover."?': 'No',
'Does the word "clear" mean the same thing in sentences "Clear a debt." and "Clear your desk."?': 'No',
'Does the word "tantrum" mean the same thing in sentences "She threw a tantrum." and "Many parents become embarrassed by their children throwing tantrums in public places."?': 'Yes',
'Does the word "theme" mean the same thing in sentences "The theme is announced in the first measures." and "It was the usual ` boy gets girl\' theme."?': 'No',
'Does the word "kettle" mean the same thing in sentences "There\'s a hot kettle of soup on the stove." and "To cook pasta, you first need to put the kettle on."?': 'Yes',
'Does the word "hedge" mean the same thing in sentences "To hedge a field or garden." and "Hedge the property."?': 'Yes',
'Does the word "river" mean the same thing in sentences "The river was navigable for 50 miles." and "Occasionally rivers overflow their banks and cause floods."?': 'Yes',
'Does the word "ingenuity" mean the same thing in sentences "Poverty is the mother of ingenuity." and "Ingenuity is one of the characteristics of a beaver."?': 'Yes',
'Does the word "suit" mean the same thing in sentences "In bridge you must follow suit." and "A flush is five cards in the same suit."?': 'Yes',
'Does the word "security" mean the same thing in sentences "We support the armed services in the name of national security." and "He held several valuable securities."?': 'No',
'Does the word "preferment" mean the same thing in sentences "Preferment of charges." and "The preferment went to the younger candidate."?': 'No',
'Does the word "share" mean the same thing in sentences "I\'d like to share this idea with you." and "We shared the cake."?': 'No',
'Does the word "follow" mean the same thing in sentences "These people still follow the laws of their ancient religion." and "She followed dinner with a brandy."?': 'No',
'Does the word "close" mean the same thing in sentences "Close a book." and "Close a wound."?': 'Yes',
'Does the word "override" mean the same thing in sentences "Health considerations override financial concerns." and "The Congress promptly overrode the president\'s veto, passing the bill into law."?': 'No',
'Does the word "bump" mean the same thing in sentences "Bump and grind." and "My car bumped into the tree."?': 'No',
'Does the word "nest" mean the same thing in sentences "A nest of thieves." and "A nest of snipers."?': 'No',
'Does the word "partition" mean the same thing in sentences "Partition a room off." and "The Arab peninsula was partitioned by the British."?': 'No',
'Does the word "degree" mean the same thing in sentences "A second degree burn." and "Murder in the second degree."?': 'Yes',
'Does the word "establishment" mean the same thing in sentences "He quickly became recognized as a member of the establishment." and "The firm celebrated twenty years since their establishment by updating their look."?': 'Yes',
'Does the word "shock" mean the same thing in sentences "Subjects received a small electric shock when they made the wrong response." and "Corn is bound in small sheaves and several sheaves are set up together in shocks."?': 'No',
'Does the word "march" mean the same thing in sentences "The march of time." and "The march of science."?': 'Yes',
'Does the word "derivative" mean the same thing in sentences "The derivative of f : f(x ) = x^2 is f\':f\'(x ) = 2x." and "` electricity\' is a derivative of ` electric\'."?': 'No',
'Does the word "reproach" mean the same thing in sentences "Words of reproach." and "He brought reproach upon his family."?': 'No',
'Does the word "fortune" mean the same thing in sentences "It was as if fortune guided his hand." and "Whatever my fortune may be."?': 'No',
'Does the word "cascade" mean the same thing in sentences "Separation of isotopes by a cascade of processes." and "Progressing in severity as though a cascade of genetic damage was occurring."?': 'Yes',
'Does the word "enterprise" mean the same thing in sentences "A micro enterprise is defined as a business having 5 or fewer employees and a low seed capital." and "A growing enterprise must have a bold leader."?': 'Yes',
'Does the word "entertain" mean the same thing in sentences "Entertain interesting notions." and "He entertained the notion of moving to South America."?': 'No',
'Does the word "exile" mean the same thing in sentences "Men in exile dream of hope." and "They chose exile rather than assimilation."?': 'Yes',
'Does the word "union" mean the same thing in sentences "There is strength in union." and "Lightning produced an unusual union of the metals."?': 'No',
'Does the word "topic" mean the same thing in sentences "It was a very sensitive topic." and "He kept drifting off the topic."?': 'No',
'Does the word "reading" mean the same thing in sentences "Suggestions for further reading." and "His main reading was detective stories."?': 'Yes',
'Does the word "comply" mean the same thing in sentences "You must comply or else !" and "He complied with my instructions."?': 'Yes',
'Does the word "immunity" mean the same thing in sentences "The prosecutor offered the lieutenant immunity for all the crimes he would testify having known to be planned by the elusive drug baron." and "He was granted immunity from prosecution."?': 'Yes',
'Does the word "cloud" mean the same thing in sentences "He opened the door and was greeted by a cloud of bats." and "Clouds of blossoms."?': 'Yes',
'Does the word "marshal" mean the same thing in sentences "Marshal the troops." and "Marshal facts or arguments."?': 'No',
'Does the word "smokescreen" mean the same thing in sentences "All that talk is just a smokescreen to disguise the fact that he has nothing to say." and "Requesting new powers of surveillance is just a smokescreen to hide their failures."?': 'Yes',
'Does the word "color" mean the same thing in sentences "She used a different color for the trim." and "Color has been a sensitive issue in many societies."?': 'No',
'Does the word "channel" mean the same thing in sentences "Poison is released through a channel in the snake\'s fangs." and "Gutters carried off the rainwater into a series of channels under the street."?': 'No',
'Does the word "climax" mean the same thing in sentences "The deathbed scene is the climax of the play." and "The climax of the artist\'s career."?': 'No',
'Does the word "marry" mean the same thing in sentences "A justice of the peace will marry Jones and Smith." and "The minister married us on Saturday."?': 'Yes',
'Does the word "penetrate" mean the same thing in sentences "The hikers did not manage to penetrate the dense forest." and "She was penetrated with sorrow."?': 'No',
'Does the word "darkness" mean the same thing in sentences "He moved off into the darkness." and "His lectures dispelled the darkness."?': 'No',
'Does the word "legitimation" mean the same thing in sentences "He has filial rights because he obtained letters of legitimation from the king." and "His parents\' subsequent marriage resulted in his legitimation."?': 'Yes',
'Does the word "sex" mean the same thing in sentences "He wanted a better sex life." and "The film contained no sex or violence."?': 'Yes',
'Does the word "theology" mean the same thing in sentences "Jewish theology." and "He studied theology at Oxford."?': 'No',
'Does the word "production" mean the same thing in sentences "They export most of their agricultural production." and "The widget making machine is being used for production now."?': 'No',
'Does the word "void" mean the same thing in sentences "Void a plea." and "He voided the check and returned it."?': 'Yes',
'Does the word "question" mean the same thing in sentences "There is no question about the validity of the enterprise." and "There was a question about my training."?': 'No',
'Does the word "fault" mean the same thing in sentences "The fault lies with you." and "It was John\'s fault."?': 'Yes',
'Does the word "rounding" mean the same thing in sentences "Taxes are rounded off to the nearest dollar but the rounding error is surprisingly small." and "The error in the calculation was attributable to rounding."?': 'Yes',
'Does the word "devotee" mean the same thing in sentences "A devotee of classical music." and "He was a devotee of Arnold Schwarzenegger."?': 'Yes',
'Does the word "cancel" mean the same thing in sentences "Cancel cheques or tickets." and "The corrective feedback mechanism cancels out the noise."?': 'No',
'Does the word "captain" mean the same thing in sentences "A captain of industry." and "John Henry said to the captain, " A man ai n\'t nothing but a man. "."?': 'Yes',
'Does the word "signage" mean the same thing in sentences "The signage in the airport is designed to point the way clearly to important locations." and "The poor signage at Heathrow is only in English."?': 'Yes',
'Does the word "despair" mean the same thing in sentences "He turned around in despair, aware that he was not going to survive." and "One harsh word would send her into the depths of despair."?': 'No',
'Does the word "air" mean the same thing in sentences "If you\'ve time to spare go by air." and "Air travel involves too much waiting in airports."?': 'Yes',
'Does the word "excursion" mean the same thing in sentences "While driving home I took an excursion and saw some deer." and "Many summer excursions to the shore."?': 'Yes',
'Does the word "scan" mean the same thing in sentences "You could see the tumor in the CAT scan." and "He made a thorough scan of the beach with his binoculars."?': 'No',
'Does the word "distress" mean the same thing in sentences "Originally distress was a landlord\'s remedy against a tenant for unpaid rents or property damage but now the landlord is given a landlord\'s lien." and "The patient appeared to be in distress."?': 'No',
'Does the word "suppress" mean the same thing in sentences "The rescue team managed to suppress the flow of oil by blasting the drilling hole." and "This drug can suppress the hemorrhage."?': 'Yes',
'Does the word "leakiness" mean the same thing in sentences "The leakiness of the roof." and "The leakiness of the boat made it dangerous to use."?': 'Yes',
'Does the word "support" mean the same thing in sentences "I support France in the World Cup." and "She supported him during the illness."?': 'Yes',
'Does the word "meter" mean the same thing in sentences "Meter the flow of water." and "Meter the mail."?': 'No',
'Does the word "popularity" mean the same thing in sentences "This gallant laboring to avoid popularity falls into a habit of affectation. — Ben Jonson." and "His charm soon won him affection and popularity."?': 'Yes',
'Does the word "face" mean the same thing in sentences "An angry face." and "When he returned to work he met many new faces."?': 'No',
'Does the word "upset" mean the same thing in sentences "The biggest upset since David beat Goliath." and "Everyone gets stomach upsets from time to time."?': 'No',
'Does the word "shopping" mean the same thing in sentences "Does her shopping at the mall rather than down town." and "Went shopping for a reliable plumber."?': 'Yes',
'Does the word "wear" mean the same thing in sentences "The tires showed uneven wear." and "Maternity wear."?': 'No',
'Does the word "substitute" mean the same thing in sentences "They tend to substitute for more mundane things." and "I had no shallots so I substituted onion."?': 'Yes',
'Does the word "occasion" mean the same thing in sentences "At this point, she seized the occasion to make her own observation." and "On the occasion of his 60th birthday."?': 'Yes',
'Does the word "gutter" mean the same thing in sentences "Gutter the buildings." and "The cooling lava continued to gutter toward lower ground."?': 'No',
'Does the word "homer" mean the same thing in sentences "The first baseman hit a homer to lead off the ninth." and "Each of the pigeon fanciers released a homer at the same time."?': 'No',
'Does the word "break" mean the same thing in sentences "Break a record." and "He broke the men\'s 100-meter record."?': 'Yes',
'Does the word "suit" mean the same thing in sentences "The family brought suit against the landlord." and "If you take my advice, you\'ll file suit against him immediately."?': 'Yes',
'Does the word "pretension" mean the same thing in sentences "His pretension to the crown." and "The town still puts forward pretensions as a famous resort."?': 'Yes',
'Does the word "scream" mean the same thing in sentences "He heard the scream of the brakes." and "Her screaming attracted the neighbors."?': 'No',
'Does the word "taker" mean the same thing in sentences "She is known as quite a risk taker." and "The study could not confirm the real percentage of drug takers in the country."?': 'Yes',
'Does the word "corner" mean the same thing in sentences "Standing on the corner watching all the girls go by." and "The liquor store on the corner also sold lottery tickets."?': 'Yes',
'Does the word "zero" mean the same thing in sentences "The zero sign in American Sign Language is considered rude in some cultures." and "The probability is 1 in 1,000,000,000,000,000 — a number having five ciphers of zeros."?': 'Yes',
'Does the word "roominess" mean the same thing in sentences "His unselfishness gave him great intellectual roominess." and "Roominess in this size car is always a compromise."?': 'No',
'Does the word "jump" mean the same thing in sentences "The parachutist did n\'t want to jump." and "Every year, hundreds of people jump off the Golden Gate bridge."?': 'Yes',
'Does the word "drive" mean the same thing in sentences "Can you drive this four-wheel truck ?" and "He drives me mad."?': 'No',
'Does the word "consist" mean the same thing in sentences "What does love consist in ?" and "The payment consists in food."?': 'Yes',
'Does the word "bleed" mean the same thing in sentences "Bleed the radiators." and "I bled him."?': 'No',
'Does the word "opinion" mean the same thing in sentences "Truth, in matters of religion, is simply the opinion that has survived. - Oscar Wilde." and "My opinion differs from yours."?': 'Yes',
'Does the word "blood" mean the same thing in sentences "The ancients believed that blood was the seat of the emotions." and "Blood carries oxygen and nutrients to the tissues and carries away waste products."?': 'Yes',
'Does the word "satisfaction" mean the same thing in sentences "The satisfaction of their demand for better services." and "The chef tasted the sauce with great satisfaction."?': 'No',
'Does the word "adjoin" mean the same thing in sentences "The living room and dining room adjoin each other." and "I adjoin a copy of your my lawyer\'s letter."?': 'No',
'Does the word "element" mean the same thing in sentences "Water is the element of fishes." and "To be in one\'s own element."?': 'No',
'Does the word "bandwagon" mean the same thing in sentences "The gaudy bandwagon led the circus parade." and "When they saw how things were going everybody jumped on the bandwagon."?': 'No',
'Does the word "setter" mean the same thing in sentences "She has a spaniel and a red setter." and "Some crossword setters work for various newspapers under different pseudonyms."?': 'No',
'Does the word "palpitate" mean the same thing in sentences "When he just looks at me, my heart begins to palpitate with excitement." and "His heart palpitated."?': 'Yes',
'Does the word "cry" mean the same thing in sentences "A cry of rage." and "Animal cries filled the night."?': 'No',
'Does the word "roll" mean the same thing in sentences "Roll a cigarette." and "They rolled their eyes at his words."?': 'No',
'Does the word "wade" mean the same thing in sentences "Can we wade across the river to the other side ?" and "Wade the pond."?': 'Yes',
'Does the word "snap" mean the same thing in sentences "He can hear the snap of a twig." and "A cold snap in the middle of May."?': 'No',
'Does the word "haul" mean the same thing in sentences "Haul vegetables to the market." and "Haul stones from the quarry in a truck."?': 'Yes',
'Does the word "keep" mean the same thing in sentences "Keep boarders." and "He is keeping three women in the guest cottage."?': 'Yes',
'Does the word "running" mean the same thing in sentences "He has responsibility for the running of two companies at the same time." and "The running of the engine is smooth."?': 'No',
'Does the word "ingenuity" mean the same thing in sentences "A plot of great ingenuity." and "The pyramids demonstrate the ingenuity of the ancient Egyptians."?': 'No',
'Does the word "tighten" mean the same thing in sentences "Please tighten that screw a quarter-turn." and "Tighten the wire."?': 'Yes',
'Does the word "size" mean the same thing in sentences "Size gives body to a fabric." and "The size of the building seemed to have increased since I was last there."?': 'No',
'Does the word "bombshell" mean the same thing in sentences "She was a blonde bombshell." and "News of the attack came like a bombshell."?': 'No',
'Does the word "stay" mean the same thing in sentences "Some of her friends decided it was time to go home, but she wanted to stay on until sunrise." and "To stay a flat sheet in a steam boiler."?': 'No',
'Does the word "miss" mean the same thing in sentences "I miss you ! Come home soon !" and "He misses his mother."?': 'Yes',
'Does the word "press" mean the same thing in sentences "The press of business matters." and "Put the ironing in the linen press."?': 'No',
'Does the word "dash" mean the same thing in sentences "He is preparing for the 100-yard dash." and "He wooed her with the confident dash of a cavalry officer."?': 'No',
'Does the word "theory" mean the same thing in sentences "They killed him on the theory that dead men tell no tales." and "The architect has a theory that more is less."?': 'Yes',
'Does the word "need" mean the same thing in sentences "She satisfied his need for affection." and "God has no need of men to accomplish His work."?': 'Yes',
'Does the word "trade" mean the same thing in sentences "Trade in an old car for a new one." and "The bells traded places."?': 'Yes',
'Does the word "rendition" mean the same thing in sentences "Often imitations are extended to provide a more accurate rendition of the child\'s intended meaning." and "They heard a live rendition of three pieces by Schubert."?': 'No',
'Does the word "celebrate" mean the same thing in sentences "Celebrate Christmas." and "To celebrate a birthday."?': 'Yes',
'Does the word "account" mean the same thing in sentences "Do n\'t do it on my account." and "The paper was rejected on account of its length."?': 'Yes',
'Does the word "answer" mean the same thing in sentences "His answer to any problem was to get drunk." and "Their answer was to sue me."?': 'Yes',
'Does the word "movement" mean the same thing in sentences "Politicians have to respect a mass movement." and "The second movement is slow and melodic."?': 'No',
'Does the word "impulse" mean the same thing in sentences "He bought it on an impulse." and "Profound religious impulses."?': 'No',
'Does the word "pause" mean the same thing in sentences "We pause for station identification." and "When telling the scary story, he paused for effect."?': 'No',
'Does the word "intend" mean the same thing in sentences "What do his words intend ?" and "She was intended to become the director."?': 'No',
'Does the word "efflux" mean the same thing in sentences "We all age through the efflux of time." and "The efflux of matter from a boil can be painful."?': 'Yes',
'Does the word "devour" mean the same thing in sentences "She intended to devour the book." and "She devoured his novels."?': 'Yes',
'Does the word "underlay" mean the same thing in sentences "To underlay a cut, plate, or the like, for printing." and "Underlay the plate."?': 'No',
'Does the word "retie" mean the same thing in sentences "Retie the string and make it strong enough now." and "She stopped walking for a moment to retie her shoe."?': 'Yes',
'Does the word "retroversion" mean the same thing in sentences "The teacher translated Latin texts into English which he gave to his students for retroversion." and "Retroversion of the uterus."?': 'No',
'Does the word "drive" mean the same thing in sentences "Drive a car or bus." and "Drive a car."?': 'Yes',
'Does the word "truncate" mean the same thing in sentences "Truncate a series." and "Truncate a word."?': 'No',
'Does the word "conserve" mean the same thing in sentences "Children must be taught to conserve our national heritage." and "Energy is conserved in this process."?': 'No',
'Does the word "manifestation" mean the same thing in sentences "A manifestation of disease." and "A manifestation of great emotion."?': 'No',
'Does the word "major" mean the same thing in sentences "She is a math major." and "She is a linguistics major."?': 'Yes',
'Does the word "pocketbook" mean the same thing in sentences "That car is too expensive for my pocketbook." and "The publishers brought out small format pocketbooks of the whole of their nature series\'."?': 'No',
'Does the word "site" mean the same thing in sentences "The Israeli web site was damaged by hostile hackers." and "A good site for the school."?': 'No',
'Does the word "folly" mean the same thing in sentences "Trying to drive through a blizzard is the height of folly." and "This is a war of folly."?': 'Yes',
'Does the word "deliver" mean the same thing in sentences "Render a verdict ( i.e., deliver a judgment )." and "Deliver a captive from the prison."?': 'No',
'Does the word "heating" mean the same thing in sentences "They have radiant heating." and "The heating system was n\'t working."?': 'Yes',
'Does the word "chemistry" mean the same thing in sentences "The chemistry of soil." and "Their chemistry was wrong from the beginning -- they hated each other."?': 'No',
'Does the word "bank" mean the same thing in sentences "He sat on the bank of the river and watched the currents." and "That bank holds the mortgage on my home."?': 'No',
'Does the word "spin" mean the same thing in sentences "Spin silk." and "Spiders spin a fine web."?': 'No',
'Does the word "blur" mean the same thing in sentences "To blur a photograph by moving the camera while taking it." and "The fog blurs my vision."?': 'Yes',
'Does the word "martini" mean the same thing in sentences "My boss is out on another of his three martini lunches." and "Will you join us at six o\'clock for martinis ?"?': 'Yes',
'Does the word "consumption" mean the same thing in sentences "The consumption of energy has increased steadily." and "The fire\'s consumption of the forest caused ecological changes."?': 'No',
'Does the word "recognition" mean the same thing in sentences "Experimental psychologists measure the elapsed time from the onset of the stimulus to its recognition by the observer." and "A politician whose recall of names was as remarkable as his recognition of faces."?': 'Yes',
'Does the word "assume" mean the same thing in sentences "When will the new President assume office ?" and "He assumed to himself the right to fill all positions in the town."?': 'No',
'Does the word "stretch" mean the same thing in sentences "Stretch out that piece of cloth." and "Stretch the limits."?': 'No',
'Does the word "violence" mean the same thing in sentences "The storm\'s violence." and "The violence of the storm, fortunately, was more awesome than destructive."?': 'Yes',
'Does the word "nose" mean the same thing in sentences "She has a cold in the nose." and "Keep your nose out of it."?': 'No',
'Does the word "distance" mean the same thing in sentences "The distance to Petersborough is thirty miles." and "There is a long distance between Moscow and Vladivostok."?': 'Yes',
'Does the word "reserve" mean the same thing in sentences "We managed to reserve a table at Maxim\'s." and "Reserve me a seat on a flight."?': 'No',
'Does the word "character" mean the same thing in sentences "The radical character of our demands." and "A real character."?': 'No',
'Does the word "habit" mean the same thing in sentences "She had a habit twirling the ends of her hair." and "Owls have nocturnal habits."?': 'Yes',
'Does the word "flight" mean the same thing in sentences "Take flight." and "Birds are capable of flight."?': 'No',
'Does the word "coolant" mean the same thing in sentences "He added more coolant to the car\'s radiator." and "The atomic reactor used a gas coolant."?': 'Yes',
'Does the word "beat" mean the same thing in sentences "Beat the drum." and "I got beaten up by thugs on my way home."?': 'No',
'Does the word "bravura" mean the same thing in sentences "In a final bravura the ballerina appeared to be floating in water." and "The music ends with a display of bravura."?': 'Yes',
'Does the word "type" mean the same thing in sentences "The capable type." and "The word ` element\' contains five different types of character."?': 'No',
'Does the word "station" mean the same thing in sentences "Married above her station." and "The responsibilities of a man in his station."?': 'Yes',
'Does the word "die" mean the same thing in sentences "My secret will die with me !" and "Their anger died."?': 'Yes',
'Does the word "sound" mean the same thing in sentences "Sound the bell." and "Sound a certain note."?': 'Yes',
'Does the word "engage" mean the same thing in sentences "The teeth of one cogwheel engage those of another." and "Make sure the gear is engaged."?': 'Yes',
'Does the word "yoke" mean the same thing in sentences "Under the yoke of a tyrant." and "They threw off the yoke of domination."?': 'Yes',
'Does the word "excavate" mean the same thing in sentences "Excavate gold." and "Carnegie had a lake excavated for Princeton University\'s rowing team."?': 'No',
'Does the word "hearing" mean the same thing in sentences "They make good music -- you should give them a hearing." and "They condemned him without a hearing."?': 'No',
'Does the word "complexion" mean the same thing in sentences "A rugged complexion." and "Diphthongs are complexions of vowels."?': 'No',
'Does the word "rein" mean the same thing in sentences "Rein a horse." and "He reined in his horses in front of the post office."?': 'No',
'Does the word "stretch" mean the same thing in sentences "Beyond any stretch of his understanding." and "A stretch of clear water."?': 'No',
'Does the word "vagina" mean the same thing in sentences "The vagina receives the penis during coitus." and "The vagina is elastic enough to allow the passage of a fetus."?': 'Yes',
'Does the word "unclothe" mean the same thing in sentences "Unclothe your heart of envy." and "She unclothed her innermost feelings."?': 'No',
'Does the word "tiger" mean the same thing in sentences "He\'s a tiger on the tennis court." and "It aroused the tiger in me."?': 'Yes',
'Does the word "population" mean the same thing in sentences "The population of New Jersey will not stand for this !" and "They hired hunters to keep down the deer population."?': 'Yes',
'Does the word "smoke" mean the same thing in sentences "Do you smoke ?" and "He\'s smoking his pipe."?': 'Yes',
'Does the word "spark" mean the same thing in sentences "A spark of interest." and "A spark of decency."?': 'Yes',
'Does the word "notice" mean the same thing in sentences "We received a notice to vacate the premises." and "He escaped the notice of the police."?': 'No',
'Does the word "clean" mean the same thing in sentences "Clean your fingernails before dinner." and "Clean up before you see your grandparents."?': 'Yes',
'Does the word "population" mean the same thing in sentences "He deplored the population of colonies with convicted criminals." and "It is an estimate of the mean of the population."?': 'No',
'Does the word "lot" mean the same thing in sentences "They were an angry lot." and "A lot of money."?': 'No',
'Does the word "close" mean the same thing in sentences "Close the circuit." and "Close up an umbrella."?': 'Yes',
'Does the word "concord" mean the same thing in sentences "Both philosophers concord on this point." and "Their ideas concorded."?': 'No',
'Does the word "plant" mean the same thing in sentences "Plant fish." and "Plant a thought in the students\' minds."?': 'No',
'Does the word "wind" mean the same thing in sentences "When there is no wind, row." and "He put the key in the old clock and gave it a good wind."?': 'No',
'Does the word "plus" mean the same thing in sentences "He is a real plus to the team." and "A water molecule is made up of two hydrogen atoms plus one of oxygen."?': 'No',
'Does the word "creation" mean the same thing in sentences "From its creation the plan was doomed to failure." and "The restructure resulted in the creation of a number of shared services."?': 'No',
'Does the word "association" mean the same thing in sentences "The host is not always injured by association with a parasite." and "His association of his father with being beaten was too strong to break."?': 'No',
'Does the word "evacuate" mean the same thing in sentences "Evacuate the bottle." and "After the earthquake, residents were evacuated."?': 'No',
'Does the word "park" mean the same thing in sentences "There are laws that protect the wildlife in this park." and "The put the car in park and got out."?': 'No',
'Does the word "grip" mean the same thing in sentences "He was in the grip of a powerful emotion." and "A good grip on French history."?': 'Yes',
'Does the word "despair" mean the same thing in sentences "They were rescued from despair at the last minute." and "In the depths of despair."?': 'Yes',
'Does the word "grow" mean the same thing in sentences "We grow wheat here." and "The child grew fast."?': 'No',
'Does the word "man" mean the same thing in sentences "He awaited word from his man in Havana." and "He taught me to set up the men on the chess board."?': 'No',
'Does the word "map" mean the same thing in sentences "We have n\'t even begun to map the many galaxies that we know exist." and "Sorrow was mapped on the mother\'s face."?': 'No',
'Does the word "perceive" mean the same thing in sentences "I could perceive the ship coming over the horizon." and "She finally perceived the futility of her protest."?': 'No',
'Does the word "people" mean the same thing in sentences "Old people." and "There were at least 200 people in the audience."?': 'Yes',
'Does the word "performance" mean the same thing in sentences "We congratulated him on his performance at the rehearsal." and "An inspired performance of Mozart\'s C minor concerto."?': 'Yes',
'Does the word "definition" mean the same thing in sentences "The definition of a telescope." and "Exercise had given his muscles superior definition."?': 'Yes',
'Does the word "pleasure" mean the same thing in sentences "The pleasure of his company." and "He was tingling with pleasure."?': 'No',
'Does the word "refer" mean the same thing in sentences "Refer a bill to a committee." and "Refer to your notes."?': 'No',
'Does the word "voice" mean the same thing in sentences "After the fire a still small voice. — 1 Kings 19:12." and "A singer takes good care of his voice."?': 'Yes',
'Does the word "leviathan" mean the same thing in sentences "It was a leviathan among redwoods." and "They were assigned the leviathan of textbooks."?': 'Yes',
'Does the word "endorse" mean the same thing in sentences "Endorse cheques." and "Endorse a new project."?': 'No',
'Does the word "security" mean the same thing in sentences "The watch dog gave her a feeling of security." and "Bankers are reluctant to lend without good security."?': 'No',
'Does the word "grace" mean the same thing in sentences "The Virgin lived in a state of grace." and "It was debated whether saving grace could be obtained outside the membership of the church."?': 'Yes',
'Does the word "hiccup" mean the same thing in sentences "There was a loud hiccup from the back of the room and the class erupted in laughter." and "He has the hiccups - give him a glass of water."?': 'Yes',
'Does the word "deliverable" mean the same thing in sentences "We packaged the deliverable, a program called FLOOD.EXE, in an installer file." and "Under this contract the deliverables include both software and hardware."?': 'Yes',
'Does the word "chance" mean the same thing in sentences "We ran into each other by pure chance." and "There is a 30 % chance of rain tomorrow."?': 'No',
'Does the word "surrender" mean the same thing in sentences "The squatters had to surrender the building after the police moved in." and "The last Taleban fighters finally surrendered."?': 'No',
'Does the word "twin" mean the same thing in sentences "Twin crystals." and "The polished surface twinned his face and chest in reverse."?': 'No',
'Does the word "submit" mean the same thing in sentences "I submit these plans for your approval." and "The lawyers submitted the material to the court."?': 'Yes',
'Does the word "selection" mean the same thing in sentences "The store carried a large selection of shoes." and "I\'ve brought a selection of fine cheeses to go with your wine."?': 'No',
'Does the word "patent" mean the same thing in sentences "Should I patent this invention ?" and "I patented my idea."?': 'Yes',
'Does the word "enjoy" mean the same thing in sentences "Enjoy your holidays !" and "The industry enjoyed a boom."?': 'No',
'Does the word "overstate" mean the same thing in sentences "He was suggested not to overstate at the interview." and "I think it is overstating matters to say that an hour online is spending all night on the computer."?': 'Yes',
'Does the word "salute" mean the same thing in sentences "When the officers show up, the soldiers have to salute." and "A terrible stench saluted our nostrils."?': 'No',
'Does the word "execute" mean the same thing in sentences "To execute a turn in ballet." and "The President executed the treaty."?': 'No',
'Does the word "end" mean the same thing in sentences "The phone rang at the other end." and "Both ends wrote at the same time."?': 'Yes',
'Does the word "principle" mean the same thing in sentences "A man of principle." and "He will not violate his principles."?': 'Yes',
'Does the word "baste" mean the same thing in sentences "Baste a roast." and "Baste a hem."?': 'No',
'Does the word "calve" mean the same thing in sentences "The whales calve at this time of year." and "The icebergs and glaciers calve."?': 'No',
'Does the word "construction" mean the same thing in sentences "She wore her hair in an amazing construction of whirls and ribbons." and "During the construction we had to take a detour."?': 'No',
'Does the word "fill" mean the same thing in sentences "The bucket filled with rain." and "Fill the child with pride."?': 'Yes',
'Does the word "replay" mean the same thing in sentences "Replay a point." and "We replayed the game."?': 'Yes',
'Does the word "fence" mean the same thing in sentences "If we fence in that field, it will be a good pasture for a horse." and "We fenced in our yard."?': 'Yes',
'Does the word "efficiency" mean the same thing in sentences "The efficiency of the planning department is deplorable." and "She did the work with great efficiency."?': 'Yes',
'Does the word "radiate" mean the same thing in sentences "This plants radiate spines in all directions." and "Spokes radiate from the hub of the wheel."?': 'Yes',
'Does the word "back" mean the same thing in sentences "To back the oars." and "The car backed up and hit the tree."?': 'Yes',
'Does the word "seize" mean the same thing in sentences "To seize a ship after libeling." and "The movie seized my imagination."?': 'No',
'Does the word "test" mean the same thing in sentences "The test was standardized on a large sample of students." and "When the test was stolen the professor had to make a new set of questions."?': 'No',
'Does the word "steamroller" mean the same thing in sentences "Steamroller the road." and "The Senator steamrollered the bill to defeat."?': 'No',
'Does the word "produce" mean the same thing in sentences "The tree would not produce fruit." and "Produce a new play."?': 'No',
'Does the word "leg" mean the same thing in sentences "After six days, we\'re finally in the last leg of our cross-country trip." and "Dan wo n\'t be able to come to the party, since he broke his leg last week and is now on crutches."?': 'No',
'Does the word "attest" mean the same thing in sentences "When will the appraiser attest the date of the painting ?" and "I attest this signature."?': 'Yes',
'Does the word "attention" mean the same thing in sentences "She was the center of attention." and "The old car needs constant attention."?': 'No',
'Does the word "richness" mean the same thing in sentences "They studied forerunners of richness or poverty." and "The cut of her clothes and the richness of the fabric were distinctive."?': 'No',
'Does the word "kid" mean the same thing in sentences "Kid gloves." and "They were able to send their kids to college."?': 'No',
'Does the word "purse" mean the same thing in sentences "He and his wife shared a common purse." and "He made the contribution out of his own purse."?': 'Yes',
'Does the word "park" mean the same thing in sentences "Park the children with the in-laws." and "Can you park right here ?"?': 'No',
'Does the word "month" mean the same thing in sentences "July is my favourite month." and "He paid the bill last month."?': 'Yes',
'Does the word "rotation" mean the same thing in sentences "Crop rotation." and "The rotation of the dancer kept time with the music."?': 'No',
'Does the word "turnout" mean the same thing in sentences "A large turnout for the meeting." and "The turnout for the rally."?': 'No',
'Does the word "silence" mean the same thing in sentences "He needed silence in order to sleep." and "When the motor stopped, the silence was almost deafening."?': 'Yes',
'Does the word "shade" mean the same thing in sentences "As the sun moved he readjusted the shade." and "They used umbrellas as shades."?': 'Yes',
'Does the word "patient" mean the same thing in sentences "The subject of a passive verb is usually a patient." and "The number of emergency patients has grown rapidly."?': 'No',
'Does the word "probe" mean the same thing in sentences "Probe an anthill." and "If you probe further, you may discover different reasons."?': 'No',
'Does the word "effect" mean the same thing in sentences "The magnetic effect was greater when the rod was lengthwise." and "The effect of the hurricane was a devastated landscape."?': 'Yes',
'Does the word "add" mean the same thing in sentences "To add numbers." and "This adds a light note to the program."?': 'No',
'Does the word "apply" mean the same thing in sentences "The same laws apply to you !" and "He applied this racial slur to me !"?': 'No',
'Does the word "local" mean the same thing in sentences "The expresses skipped my station, so I had to take a local." and "The local seemed to take forever to get to New York."?': 'Yes',
'Does the word "blow" mean the same thing in sentences "Blow a horse." and "To blow a horse."?': 'Yes',
'Does the word "back" mean the same thing in sentences "The fall broke his back." and "The titles are printed on the backs of the books."?': 'No',
'Does the word "swarm" mean the same thing in sentences "There swarm bees in the garden." and "The garden was swarming with bees."?': 'Yes',
'Does the word "truth" mean the same thing in sentences "The situation brought home to us the blunt truth of the military threat." and "He was famous for the truth of his portraits."?': 'Yes',
'Does the word "heat" mean the same thing in sentences "I\'ll heat up the water." and "The massage heated her up."?': 'No',
'Does the word "progress" mean the same thing in sentences "Great progress in the arts." and "Science has made extraordinary progress in the last fifty years."?': 'Yes',
'Does the word "rascal" mean the same thing in sentences "That little rascal bit me !" and "If you have deer in the area, you may have to put a fence around your garden to keep the rascals out."?': 'Yes',
'Does the word "party" mean the same thing in sentences "The contract requires that the party of the first part pay the fee." and "I\'m throwing a huge party for my 21st birthday."?': 'No',
'Does the word "affinity" mean the same thing in sentences "The mysterious affinity between them." and "In anatomical structure prehistoric man shows close affinity with modern humans."?': 'No',
'Does the word "model" mean the same thing in sentences "Model a head with clay." and "The sculptor modelled the clay into the form of a dolphin."?': 'Yes',
'Does the word "suppress" mean the same thing in sentences "Suppress a yawn." and "Hot blackcurrant juice mixed with honey may suppress cough."?': 'Yes',
'Does the word "give" mean the same thing in sentences "Give a divorce." and "Give one\'s talents to a good cause."?': 'No',
'Does the word "defensibility" mean the same thing in sentences "Client complaints create a felt need for the defensibility of individual actions." and "They built their castles with an eye to their defensibility."?': 'Yes',
'Does the word "afterthought" mean the same thing in sentences "The garage was an afterthought." and "This whole remodeling was an afterthought, we were only going to paint and call it good."?': 'Yes',
'Does the word "wave" mean the same thing in sentences "A wave of emotion swept over him." and "A heat wave."?': 'No',
'Does the word "take" mean the same thing in sentences "The take must be repeated." and "Act seven, scene three, take two."?': 'Yes',
'Does the word "rail" mean the same thing in sentences "Rail off the crowds from the Presidential palace." and "She railed against the bad social policies."?': 'No',
'Does the word "begin" mean the same thing in sentences "You can not even begin to understand the problem we had to deal with during the war." and "This economic measure does n\'t even begin to deal with the problem of inflation."?': 'Yes',
'Does the word "exploit" mean the same thing in sentences "We must exploit the resources we are given wisely." and "He is exploiting the students."?': 'No',
'Does the word "bitch" mean the same thing in sentences "She said her son thought Hillary was a bitch." and "Ann gossiped about me and mocked my work ; sometimes she can be a real bitch !"?': 'Yes',
'Does the word "frost" mean the same thing in sentences "Frost the glass." and "She frosts her hair."?': 'Yes',
'Does the word "kindling" mean the same thing in sentences "Go and collect some kindling." and "The kindlings of love."?': 'No',
'Does the word "treat" mean the same thing in sentences "Treat the water so it can be drunk." and "Treat him with caution, please."?': 'No',
'Does the word "screen" mean the same thing in sentences "Screen the job applicants." and "Screen the suitcases."?': 'No',
'Does the word "magnitude" mean the same thing in sentences "About the magnitude of a small pea." and "They tried to predict the magnitude of the explosion."?': 'Yes',
'Does the word "hand" mean the same thing in sentences "Hand me the spoon, please." and "He handed them the letter."?': 'Yes',
'Does the word "poor" mean the same thing in sentences "The urban poor need assistance." and "The poor are always with us."?': 'Yes',
'Does the word "mistake" mean the same thing in sentences "He made a bad mistake." and "Make no mistake about his intentions."?': 'No',
'Does the word "sleep" mean the same thing in sentences "We need to conduct an overnight sleep test to diagnose your sleep problem." and "He did n\'t get enough sleep last night."?': 'Yes',
'Does the word "impose" mean the same thing in sentences "I can not impose that I get what I want." and "I can not impose my opinion."?': 'Yes',
'Does the word "roughness" mean the same thing in sentences "The roughness of her voice was a signal to keep quiet." and "Men associate the roughness of nonstandard working-class speech with masculinity."?': 'Yes',
'Does the word "feedstock" mean the same thing in sentences "The filtered extract of the mash is the feedstock for the fermenter." and "Bio-materials may be substituted for petrochemical feedstocks as petroleum prices rise."?': 'Yes',
'Does the word "round" mean the same thing in sentences "We enjoyed our round of the local bars." and "Hospital rounds."?': 'No',
'Does the word "deuce" mean the same thing in sentences "Love is a bodily infirmity... which breaks out the deuce knows how or why ( Thackeray )." and "The deuce with it."?': 'Yes',
'Does the word "lodge" mean the same thing in sentences "Lodge a bullet in the table." and "We are lodging three foreign students this semester."?': 'No',
'Does the word "pillar" mean the same thing in sentences "A pillar of smoke." and "A thin pillar of smoke betrayed their campsite."?': 'No',
'Does the word "pluck" mean the same thing in sentences "Pluck the flowers off the bush." and "She plucked the phone from her bag and dialled."?': 'Yes',
'Does the word "tense" mean the same thing in sentences "Alternately relax and tense your calf muscle." and "Tense the rope manually before tensing the spring."?': 'Yes',
'Does the word "inexperience" mean the same thing in sentences "I failed to get the job due to inexperience." and "Procedural inexperience created difficulties."?': 'Yes',
'Does the word "show" mean the same thing in sentences "All he had to show for four years of attendance at college was a framed piece of paper." and "She shows her dogs frequently."?': 'Yes',
'Does the word "narcolepsy" mean the same thing in sentences "He believes that narcolepsy is attributable to an inability to suppress REM sleep during waking." and "Excitement induced narcolepsy caused him to sleep through the most important events of his life."?': 'Yes',
'Does the word "cast" mean the same thing in sentences "Cast lots." and "Cast a spell."?': 'No',
'Does the word "deck" mean the same thing in sentences "Wow, did you see her deck that guy who pinched her ?" and "Deck the halls with holly."?': 'No',
'Does the word "heartbeat" mean the same thing in sentences "New York is the commercial heartbeat of America." and "Music is the heartbeat of the people."?': 'Yes',
'Does the word "cradle" mean the same thing in sentences "He was taught from the cradle never to cry." and "The cradle of liberty."?': 'No',
'Does the word "path" mean the same thing in sentences "The hurricane demolished houses in its path." and "The path of a meteor, of a caravan, or of a storm."?': 'Yes',
'Does the word "craft" mean the same thing in sentences "He represented the craft of brewers." and "He learned his craft as an apprentice."?': 'Yes',
'Does the word "change" mean the same thing in sentences "He attributed the change to their marriage." and "It is a refreshing change to meet a woman mechanic."?': 'No',
'Does the word "bar" mean the same thing in sentences "He drowned his sorrows in whiskey at the bar." and "The street was lined with all-night bars."?': 'Yes',
'Does the word "fact" mean the same thing in sentences "Your fears have no basis in fact." and "How much of the story is fact and how much fiction is hard to tell."?': 'Yes',
'Does the word "header" mean the same thing in sentences "Your header is too long ; " Local Cannibals " will suffice." and "This wall has four header courses."?': 'No',
'Does the word "host" mean the same thing in sentences "Atlanta was chosen to be host for the Olympic Games." and "Viruses depend on the host that they infect in order to be able to reproduce."?': 'No',
'Does the word "set" mean the same thing in sentences "Set a broken bone." and "We set up the sprinkler."?': 'No',
'Does the word "disentangle" mean the same thing in sentences "Can you disentangle the cord ?" and "How can I disentangle myself from her personal affairs ?"?': 'No',
'Does the word "debt" mean the same thing in sentences "He is badly in debt." and "I am in your debt."?': 'Yes',
'Does the word "call" mean the same thing in sentences "I would not call her beautiful." and "The unions called a general strike for Sunday."?': 'No',
'Does the word "raise" mean the same thing in sentences "We raise hogs here." and "Raise an army."?': 'No',
'Does the word "ply" mean the same thing in sentences "Ply an axe." and "Ply the seven seas."?': 'No',
'Does the word "help" mean the same thing in sentences "He was a great help to me when I was moving house." and "I\'ve printed out a list of math helps."?': 'Yes',
'Does the word "underbelly" mean the same thing in sentences "The soft underbelly of the Axis- Winston Churchill." and "The area by the canal is the underbelly of town - not something you\'d see in the guidebooks."?': 'No',
'Does the word "personality" mean the same thing in sentences "The president has a unique personality." and "She is a Hollywood personality."?': 'Yes',
'Does the word "release" mean the same thing in sentences "The release of iodine from the thyroid gland." and "A new release from the London Symphony Orchestra."?': 'No',
'Does the word "obstruction" mean the same thing in sentences "Obstruction of justice." and "The filibuster was a major obstruction to the success of their plan."?': 'No',
'Does the word "detest" mean the same thing in sentences "I detest snakes." and "She detests politicians."?': 'Yes',
'Does the word "triumph" mean the same thing in sentences "The triumph of knowledge." and "The agreement was a triumph for common sense."?': 'Yes',
'Does the word "number" mean the same thing in sentences "The word “ fifteen ”, the Arabic numeral “ 15 ”, and the Roman numeral “ XV ” are all numerals representing the number fifteen." and "Zero, one, -1, 2.5, and pi are all numbers."?': 'Yes',
'Does the word "stock" mean the same thing in sentences "He owns a controlling share of the company\'s stock." and "They will cut round stock to 1-inch diameter."?': 'No',
'Does the word "condition" mean the same thing in sentences "I condition my hair after washing it." and "He conditions old cars."?': 'No',
'Does the word "anesthetize" mean the same thing in sentences "Anesthetize the gum before extracting the teeth." and "The patient must be anesthetized before the operation."?': 'Yes',
'Does the word "active" mean the same thing in sentences "The ball was kicked by Fred. would, in active voice, be " Fred kicked the ball. "." and "` The boy threw the ball\' uses the active voice."?': 'Yes',
'Does the word "lid" mean the same thing in sentences "He raised the piano lid." and "His lids would stay open no longer."?': 'No',
'Does the word "raise" mean the same thing in sentences "The boss gave me a raise." and "He got a 3 % raise."?': 'Yes',
'Does the word "scare" mean the same thing in sentences "A bomb scare led them to evacuate the building." and "A war scare."?': 'Yes',
'Does the word "drive" mean the same thing in sentences "He sliced his drive out of bounds." and "His drive and energy exhausted his co-workers."?': 'No',
'Does the word "step" mean the same thing in sentences "He paused on the bottom step." and "The police took steps to reduce crime."?': 'No',
'Does the word "value" mean the same thing in sentences "The value assigned was 16 milliseconds." and "I establish the colors and principal values by organizing the painting into three values -- dark, medium... and light - Joe Hing Lowe."?': 'No',
'Does the word "standing" mean the same thing in sentences "A member in good standing." and "Of equal standing."?': 'Yes',
'Does the word "confront" mean the same thing in sentences "You must confront your opponent." and "John confronted it with emergency measures."?': 'Yes',
'Does the word "assent" mean the same thing in sentences "I will give this act my assent." and "He gave his assent eagerly."?': 'Yes',
'Does the word "refocus" mean the same thing in sentences "The group needs to refocus its goals." and "Refocus the image until it is very sharp."?': 'No',
'Does the word "precedence" mean the same thing in sentences "Its precedence as the world\'s leading manufacturer of pharmaceuticals." and "Family takes precedence over work, in an emergency."?': 'Yes',
'Does the word "bank" mean the same thing in sentences "Where do you bank in this town ?" and "He banked with Barclays."?': 'Yes',
'Does the word "squeeze" mean the same thing in sentences "A gentle squeeze on the arm." and "Increased expenses put a squeeze on profits."?': 'No',
'Does the word "light" mean the same thing in sentences "They saw the light of the beacon." and "There was a light at every corner."?': 'Yes',
'Does the word "projection" mean the same thing in sentences "A prime ingredient of public speaking is projection of the voice." and "Our ukuleles have been designed to have superior sound and projection."?': 'Yes',
'Does the word "lover" mean the same thing in sentences "A lover of fine wines." and "A lover of his country."?': 'Yes',
'Does the word "comparison" mean the same thing in sentences "They made a comparison of noise levels." and "To bring a thing into comparison with another."?': 'Yes',
'Does the word "pack" mean the same thing in sentences "You had better pack your swollen ankle with ice." and "The nurse packed gauze in the wound."?': 'Yes',
'Does the word "screen" mean the same thing in sentences "A metal screen protected the observers." and "They put screens in the windows for protection against insects."?': 'Yes',
'Does the word "press" mean the same thing in sentences "Press your thumb on this spot." and "He pressed down on the boards."?': 'Yes',
'Does the word "irritate" mean the same thing in sentences "Irritate the glands of a leaf." and "Aspirin irritates my stomach."?': 'No',
'Does the word "unemployed" mean the same thing in sentences "The long-term unemployed need assistance." and "The unemployed are a growing portion of the population."?': 'Yes',
'Does the word "deflate" mean the same thing in sentences "Deflate a balloon." and "The bad review of his work deflated his self-confidence."?': 'No',
'Does the word "mold" mean the same thing in sentences "Mold public opinion." and "The furniture molded in the old house."?': 'No',
'Does the word "top" mean the same thing in sentences "I do n\'t want to be bald, so just top my hair." and "Top trees and bushes."?': 'Yes',
'Does the word "freshness" mean the same thing in sentences "The freshness of the air revived him." and "She loved the freshness of newly baked bread."?': 'Yes',
'Does the word "dawn" mean the same thing in sentences "The dawn of civilization." and "It was the dawn of the Roman Empire."?': 'No',
'Does the word "slave" mean the same thing in sentences "His mother was his abject slave." and "Art thou the slave that with thy breath hast kill\'d/ Mine innocent child ? Shakespeare. Much Ado About Nothing."?': 'Yes',
'Does the word "march" mean the same thing in sentences "It was a long march." and "The march went up Fifth Avenue."?': 'No',
'Does the word "trim" mean the same thing in sentences "They traditionally trim the tree on Christmas Eve." and "Trim lumber."?': 'No',
'Does the word "overwinter" mean the same thing in sentences "Insects may overwinter in fallen fruit if it is not removed." and "Shackleton\'s men overwintered on Elephant Island."?': 'Yes',
'Does the word "analogy" mean the same thing in sentences "The models show by analogy how matter is built up." and "The operation of a computer presents and interesting analogy to the working of the brain."?': 'Yes',
'Does the word "wanton" mean the same thing in sentences "Wanton one\'s money away." and "The young man wantoned away his inheritance."?': 'Yes',
'Does the word "escape" mean the same thing in sentences "Their escape route." and "They installed a second hatch as an escape."?': 'Yes',
'Does the word "section" mean the same thing in sentences "No section of the nation is more ardent than the South." and "He always turns first to the business section."?': 'No',
'Does the word "playbook" mean the same thing in sentences "They borrowed a page from the playbook of the opposition." and "The 1963 playbook leaves out the whole first scene."?': 'No',
'Does the word "spin" mean the same thing in sentences "One of the planet\'s moons has a slower spin than the others." and "The car went into a spin."?': 'Yes',
'Does the word "cover" mean the same thing in sentences "When the pot comes to a boil, cover it and reduce the heat to medium." and "This farm covers some 200 acres."?': 'No',
'Does the word "truth" mean the same thing in sentences "He thought of answering with the truth but he knew they would n\'t believe it." and "Hunger and jealousy are just eternal truths of human existence."?': 'Yes',
'Does the word "splash" mean the same thing in sentences "Her red hat gave her outfit a splash of color." and "Just a splash of whiskey."?': 'No',
'Does the word "match" mean the same thing in sentences "Their interests did n\'t match, so it took a long time to agree what to do together." and "These two copies are supposed to be identical, but they do n\'t match."?': 'Yes',
'Does the word "unearth" mean the same thing in sentences "To unearth a secret." and "The CIA unearthed a plot to kill the President."?': 'Yes',
'Does the word "bend" mean the same thing in sentences "Copper rods bend easily." and "The road bends to the right."?': 'No',
'Does the word "reason" mean the same thing in sentences "The reason I robbed the bank was that I needed the money." and "We are told that man is endowed with reason and capable of distinguishing good from evil."?': 'No',
'Does the word "earth" mean the same thing in sentences "The earth shook for several minutes." and "She sighed when the plane\'s wheels finally touched earth."?': 'No',
'Does the word "shower" mean the same thing in sentences "Her church group has planned an adoption shower." and "Would male strippers be appropriate for the divorce shower ?"?': 'Yes',
'Does the word "warm" mean the same thing in sentences "He came inside to warm up." and "The soup warmed slowly on the stove."?': 'Yes',
'Does the word "render" mean the same thing in sentences "To render Latin into English." and "She rendered the French poem into English."?': 'Yes',
'Does the word "reunite" mean the same thing in sentences "After ten years apart, the band will reunite." and "Two of the members tried several times, but failed to reunite the band."?': 'Yes',
'Does the word "decree" mean the same thing in sentences "The King only can decree." and "The King decreed that all firstborn males should be killed."?': 'No',
'Does the word "planting" mean the same thing in sentences "The landscape architect suggested a small planting in the northwest corner." and "He ordered the planting of policemen outside every doorway."?': 'No',
'Does the word "slick" mean the same thing in sentences "Careful in turn three — there\'s an oil slick on the road." and "The oil slick has now spread to cover the entire bay, critically endangering the sea life."?': 'Yes',
'Does the word "heartland" mean the same thing in sentences "It was in the heartland of the French wine growing areas." and "The home counties are the Conservative heartland."?': 'Yes',
'Does the word "gum" mean the same thing in sentences "If you gum the tape it is stronger." and "These trees gum in the Spring."?': 'No',
'Does the word "handwriting" mean the same thing in sentences "Handwriting can be slow and painful for one with arthritis." and "She recognized his handwriting."?': 'No',
'Does the word "mail" mean the same thing in sentences "I\'ll mail you the paper when it\'s written." and "I\'ll mail you the check tomorrow."?': 'No',
'Does the word "instinct" mean the same thing in sentences "Many animals fear fire by instinct." and "Altruistic instincts in social animals."?': 'Yes',
'Does the word "recourse" mean the same thing in sentences "Have recourse to the courts." and "His only recourse was the police."?': 'No',
'Does the word "exchange" mean the same thing in sentences "All in all, it was an even exchange." and "Deductible losses on sales or exchanges of property are allowable."?': 'No',
'Does the word "tube" mean the same thing in sentences "In Paris the subway system is called the ` metro\' and in London it is called the ` tube\' or the ` underground\'." and "No mate, I am taking the tube !"?': 'Yes',
'Does the word "nature" mean the same thing in sentences "They tried to preserve nature as they found it." and "It is his nature to help others."?': 'No',
'Does the word "play" mean the same thing in sentences "The Democrats still have some cards to play before they will concede the electoral victory." and "He is playing his cards close to his chest."?': 'Yes',
'Does the word "alleviator" mean the same thing in sentences "Blessed is he who is an alleviator of suffering." and "Aspirin is an alleviator of headaches."?': 'Yes',
'Does the word "involvement" mean the same thing in sentences "He escaped involvement in the accident." and "There was additional involvement of the liver and spleen."?': 'Yes',
'Does the word "catch" mean the same thing in sentences "If you leave now you might catch him." and "Did you catch that allusion ?"?': 'No',
'Does the word "communication" mean the same thing in sentences "The node had established communication with the network, but had as yet sent no data." and "A secret passageway provided communication between the two rooms."?': 'No',
'Does the word "walk" mean the same thing in sentences "The patient can not walk yet." and "Walk with God."?': 'No',
'Does the word "approach" mean the same thing in sentences "He lost the hole when his approach rolled over the green." and "The approach of winter."?': 'No',
'Does the word "truth" mean the same thing in sentences "He told the truth." and "They debated the truth of the proposition."?': 'No',
'Does the word "constituent" mean the same thing in sentences "The animal constituent of plankton." and "Needs continued support by constituents to be re-elected."?': 'No',
'Does the word "demand" mean the same thing in sentences "The automobile reduced the demand for buggywhips." and "The demand exceeded the supply."?': 'Yes',
'Does the word "smolder" mean the same thing in sentences "The remains of the bonfire were left to smolder for hours." and "A smoldering fire."?': 'Yes',
'Does the word "place" mean the same thing in sentences "Place a bet." and "He placed the glass on the table."?': 'No',
'Does the word "manage" mean the same thing in sentences "The young violinist did n\'t manage her bow very well." and "She managed her parents\' affairs after they got too old."?': 'No',
'Does the word "iceberg" mean the same thing in sentences "Iceberg is still the most popular lettuce." and "The Titanic hit an iceberg and sank."?': 'No',
'Does the word "demand" mean the same thing in sentences "I demand to see the manager." and "The boss demanded that he be fired immediately."?': 'Yes',
'Does the word "react" mean the same thing in sentences "The hydrogen and the oxygen react." and "She reacts negatively to everything I say."?': 'No',
'Does the word "symbol" mean the same thing in sentences "The lamb is the symbol of meekness or patience." and "The lion is the symbol of courage."?': 'Yes',
'Does the word "identification" mean the same thing in sentences "Identification of the gun was an important clue." and "The thief\'s identification was followed quickly by his arrest."?': 'Yes',
'Does the word "pig" mean the same thing in sentences "After the ill-advised trade, the investor was stuck with worthless options for 10,000 tons of iron pig." and "The conveyor carried the pigs from the smelter to the freight cars."?': 'Yes',
'Does the word "slice" mean the same thing in sentences "A slice of the company\'s revenue." and "A slice of bread."?': 'No',
'Does the word "work" mean the same thing in sentences "Work the body onto the flatbed truck." and "He is working his servants hard."?': 'No',
'Does the word "pick" mean the same thing in sentences "Pick flowers." and "Pick mushrooms."?': 'Yes',
'Does the word "smear" mean the same thing in sentences "Smear the wall with paint." and "She smeared her lips with lipstick."?': 'Yes',
'Does the word "gesture" mean the same thing in sentences "The middle-finger gesture is really a nonverbal swear." and "This Web browser can be controlled with mouse gestures."?': 'Yes',
'Does the word "charge" mean the same thing in sentences "The wrestler\'s charge carried him past his adversary." and "The battle began with a cavalry charge."?': 'Yes',
'Does the word "discovery" mean the same thing in sentences "The defense argued that the plaintiff\'s discovery was inadequate." and "The purpose of the voyage was discovery."?': 'No',
'Does the word "contribution" mean the same thing in sentences "I am proud of my contribution in advancing the project." and "He expected his contribution to be repaid with interest."?': 'No',
'Does the word "manner" mean the same thing in sentences "What manner of man are you ?" and "Her dignified manner."?': 'No',
'Does the word "value" mean the same thing in sentences "Value the jewelry and art work in the estate." and "The dealer valued the book."?': 'Yes',
'Does the word "accept" mean the same thing in sentences "To accept the report of a committee." and "The cow accepted the bull."?': 'No',
'Does the word "fend" mean the same thing in sentences "The youngsters had to fend for themselves after their parents died." and "The mountain climbers had to fend against the ice and snow."?': 'No',
'Does the word "hour" mean the same thing in sentences "The hour grows late and I must go home." and "The hour is getting late."?': 'Yes',
'Does the word "whiff" mean the same thing in sentences "Whiff a pipe." and "The gust of air whiffed away the clouds."?': 'No',
'Does the word "runoff" mean the same thing in sentences "The runoff of nitrates is poisoning the lake." and "When there is very heavy rainfall on California hillsides over a prolonged period, there is almost always ensuing surface runoff."?': 'Yes',
'Does the word "trouble" mean the same thing in sentences "What is the trouble ?" and "The troubles in Northern Ireland."?': 'Yes',
'Does the word "hear" mean the same thing in sentences "We must hear the expert before we make a decision." and "I heard a sound from outside the window."?': 'Yes',
'Does the word "demonstration" mean the same thing in sentences "The lecturer shot off a pistol as a demonstration of the startle response." and "The lecture was accompanied by dramatic demonstrations."?': 'Yes',
'Does the word "compass" mean the same thing in sentences "This writer attempts more than his talents can compass." and "We compassed the earth."?': 'No',
'Does the word "see" mean the same thing in sentences "Do you see what I mean ?" and "See that the curtains are closed."?': 'No',
'Does the word "arrange" mean the same thing in sentences "Arrange my schedule." and "Arrange the books on the shelves in chronological order."?': 'No',
'Does the word "contact" mean the same thing in sentences "The wire must not contact the metal cover." and "The surfaces contact at this point."?': 'Yes',
'Does the word "quest" mean the same thing in sentences "A quest for diamonds." and "A quest for wealth."?': 'No',
'Does the word "edition" mean the same thing in sentences "They issued a limited edition of Bach recordings." and "The boy is a younger edition of his father."?': 'No',
'Does the word "fundus" mean the same thing in sentences "The fundus of the stomach." and "The uterine fundus."?': 'Yes',
'Does the word "grind" mean the same thing in sentences "Grind the spices in a mortar." and "Grind a lens."?': 'No',
'Does the word "trim" mean the same thing in sentences "Trim the christmas tree." and "Trim a shop window."?': 'Yes',
'Does the word "overhaul" mean the same thing in sentences "You should overhaul your car engine." and "Overhaul the health care system."?': 'Yes',
'Does the word "change" mean the same thing in sentences "Ask the janitor to come and change the lightbulb." and "After a brisk walk, I washed up and changed my shirt."?': 'Yes',
'Does the word "populate" mean the same thing in sentences "Populate the forest with deer and wild boar for hunting." and "Deer are populating the woods."?': 'No',
'Does the word "function" mean the same thing in sentences "Price is a function of supply and demand." and "Height is a function of age."?': 'Yes',
'Does the word "brandish" mean the same thing in sentences "Brandish a sword." and "He brandished his sword at the pirates."?': 'No',
'Does the word "outdoors" mean the same thing in sentences "He wanted to get outdoors a little." and "She loves the outdoors."?': 'Yes',
'Does the word "line" mean the same thing in sentences "To line a copy book." and "Trees line the riverbank."?': 'No',
'Does the word "satellite" mean the same thing in sentences "A spent upper stage is a derelict satellite." and "The Moon is a natural satellite of the Earth."?': 'Yes',
'Does the word "reform" mean the same thing in sentences "Reform your conduct." and "The Church reformed me."?': 'Yes',
'Does the word "study" mean the same thing in sentences "He knocked lightly on the closed door of the study." and "Anthropology is the study of human beings."?': 'No',
'Does the word "age" mean the same thing in sentences "Age has n\'t slowed him down at all." and "Old age is not for sissies."?': 'Yes',
'Does the word "version" mean the same thing in sentences "His version of the fight was different from mine." and "An experimental version of the night fighter."?': 'No',
'Does the word "pump" mean the same thing in sentences "Pump the gas pedal." and "I\'ve pumped over 1000 gallons of water in the last ten minutes."?': 'Yes',
'Does the word "cattle" mean the same thing in sentences "So many head of cattle." and "Do you want to raise cattle ?"?': 'Yes',
'Does the word "fly" mean the same thing in sentences "All nations fly their flags in front of the U.N." and "The pilot flew to Cuba."?': 'No',
'Does the word "shipment" mean the same thing in sentences "We\'re expecting another shipment of oranges tomorrow." and "Shipment of hazardous waste on this route is strictly prohibited."?': 'No',
'Does the word "shake" mean the same thing in sentences "Do n\'t shake your fist at me !" and "These stories shook the community."?': 'No',
'Does the word "bone" mean the same thing in sentences "I had to bone up on my Latin verbs before the final exam." and "Bone the turkey before roasting it."?': 'No',
'Does the word "start" mean the same thing in sentences "To start a stream of water." and "We simply could not start the engine."?': 'Yes',
'Does the word "pulse" mean the same thing in sentences "Pulse waves." and "A transmitter pulsed by an electronic tube."?': 'Yes',
'Does the word "foundation" mean the same thing in sentences "The foundation of a new scientific society." and "There is little foundation for his objections."?': 'No',
'Does the word "material" mean the same thing in sentences "The archives provided rich material for a definitive biography." and "Coal is a hard black material."?': 'No',
'Does the word "foul" mean the same thing in sentences "To foul the face or hands with mire." and "She\'s fouled her diaper."?': 'Yes',
'Does the word "buy" mean the same thing in sentences "I\'m going to buy my father something nice for his birthday." and "She buys for the big department store."?': 'Yes',
'Does the word "coat" mean the same thing in sentences "A second coat of paint." and "When the dog shed its coat, it left hair all over the furniture and the carpet."?': 'No',
'Does the word "resume" mean the same thing in sentences "Resume a title." and "Resume one\'s duties."?': 'Yes',
'Does the word "answer" mean the same thing in sentences "There is no simple answer to corruption." and "The answers were in the back of the book."?': 'Yes',
'Does the word "prize" mean the same thing in sentences "We prize his creativity." and "I prize these old photographs."?': 'No',
'Does the word "deposit" mean the same thing in sentences "Deposit the suitcase on the bench." and "She deposits her paycheck every month."?': 'No',
'Does the word "graduate" mean the same thing in sentences "Graduate a cylinder." and "This school graduates 2,000 students each year."?': 'No',
'Does the word "shade" mean the same thing in sentences "The butterfly wings shade to yellow." and "Shade your eyes when you step out into the bright sunlight."?': 'No',
'Does the word "point" mean the same thing in sentences "Point a chimney." and "He pointed to the empty parking space."?': 'No',
'Does the word "lesson" mean the same thing in sentences "The accident was a good lesson to me." and "I hope this accident taught you a lesson !"?': 'Yes',
'Does the word "grace" mean the same thing in sentences "A beautiful figure which she used in subtle movements of unparalleled grace." and "The conception of grace developed alongside the conception of sin."?': 'No',
'Does the word "negociate" mean the same thing in sentences "Negociate securities." and "The parties negociated all night."?': 'No',
'Does the word "toehold" mean the same thing in sentences "American diplomacy provided a toehold on which to proceed toward peace talks." and "His father gave him a toehold in the oil business."?': 'Yes',
'Does the word "garden" mean the same thing in sentences "Our garden is overgrown with weeds." and "This house has a swimming pool, a tent, a swing set and a fountain in the garden."?': 'Yes',
'Does the word "protection" mean the same thing in sentences "They had no protection from the fallout." and "The witnesses demanded police protection."?': 'No',
'Does the word "loosen" mean the same thing in sentences "Loosen the tension on a rope." and "Loosen the soil."?': 'No',
'Does the word "discipline" mean the same thing in sentences "He insisted on discipline among the troops." and "The offenders deserved the harsh discipline they received."?': 'No',
'Does the word "waste" mean the same thing in sentences "Waste heat." and "Waste a joke on an unappreciative audience."?': 'Yes',
'Does the word "act" mean the same thing in sentences "You should act like an adult." and "He\'s acting strangely - I think there\'s something wrong with him."?': 'Yes',
'Does the word "card" mean the same thing in sentences "They have to card anybody who looks 21 or younger." and "I heard you do n\'t get carded at the other liquor store."?': 'Yes',
'Does the word "evoke" mean the same thing in sentences "Evoke sympathy." and "This remark evoked sadness."?': 'No',
'Does the word "blanket" mean the same thing in sentences "The city woke under a thick blanket of fog." and "There was a blanket of snow."?': 'No',
'Does the word "inundate" mean the same thing in sentences "The Dutch would sometimes inundate the land to hinder the Spanish army." and "The basement was inundated after the storm."?': 'No',
'Does the word "lump" mean the same thing in sentences "A lump of coal." and "Do you want one lump or two with your coffee ?"?': 'No',
'Does the word "snag" mean the same thing in sentences "Snag a bargain." and "I snagged my stocking."?': 'No',
'Does the word "hollow" mean the same thing in sentences "He built himself a cabin in a hollow high up in the Appalachians." and "Hunger had caused the hollows in their cheeks."?': 'No',
'Does the word "do" mean the same thing in sentences "What makes her do this way ?" and "Do n\'t behave like a fool."?': 'Yes',
'Does the word "network" mean the same thing in sentences "A network of canals." and "Retirement meant dropping out of a whole network of people who had been part of my life."?': 'No',
'Does the word "tad" mean the same thing in sentences "A tad too expensive." and "Not a tad of difference."?': 'Yes',
'Does the word "avert" mean the same thing in sentences "Avert a strike." and "How can the danger be averted ?"?': 'Yes',
'Does the word "receive" mean the same thing in sentences "These aspects of civilization do not find expression or receive an interpretation." and "His movie received a good review."?': 'Yes',
'Does the word "inwardness" mean the same thing in sentences "The sensitiveness of James\'s characters, their seeming inwardness." and "Inwardness is what an Englishman quite simply has, painlessly, as a birthright."?': 'Yes',
'Does the word "cure" mean the same thing in sentences "Cure pickles." and "The smoke and heat cures the meat."?': 'Yes',
'Does the word "speculation" mean the same thing in sentences "He knew the stock was a speculation when he bought it." and "Speculations about the outcome of the election."?': 'No',
'Does the word "minute" mean the same thing in sentences "He ran a 4 minute mile." and "You have twenty minutes to complete the test."?': 'Yes',
'Does the word "dereliction" mean the same thing in sentences "The new soldier did not clean his cabin and was scolded for dereliction and disobedience." and "What he did was a terrible dereliction of duty."?': 'Yes',
'Does the word "interpretation" mean the same thing in sentences "The report included his interpretation of the forensic evidence." and "The edict was subject to many interpretations."?': 'No',
'Does the word "administer" mean the same thing in sentences "Administer the last unction." and "Administer an exam."?': 'No',
'Does the word "juncture" mean the same thing in sentences "We\'re at a crucial juncture in our relationship." and "At that juncture he had no idea what to do."?': 'Yes',
'Does the word "vow" mean the same thing in sentences "The old hermit, up in the mountains, took a vow of silence." and "They took vows of poverty."?': 'Yes',
'Does the word "boil" mean the same thing in sentences "Add the noodles when the water comes to the boil." and "They brought the water to a boil."?': 'Yes',
'Does the word "spike" mean the same thing in sentences "The seismograph showed a sharp spike in response to the temblor." and "The second baseman sharpened his spikes before every game."?': 'No',
'Does the word "puff" mean the same thing in sentences "Puff a cigar." and "He huffed and puffed as he made his way up the mountain."?': 'No',
'Does the word "demand" mean the same thing in sentences "I demand an explanation." and "I demand an immediate explanation."?': 'Yes',
'Does the word "meet" mean the same thing in sentences "Meet a need." and "My proposal met with much opposition."?': 'No',
'Does the word "synthesize" mean the same thing in sentences "His operas synthesize music and drama in perfect harmony." and "The liver synthesizes vitamins."?': 'Yes',
'Does the word "flagship" mean the same thing in sentences "It is their flagship newspaper." and "The retail company\'s store in London is their flagship."?': 'Yes',
'Does the word "stringer" mean the same thing in sentences "A second stringer." and "A first stringer."?': 'Yes',
'Does the word "thing" mean the same thing in sentences "I could n\'t tell what the thing was." and "Things are going well."?': 'No',
'Does the word "carrier" mean the same thing in sentences "The bonds were transmitted by carrier." and "Refrigerated carriers have revolutionized the grocery business."?': 'No',
'Does the word "foul" mean the same thing in sentences "Foul the water." and "Jones fouled for strike one."?': 'No',
'Does the word "ping" mean the same thing in sentences "I\'ll ping my accountant -- April 15 is nearing." and "My car was pinging until my last oil change."?': 'No',
'Does the word "receive" mean the same thing in sentences "Receive letters from the front." and "The lady is receiving Sunday morning."?': 'No',
'Does the word "rule" mean the same thing in sentences "Rule one\'s temper." and "Rule the margins."?': 'No',
'Does the word "contrast" mean the same thing in sentences "The students contrast considerably in their artistic abilities." and "The middle school teacher contrasted her best student\'s work with that of her weakest student."?': 'No',
'Does the word "tier" mean the same thing in sentences "Tier upon tier of huge casks." and "A three tier wedding cake."?': 'Yes',
'Does the word "stone" mean the same thing in sentences "Her face was as hard as stone." and "Stone is abundant in New England and there are many quarries."?': 'No',
'Does the word "zero" mean the same thing in sentences "The temperature outside is ten degrees below zero." and "The electromagnetic field does not drop all of the way to zero before a reversal."?': 'No',
'Does the word "consolidation" mean the same thing in sentences "The defendants asked for a consolidation of the actions against them." and "A consolidation of two corporations."?': 'Yes',
'Does the word "commit" mean the same thing in sentences "Commit a random act of kindness." and "She committed herself to the work of God."?': 'No',
'Does the word "erase" mean the same thing in sentences "Please erase the formula on the blackboard -- it is wrong !" and "Her painful memories seemingly erased completely."?': 'Yes',
'Does the word "sawmill" mean the same thing in sentences "He brought his portable sawmill and turned the old beams into interesting flooring." and "The sawmill sells lumber to carpenters and sawdust to gardeners."?': 'Yes',
'Does the word "embarrassment" mean the same thing in sentences "An embarrassment of riches." and "The outcome of the vote was an embarrassment for the liberals."?': 'No',
'Does the word "view" mean the same thing in sentences "View a problem." and "He viewed the painting and praised the artist for his masterpiece."?': 'Yes',
'Does the word "sincerity" mean the same thing in sentences "His sincerity inspired belief." and "They demanded some proof of my sincerity."?': 'Yes',
'Does the word "hair" mean the same thing in sentences "He combed his hair." and "In the western world, women usually have long hair while men usually have short hair."?': 'Yes',
'Does the word "issue" mean the same thing in sentences "She found an old issue of the magazine in her dentist\'s waiting room." and "Politicians never discuss the real issues."?': 'No',
'Does the word "square" mean the same thing in sentences "Square the wood with a file." and "He squared his shoulders."?': 'No',
'Does the word "concentrate" mean the same thing in sentences "Population tends to concentrate in cities." and "These groups concentrate in the inner cities."?': 'Yes',
'Does the word "leap" mean the same thing in sentences "Leap into fame." and "The child leapt across the puddle."?': 'No',
'Does the word "relief" mean the same thing in sentences "He enjoyed his relief from responsibility." and "The contributions provided some relief for the victims."?': 'No',
'Does the word "balance" mean the same thing in sentences "Balance the two weights." and "She has to balance work and her domestic duties."?': 'Yes',
'Does the word "oxygenate" mean the same thing in sentences "Oxygenate blood." and "After we oxygenated the river, the fish returned."?': 'Yes',
'Does the word "flame" mean the same thing in sentences "The sky seemed to flame in the Hawaiian sunset." and "The person who posted an inflammatory message got flamed."?': 'No',
'Does the word "weather" mean the same thing in sentences "They were hoping for good weather." and "Wooden garden furniture must be well oiled as it is continuously exposed to weather."?': 'Yes',
'Does the word "extension" mean the same thing in sentences "Files with the.txt extension usually contain text." and "Most applications provide extensions for the files they create."?': 'Yes',
'Does the word "balance" mean the same thing in sentences "To balance a set of books." and "This final payment, or credit, balances the account."?': 'Yes',
'Does the word "cut" mean the same thing in sentences "He played the first cut on the cd." and "He insisted that we give him the last cut before every deal."?': 'No',
'Does the word "disorder" mean the same thing in sentences "The class was thrown into disorder when the teacher left the room." and "The army tried to prevent disorder when claims the elections had been rigged grew stronger."?': 'Yes',
'Does the word "object" mean the same thing in sentences "The sole object of her trip was to see her children." and "It was full of rackets, balls and other objects."?': 'No',
'Does the word "part" mean the same thing in sentences "The first violin part in this concerto is very challenging." and "He tried to sing the tenor part."?': 'Yes',
'Does the word "appeal" mean the same thing in sentences "An appeal to raise money for starving children." and "Their appeal was denied in the superior court."?': 'No',
'Does the word "draft" mean the same thing in sentences "I have to revise the first draft of my term paper." and "They served beer on draft."?': 'No',
'Does the word "plant" mean the same thing in sentences "The police used a plant to trick the thieves." and "He claimed that the evidence against him was a plant."?': 'Yes',
'Does the word "wall" mean the same thing in sentences "Competition was pushing them to the wall." and "His back was to the wall."?': 'Yes',
'Does the word "space" mean the same thing in sentences "The first major milestone in space exploration was in 1957, when the USSR\'s Sputnik 1 orbited the Earth." and "He said the space is the most important character in the alphabet."?': 'No',
'Does the word "ice" mean the same thing in sentences "Look at the ice on that dame !" and "Americans like ice in their drinks."?': 'No',
'Does the word "outburst" mean the same thing in sentences "The man let out an outburst of invective." and "An outburst of heavy rain."?': 'No',
'Does the word "fix" mean the same thing in sentences "Fix breakfast for the guests, please." and "Fix the variables."?': 'No',
'Does the word "pizza" mean the same thing in sentences "He ate a whole pizza !" and "Should we cook a frozen pizza for dinner ?"?': 'Yes',
'Does the word "bask" mean the same thing in sentences "To bask in someone\'s favour." and "The seals were basking in the sun."?': 'No',
'Does the word "run" mean the same thing in sentences "Run amok." and "Run riot."?': 'Yes',
'Does the word "cream" mean the same thing in sentences "The cream of England\'s young men were killed in the Great War." and "The cream of a collection of books or pictures."?': 'Yes',
'Does the word "channel" mean the same thing in sentences "A satellite TV channel." and "They offer more than one hundred channels."?': 'Yes',
'Does the word "negate" mean the same thing in sentences "The investigation tending to negate any supernatural influences." and "A pessimism that always negates."?': 'No',
'Does the word "navel" mean the same thing in sentences "The Incas believed that Cuzco was the navel of the universe." and "Within the navel of this hideous wood, / Immured in cypress shades, a sorcerer dwells. — Milton."?': 'Yes',
'Does the word "resolution" mean the same thing in sentences "It was his unshakeable resolution to finish the work." and "He always wrote down his New Year\'s resolutions."?': 'No',
'Does the word "guard" mean the same thing in sentences "Guard my possessions while I\'m away." and "There would be men guarding the horses."?': 'No',
'Does the word "subscription" mean the same thing in sentences "I have a monthly subscription to The Daily Telegraph newspaper." and "My library subscription is about to expire."?': 'Yes',
'Does the word "stimulate" mean the same thing in sentences "This herbal infusion does n\'t stimulate." and "Coffee and tea stimulate me."?': 'Yes',
'Does the word "catcher" mean the same thing in sentences "A catcher needs a lot of protective equipment." and "A catcher plays behind the plate."?': 'Yes',
'Does the word "piracy" mean the same thing in sentences "Air piracy." and "How shoud the international community respond to Somali piracy ?"?': 'Yes',
'Does the word "chuck" mean the same thing in sentences "This food ′s gone off - you′d better chuck it." and "Chuck these old notes."?': 'Yes',
'Does the word "sewer" mean the same thing in sentences "A sewer of fine gowns." and "All that work went down the sewer."?': 'No',
'Does the word "snapshot" mean the same thing in sentences "He carried a snapshot of his daughter." and "My snapshots have n\'t been developed yet."?': 'Yes',
'Does the word "say" mean the same thing in sentences "What does this painting say ?" and "Did his face say anything about how he felt ?"?': 'Yes',
'Does the word "see" mean the same thing in sentences "The artist must first learn to see." and "See whether it works."?': 'No',
'Does the word "face" mean the same thing in sentences "Shut your face !" and "He\'s always stuffing his face with chips."?': 'Yes',
'Does the word "tune" mean the same thing in sentences "To tune a piano or a violin." and "My piano needs to be tuned."?': 'Yes',
'Does the word "appreciate" mean the same thing in sentences "Do you appreciate the full meaning of this letter ?" and "To test the power of bees to appreciate color."?': 'Yes',
'Does the word "reach" mean the same thing in sentences "Can he reach ?" and "The water reached the doorstep."?': 'No',
'Does the word "memory" mean the same thing in sentences "He can do it from memory." and "Memory is a facility common to all animals."?': 'Yes',
'Does the word "hit" mean the same thing in sentences "One boy hit the other." and "He hit her hard in the face."?': 'Yes',
'Does the word "sober" mean the same thing in sentences "She\'s in no state to go home by herself, so make sure she does n\'t drink anything else, and she\'ll soon sober up." and "Keep him in bed until he sobers up."?': 'Yes',
'Does the word "truncation" mean the same thing in sentences "It is an obvious truncation of the verse." and "They were sentenced to a truncation of their limbs."?': 'Yes',
'Does the word "analyze" mean the same thing in sentences "Analyze a sonnet by Shakespeare." and "The inspector analyzed the building."?': 'Yes',
'Does the word "remainder" mean the same thing in sentences "17 leaves a remainder of 2 when divided by 3." and "11 divided by 2 is 5 remainder 1."?': 'Yes',
'Does the word "experiment" mean the same thing in sentences "Students experiment sexually." and "The composer experimented with a new style."?': 'Yes',
'Does the word "wrench" mean the same thing in sentences "Wrench a window off its hinges." and "Wrench one\'s ankle."?': 'No',
'Does the word "tenderness" mean the same thing in sentences "He noted her extreme tenderness when he touched the bruise on her thigh." and "The best results are generally obtained by inserting the needle into the point of maximum tenderness."?': 'Yes',
'Does the word "honour" mean the same thing in sentences "She swore on her honour." and "His honour is at stake."?': 'Yes',
'Does the word "lash" mean the same thing in sentences "And big waves lash the frighted shores. — John Dryden." and "The whale lashes the sea with its tail."?': 'Yes',
'Does the word "origin" mean the same thing in sentences "Jupiter was the origin of the radiation." and "He comes from good origins."?': 'No',
'Does the word "constitutionalize" mean the same thing in sentences "A good way of exercising is to constitutionalize." and "A woman\'s right to an abortion was constitutionalized in the 1970\'s."?': 'No',
'Does the word "run" mean the same thing in sentences "Run a subject." and "Run a process."?': 'Yes',
'Does the word "start" mean the same thing in sentences "Who will start ?" and "I start at eight in the morning."?': 'No',
'Does the word "warrant" mean the same thing in sentences "I warrant this information." and "The dealer warrants all the cars he sells."?': 'Yes',
'Does the word "enjoy" mean the same thing in sentences "I plan to go travelling while I still enjoy good health." and "Enjoy privileges."?': 'Yes',
'Does the word "dig" mean the same thing in sentences "Dig a hole." and "My seven-year-old son always digs a hole in the middle of his mashed potatoes and fills it with gravy before he starts to eat them."?': 'Yes',
'Does the word "pop" mean the same thing in sentences "Listen to the pop of a champagne cork." and "Pop music, despite its commercial focus, includes many artistic gems."?': 'No',
'Does the word "spin" mean the same thing in sentences "He put some spin on the cue ball." and "He gave the crank a spin."?': 'No',
'Does the word "ribbon" mean the same thing in sentences "A steel or magnesium ribbon." and "Sails torn to ribbons."?': 'Yes',
'Does the word "tick" mean the same thing in sentences "Indicate that you are willing to receive marketing material by putting a tick in the box." and "He counted the ticks of the clock."?': 'No',
'Does the word "network" mean the same thing in sentences "A railroad network." and "To get a job in today\'s economy, it is important to have a strong network."?': 'No',
'Does the word "passing" mean the same thing in sentences "The passing of winter." and "The coach sent in a passing play on third and long."?': 'No',
'Does the word "draft" mean the same thing in sentences "A preliminary draft." and "The final draft of the constitution."?': 'Yes',
'Does the word "stater" mean the same thing in sentences "Keystone stater." and "Farm staters."?': 'Yes',
'Does the word "salvation" mean the same thing in sentences "They turned to individualism as their salvation." and "Tourism was their economic salvation."?': 'Yes',
'Does the word "generate" mean the same thing in sentences "We ca n\'t generate enough power for the entire city." and "The hydroelectric plant needs to generate more electricity."?': 'Yes',
'Does the word "crumb" mean the same thing in sentences "He gave only a crumb of information about his plans." and "There were few crumbs of comfort in the report."?': 'Yes',
'Does the word "start" mean the same thing in sentences "It was off to a good start." and "They got an early start."?': 'No',
'Does the word "takeaway" mean the same thing in sentences "In England they call takeout food ` takeaway\'." and "If you\'re hungry, there\'s a takeaway just around the corner."?': 'Yes',
'Does the word "wind" mean the same thing in sentences "Wind your watch." and "The river winds through the hills."?': 'No',
'Does the word "attack" mean the same thing in sentences "The visiting team started to attack." and "The cancer cells are attacking his liver."?': 'No',
'Does the word "discourage" mean the same thing in sentences "We should discourage this practice among our youth." and "Do n\'t be discouraged by the amount of work left to do : you\'ll finish it in good time."?': 'No',
'Does the word "issue" mean the same thing in sentences "He died without issue." and "The last issue of penicillin was over a month ago."?': 'No',
'Does the word "trip" mean the same thing in sentences "He recited the whole poem without a single trip." and "An acid trip."?': 'No',
'Does the word "blue" mean the same thing in sentences "The boys in blue marched to the pipers." and "She was wearing blue."?': 'Yes',
'Does the word "relation" mean the same thing in sentences "He was the hero according to his own relation." and "He has distant relations back in New Jersey."?': 'No',
'Does the word "work" mean the same thing in sentences "Work stone into tools." and "Work the metal."?': 'Yes',
'Does the word "check" mean the same thing in sentences "Check the spelling of this word." and "Check your facts."?': 'Yes',
'Does the word "channel" mean the same thing in sentences "We will channel the traffic to the left with these cones." and "Channel information towards a broad audience."?': 'Yes',
'Does the word "enclosure" mean the same thing in sentences "He faced punishment for creating the fenced enclosure in a public park." and "The winning horse was first into the unsaddling enclosure."?': 'Yes',
'Does the word "third" mean the same thing in sentences "He was cut down on a close play at third." and "The play ended with Jones standing on third."?': 'Yes',
'Does the word "influence" mean the same thing in sentences "Her wishes had a great influence on his thinking." and "The influence of mechanical action."?': 'No',
'Does the word "hi" mean the same thing in sentences "I just dropped by to say “ hi ”." and "Hi, how are you ?"?': 'Yes',
'Does the word "infamy" mean the same thing in sentences "A date which will live in infamy - Franklin D. Roosevelt in response to the Japanese attack on Pearl Harbour." and "Infamy, infamy - they\'ve all got it in for me ! - Kenneth Williams as Julius Caesar in Carry On Cleo."?': 'Yes',
'Does the word "handcart" mean the same thing in sentences "He used a handcart to carry the rocks away." and "Vendors were selling sunglasses, bagged snacks, and sundry other items from handcarts in the square."?': 'Yes',
'Does the word "fuckup" mean the same thing in sentences "You\'ve got to fire that fuckup." and "Forgetting our anniversary was a major fuckup."?': 'No',
'Does the word "difference" mean the same thing in sentences "You need to learn to be more tolerant of difference." and "There are many differences between jazz and rock."?': 'Yes',
'Does the word "problem" mean the same thing in sentences "What\'s the problem ?" and "She and her husband are having problems."?': 'No',
'Does the word "fantasy" mean the same thing in sentences "A schoolgirl fantasy." and "She made a lot of money writing romantic fantasies."?': 'No',
'Does the word "mingle" mean the same thing in sentences "He was about to mingle in an unpleasant affair." and "Resourcefully he mingled music and dance."?': 'No',
'Does the word "luck" mean the same thing in sentences "Sometimes it takes a bit of luck to get success." and "The raffle is just a matter of luck."?': 'Yes',
'Does the word "stampede" mean the same thing in sentences "Thunderbolts can stampede animals." and "The tavern owners stampeded us into overeating."?': 'No',
'Does the word "value" mean the same thing in sentences "The value of a crotchet is twice that of a quaver." and "He has old-fashioned values."?': 'No',
'Does the word "congener" mean the same thing in sentences "The American shopkeeper differs from his European congener." and "Lard was also used, though its congener, butter, was more frequently employed."?': 'Yes',
'Does the word "sculpt" mean the same thing in sentences "Sculpt a swan out of a block of ice." and "She is sculpting the block of marble into an image of her husband."?': 'No',
'Does the word "curve" mean the same thing in sentences "To curve a line." and "The road curved sharply."?': 'Yes',
'Does the word "do" mean the same thing in sentences "All you ever do is surf the Internet." and "She did a little dance."?': 'Yes',
'Does the word "crook" mean the same thing in sentences "A crook in the path." and "She held the baby in the crook of her arm."?': 'Yes',
'Does the word "fit" mean the same thing in sentences "He had a laughing fit which lasted more than ten minutes." and "He threw a fit when his car broke down."?': 'Yes',
'Does the word "tack" mean the same thing in sentences "The sailors decided to tack the boat." and "The boat tacked."?': 'Yes',
'Does the word "disaster" mean the same thing in sentences "The earthquake was a disaster." and "His policies were a disaster."?': 'No',
'Does the word "fiddle" mean the same thing in sentences "The reporter fiddle with the facts." and "Zuckerman fiddled that song very nicely."?': 'No',
'Does the word "witness" mean the same thing in sentences "She can bear witness, since she was there at the time." and "As a witness to the event, I can tell you that he really said that."?': 'No',
'Does the word "organization" mean the same thing in sentences "Over time, the spontaneous movement had become an organization." and "His compulsive organization was not an endearing quality."?': 'No',
'Does the word "steam" mean the same thing in sentences "Steam and clean the upholstered sofa." and "It really steams me to see her treat him like that."?': 'No',
'Does the word "submission" mean the same thing in sentences "What was the date of submission of your proposal ?" and "Several of his submissions were rejected by publishers."?': 'Yes',
'Does the word "lead" mean the same thing in sentences "The lead was in the dummy." and "The children were playing with lead soldiers."?': 'No',
'Does the word "elaborate" mean the same thing in sentences "What do you mean you did n\'t come home last night ? Would you care to elaborate ?" and "She elaborated on the main ideas in her dissertation."?': 'No',
'Does the word "pass" mean the same thing in sentences "He attempted the examination, but did not expect to pass." and "The sports car passed all the trucks."?': 'No',
'Does the word "bake" mean the same thing in sentences "Potatoes bake to a crisp in the oven." and "Bake a cake."?': 'No',
'Does the word "foot" mean the same thing in sentences "Foot the bill." and "To foot ( or foot up ) an account."?': 'No',
'Does the word "sufficiency" mean the same thing in sentences "He questioned the sufficiency of human intelligence." and "Her father questioned the young suitor\'s sufficiency."?': 'No',
'Does the word "reorganization" mean the same thing in sentences "The reorganization was prescribed by federal bankruptcy laws." and "After the takeover there was a thorough reorganization."?': 'Yes',
'Does the word "ship" mean the same thing in sentences "Ship the cargo in the hold of the vessel." and "To ship the tiller or rudder."?': 'Yes',
'Does the word "tone" mean the same thing in sentences "A clergyman improved the tone of the meeting." and "They tested his hearing with pure tones of different frequencies."?': 'No',
'Does the word "responsibility" mean the same thing in sentences "Every right implies a responsibility." and "Why did n\'t you clean the house ? That was your responsibility !"?': 'Yes',
'Does the word "career" mean the same thing in sentences "He had a long career in the law." and "The general had had a distinguished career."?': 'Yes',
'Does the word "increase" mean the same thing in sentences "They proposed an increase of 15 percent in the fare." and "The increase in unemployment."?': 'No',
'Does the word "law" mean the same thing in sentences "The law came looking for him." and "The laws of thermodynamics."?': 'No',
'Does the word "here" mean the same thing in sentences "Where do we go from here ?" and "An Alzheimer patient\'s in here may in his mind be anywhere he called home in the time he presently re-lives."?': 'Yes',
'Does the word "water" mean the same thing in sentences "The smell of fried onions makes my mouth water." and "His eyes were watering."?': 'Yes',
'Does the word "desire" mean the same thing in sentences "Too much desire can seriously affect one\'s judgment." and "A man of many desires."?': 'No',
'Does the word "capacity" mean the same thing in sentences "The capacity of a hard disk drive is usually expressed in megabytes." and "The plant is working at 80 per cent capacity."?': 'No',
'Does the word "care" mean the same thing in sentences "He wrote the manual on car care." and "Care had aged him."?': 'No',
'Does the word "stretch" mean the same thing in sentences "Stretch my patience." and "During the Inquisition, the torturers would stretch their victims on a rack."?': 'No',
'Does the word "vegetation" mean the same thing in sentences "Pleistocene vegetation." and "Their holiday was spent in sleep and vegetation."?': 'No',
'Does the word "hand" mean the same thing in sentences "He wanted to try his hand at singing." and "Give the little lady a great big hand."?': 'No',
'Does the word "delay" mean the same thing in sentences "Do n\'t delay your application to graduate school or else it wo n\'t be considered." and "He delayed the watering."?': 'Yes',
'Does the word "crystallize" mean the same thing in sentences "Crystallize minerals." and "I crystallized the copper sulphate by slowly cooling a saturated solution."?': 'Yes',
'Does the word "dig" mean the same thing in sentences "In the wintertime, heavy truck tires dig into the road, forming potholes." and "They dug an eight - foot ditch along the side of the road."?': 'Yes',
'Does the word "stage" mean the same thing in sentences "We are in a transitional stage in which many former ideas must be revised or rejected." and "It set the stage for peaceful negotiations."?': 'No',
'Does the word "spray" mean the same thing in sentences "The bridesmaid carried a spray of lily-of-the-valley." and "A spray of bullets."?': 'No',
'Does the word "effect" mean the same thing in sentences "Effect a change." and "The best way to effect change is to work with existing stakeholders."?': 'Yes',
'Does the word "seat" mean the same thing in sentences "Seat a concert hall." and "This classroom seats two hundred students."?': 'Yes',
'Does the word "cheat" mean the same thing in sentences "Six years of a happy relationship, and then she had the nerve to cheat on me with a barman !" and "My husband cheated on me with his secretary."?': 'Yes',
'Does the word "extract" mean the same thing in sentences "To extract an essential oil from a plant." and "I extracted a promise from the Dean for two new positions."?': 'No',
'Does the word "sew" mean the same thing in sentences "To sew money into a bag." and "Can the seamstress sew me a suit by next week ?"?': 'No',
'Does the word "liquidate" mean the same thing in sentences "Liquidate a company." and "The mafia liquidated the informer."?': 'No',
'Does the word "relax" mean the same thing in sentences "Let\'s all relax after a hard day\'s work." and "He relaxed in the hot tub."?': 'Yes',
'Does the word "begin" mean the same thing in sentences "Begin a new chapter in your life." and "She began Russian at an early age."?': 'No',
'Does the word "corner" mean the same thing in sentences "Corner the gold market." and "The car corners."?': 'No',
'Does the word "music" mean the same thing in sentences "He fell asleep to the music of the wind chimes." and "You have to face the music."?': 'No',
'Does the word "appropriation" mean the same thing in sentences "A person\'s appropriation of property belonging to another is dishonest." and "The necessary funds were obtained by the government\'s appropriation of the company\'s operating unit."?': 'Yes',
'Does the word "pickup" mean the same thing in sentences "Garbage pickup is on Mondays and Thursdays." and "She needed the pickup that coffee always gave her."?': 'No',
'Does the word "declare" mean the same thing in sentences "Declare dividends." and "His wife declared at once for moving to the West Coast."?': 'No',
'Does the word "name" mean the same thing in sentences "Name your accomplices !" and "He named his demands."?': 'Yes',
'Does the word "blaze" mean the same thing in sentences "The horse had a blaze between its eyes." and "They chipped off bark to mark the trail with blazes."?': 'Yes',
'Does the word "pitch" mean the same thing in sentences "It is hard to get this pitch off of my hand." and "He was employed to see that his paper\'s news pitches were not trespassed upon by rival vendors."?': 'No',
'Does the word "mash" mean the same thing in sentences "Mash the garlic." and "To mash on a bicycle pedal."?': 'No',
'Does the word "clip" mean the same thing in sentences "Give him a clip round the ear !" and "He gave me a clip on the ear."?': 'Yes',
'Does the word "blaze" mean the same thing in sentences "The blaze spread rapidly." and "Go to blazes."?': 'No',
'Does the word "warn" mean the same thing in sentences "I warn you against false assumptions." and "I warned him not to go too far."?': 'Yes',
'Does the word "state" mean the same thing in sentences "He was in such a state you just could n\'t reason with him." and "Though legally single nations, many states comprise several distinct cultural or ethnic groups."?': 'No',
'Does the word "mold" mean the same thing in sentences "A lobster mold." and "A gelatin dessert made in a mold."?': 'Yes',
'Does the word "drain" mean the same thing in sentences "Please drain the sink. It\'s full of dirty water." and "We drained the oil tank."?': 'Yes',
'Does the word "yesterday" mean the same thing in sentences "Yesterday\'s solutions are not good enough." and "Yesterday\'s technology."?': 'Yes',
'Does the word "cook" mean the same thing in sentences "My husband does n\'t cook." and "The apothecary cooked the medicinal mixture in a big iron kettle."?': 'No',
'Does the word "toast" mean the same thing in sentences "Let us toast the birthday girl !" and "Toast bread."?': 'No',
'Does the word "writing" mean the same thing in sentences "He turned the paper over so the writing would n\'t show." and "The doctor\'s writing was illegible."?': 'Yes',
'Does the word "circuit" mean the same thing in sentences "On the lecture circuit." and "The international tennis circuit."?': 'Yes',
'Does the word "series" mean the same thing in sentences "The Masterworks concert series." and "A comedy series."?': 'Yes',
'Does the word "deify" mean the same thing in sentences "These young men deify financial success." and "The people deified their King."?': 'No',
'Does the word "signal" mean the same thing in sentences "The victory was a signal for wild celebration." and "He awaited the signal to start."?': 'Yes',
'Does the word "automation" mean the same thing in sentences "This factory floor is a showcase for automation and robotic equipment." and "Automation replaces human workers by machines."?': 'No',
'Does the word "fire" mean the same thing in sentences "The gun has only been fired two times." and "The boss fired his secretary today."?': 'No',
'Does the word "goal" mean the same thing in sentences "The winning goal came with less than a minute left to play." and "She failed in her goal to become captain of the team."?': 'No',
'Does the word "register" mean the same thing in sentences "Did this event register in your parents\' minds ?" and "Did you register any change when I pressed the button ?"?': 'No',
'Does the word "scrap" mean the same thing in sentences "Scrap the old airplane and sell the parts." and "These two fellows are always scrapping over something."?': 'No',
'Does the word "protocol" mean the same thing in sentences "Academic protocol." and "Safety protocols."?': 'Yes',
'Does the word "master" mean the same thing in sentences "The methods can master the problems." and "Her pain completely mastered her."?': 'Yes',
'Does the word "scram" mean the same thing in sentences "If you do n\'t scram, I\'ll leave instead !" and "Scram !"?': 'Yes',
'Does the word "veil" mean the same thing in sentences "Women in Afghanistan veil their faces." and "The forest fire was veiled by smoke, but I could hear it clearly."?': 'Yes',
'Does the word "pitcher" mean the same thing in sentences "Our pitcher has a sore arm." and "He has played every position except pitcher."?': 'No',
'Does the word "whang" mean the same thing in sentences "Whang away at the school reform plan." and "Whang the ball."?': 'No',
'Does the word "predecease" mean the same thing in sentences "Husbands usually predecease their wives." and "She predeceased her husband."?': 'Yes',
'Does the word "bare" mean the same thing in sentences "Bare your breasts." and "Bare your feelings."?': 'Yes',
'Does the word "pickup" mean the same thing in sentences "His car has a lot of pickup." and "Put out a pickup on that man."?': 'No',
'Does the word "gimmick" mean the same thing in sentences "It was a great sales gimmick." and "A cheap promotions gimmick for greedy businessmen."?': 'Yes',
'Does the word "restoration" mean the same thing in sentences "The restoration of this painting will take years." and "The restoration of this medieval church involved undoing all the Victorian modifications."?': 'Yes',
'Does the word "batch" mean the same thing in sentences "We made a batch of cookies to take to the party." and "A batch of letters."?': 'No',
'Does the word "focus" mean the same thing in sentences "In focus." and "The focus of infection."?': 'No',
'Does the word "socialize" mean the same thing in sentences "The old man hates to socialize." and "He never socializes with his colleagues."?': 'Yes',
'Does the word "respect" mean the same thing in sentences "Syngman Rhee kept imprisoned the Dowager Queen Yun Empress Sunjeong of the Korean Empire for fear of the respect the people held for her." and "He is an intellectual giant, and I have great respect for him."?': 'Yes',
'Does the word "rekindle" mean the same thing in sentences "Rekindle her love." and "Rekindle hopes."?': 'Yes',
'Does the word "clean" mean the same thing in sentences "Can you clean the windows today ?" and "Clean the stove !"?': 'Yes',
'Does the word "electricity" mean the same thing in sentences "They built a car that runs on electricity." and "The stage crackled with electricity whenever she was on it."?': 'No',
'Does the word "season" mean the same thing in sentences "This trip will season even the hardiest traveller." and "Season the chicken breast after roasting it."?': 'No',
'Does the word "brand" mean the same thing in sentences "There\'s a new brand of hero in the movies now." and "Whenever Sally drinks her favorite beer, Coors\' brand image floods her senses with scenes of being on a hot beach drinking something refreshing."?': 'No',
'Does the word "plague" mean the same thing in sentences "A plague of grasshoppers." and "Those children are a damn plague."?': 'No',
'Does the word "draw" mean the same thing in sentences "Draw water from a well." and "She drew water from the barrel."?': 'Yes',
'Does the word "hearth" mean the same thing in sentences "They sat on the hearth and warmed themselves before the fire." and "The hearth was black with the charcoal of many fires."?': 'No',
'Does the word "makeup" mean the same thing in sentences "She is wearing a lot of makeup." and "The two teams played a makeup one week later."?': 'No',
'Does the word "guard" mean the same thing in sentences "The prison guard unlocked the door of the cell." and "We let our guard down."?': 'No',
'Does the word "ring" mean the same thing in sentences "I\'ll give you a ring when the plane lands." and "She had rings on every finger."?': 'No',
'Does the word "home" mean the same thing in sentences "A home for outcasts." and "A home for the elderly."?': 'Yes',
'Does the word "string" mean the same thing in sentences "The child drew glass beads on a string." and "String pearls on a string."?': 'Yes',
'Does the word "investment" mean the same thing in sentences "This job calls for the investment of some hard thinking." and "He made an emotional investment in the work."?': 'Yes',
'Does the word "racket" mean the same thing in sentences "What\'s all this racket ?" and "Power tools work quickly, but they sure make a racket."?': 'Yes',
'Does the word "susurrus" mean the same thing in sentences "A soft susurrus of conversation." and "The soft susurrus and sighs of the branches. — Longfellow."?': 'Yes',
'Does the word "process" mean the same thing in sentences "Process hair." and "Process cheese."?': 'Yes',
'Does the word "edge" mean the same thing in sentences "His voice had an edge to it." and "He rounded the edges of the box."?': 'No',
'Does the word "glory" mean the same thing in sentences "The glory of the sunrise." and "He valued glory above life itself."?': 'No',
'Does the word "direction" mean the same thing in sentences "They proposed a new direction for the firm." and "He looked the other direction."?': 'No',
'Does the word "truss" mean the same thing in sentences "Truss the roofs." and "Trussed bridges."?': 'Yes',
'Does the word "refer" mean the same thing in sentences "Refer a patient to a specialist." and "This plant can be referred to a known species."?': 'No',
'Does the word "bitch" mean the same thing in sentences "Karma\'s a bitch." and "This problem is a real bitch."?': 'Yes',
'Does the word "loss" mean the same thing in sentences "Loss of livestock left the rancher bankrupt." and "The car was a total loss."?': 'Yes',
'Does the word "conflict" mean the same thing in sentences "The two proposals conflict !" and "This behavior conflicts with our rules."?': 'No',
'Does the word "wax" mean the same thing in sentences "Wax the car." and "The moon is waxing."?': 'No',
'Does the word "kilter" mean the same thing in sentences "In good kilter." and "Out of kilter."?': 'Yes',
'Does the word "pop" mean the same thing in sentences "Pop the clutch." and "This popcorn pops quickly in the microwave oven."?': 'No',
'Does the word "stimulate" mean the same thing in sentences "Stimulate my appetite." and "The book stimulated her imagination."?': 'No',
'Does the word "corrupt" mean the same thing in sentences "Corrupt the morals." and "Socrates was accused of corrupting young men."?': 'Yes',
'Does the word "program" mean the same thing in sentences "Working mothers rely on the day care program." and "You ca n\'t tell the players without a program."?': 'No',
'Does the word "vamp" mean the same thing in sentences "Vamp up an excuse for not attending the meeting." and "Vamp up an old speech."?': 'No',
'Does the word "come" mean the same thing in sentences "Nothing good will come of this." and "This kind of accident comes when you are careless."?': 'Yes',
'Does the word "crush" mean the same thing in sentences "Crush an aluminum can." and "Ye shall not offer unto the Lord that which is bruised, crushed, broken or cut. --Lev. xxii."?': 'Yes',
'Does the word "agony" mean the same thing in sentences "An agony of doubt." and "Being in an agony he prayed more earnestly. —Luke xxii. 44."?': 'Yes',
'Does the word "extent" mean the same thing in sentences "The full extent of the law." and "The extent of his knowledge of the language is a few scattered words."?': 'No',
'Does the word "room" mean the same thing in sentences "Room for improvement." and "The rooms were very small but they had a nice view."?': 'No',
'Does the word "expression" mean the same thing in sentences "The expression of milk from her breast." and "A sad expression."?': 'No',
'Does the word "ticker" mean the same thing in sentences "I checked the prices on the ticker one last time before placing the trade." and "To my surprise, the ticker showed that the deal had already gone through."?': 'Yes',
'Does the word "half" mean the same thing in sentences "Half an hour." and "You do n\'t know the half of it."?': 'Yes',
'Does the word "consideration" mean the same thing in sentences "He showed no consideration for her feelings." and "After much consideration, I have decided to stay."?': 'No',
'Does the word "walk" mean the same thing in sentences "I\'ll walk you to your car." and "I walk the dog every morning."?': 'Yes',
'Does the word "height" mean the same thing in sentences "She\'s at the height of her career." and "At the height of her career."?': 'Yes',
'Does the word "obstruct" mean the same thing in sentences "Obstruct the path." and "The trees obstruct my view of the mountains."?': 'No',
'Does the word "disposition" mean the same thing in sentences "She has a sunny disposition." and "He has such a foul disposition."?': 'Yes',
'Does the word "suit" mean the same thing in sentences "What suit is trumps ?" and "To deal and shuffle, to divide and sort Her mingled suits and sequences. —."?': 'Yes',
'Does the word "laughter" mean the same thing in sentences "He enjoyed the laughter of the crowd." and "Their loud laughter betrayed their presence."?': 'No',
'Does the word "manhood" mean the same thing in sentences "He feared the speedy decline of all manhood." and "A strapping youth poised on the brink of manhood."?': 'No',
'Does the word "fertility" mean the same thing in sentences "In most industrialized countries the fertility rate has dropped well below replacement rate." and "In America the fertility rate of recent immigrants tends to be higher than that of native population."?': 'Yes',
'Does the word "stun" mean the same thing in sentences "Stun fish." and "The news stunned her."?': 'No',
'Does the word "joke" mean the same thing in sentences "I regarded his campaign for mayor as a joke." and "He told a very funny joke."?': 'No',
'Does the word "function" mean the same thing in sentences "It was a black-tie function." and "A seemingly endless round of social functions."?': 'No',
'Does the word "stem" mean the same thing in sentences "Stem the tide." and "To stem a tide."?': 'Yes',
'Does the word "agency" mean the same thing in sentences "The agency of providence." and "Agency for Toxic Substances and Disease Registry."?': 'No',
'Does the word "club" mean the same thing in sentences "He felt as if he had been hit with a club." and "He carried a club in self defense."?': 'Yes',
'Does the word "half" mean the same thing in sentences "Half a loaf." and "A century and one half."?': 'Yes',
'Does the word "batter" mean the same thing in sentences "The first batter hit the ball into the corner for a double." and "To the dismay of his mother, the boy put his finger into the cake batter."?': 'No',
'Does the word "revival" mean the same thing in sentences "A revival of a neglected play by Moliere." and "The revival of trade."?': 'Yes',
'Does the word "recognition" mean the same thing in sentences "He looked at her for ten full minutes before recognition dawned." and "He was unable to make his motion because he could n\'t get recognition by the chairman."?': 'No',
'Does the word "splash" mean the same thing in sentences "I heard a splash when the rock landed in the pond." and "The painter put a splash of blue on the wall to make it more colorful."?': 'No',
'Does the word "prefer" mean the same thing in sentences "We prefer sleeping outside." and "Some people prefer camping to staying in hotels."?': 'Yes',
'Does the word "expense" mean the same thing in sentences "At the expense of." and "He kept a careful record of his expenses at the meeting."?': 'No',
'Does the word "alphabet" mean the same thing in sentences "In the first year of school, pupils are taught to recite the alphabet." and "The Greek alphabet has only twenty-four letters."?': 'Yes',
'Does the word "guy" mean the same thing in sentences "A nice guy." and "The guy\'s only doing it for some doll."?': 'Yes',
'Does the word "categorize" mean the same thing in sentences "Children learn early on to categorize." and "First, categorize incoming messages according to the needed actions."?': 'Yes',
'Does the word "step" mean the same thing in sentences "It is but a step." and "It\'s only a step to the drugstore."?': 'Yes',
'Does the word "fulfil" mean the same thing in sentences "You made a promise, son, and now you must fulfil it." and "My lady is positively fulfilled of grace."?': 'No',
'Does the word "reshuffle" mean the same thing in sentences "So as to prevent cheating, he was asked to reshuffle the cards." and "The new Prime Minister reshuffled his cabinet."?': 'No',
'Does the word "vent" mean the same thing in sentences "He vented her frustration when he got home." and "She vented her anger."?': 'Yes',
'Does the word "girlfriend" mean the same thing in sentences "His girlfriend kicked him out." and "Marc went to the park with his girlfriend and watched the sunset with her."?': 'Yes',
'Does the word "aroma" mean the same thing in sentences "I love the aroma of cinnamon." and "The toilet was emitting a pungent, foul aroma."?': 'Yes',
'Does the word "inclination" mean the same thing in sentences "Artillery must take account of a weapon\'s precise inclination." and "He walked with a heavy inclination to the right."?': 'No',
'Does the word "organization" mean the same thing in sentences "This painting shows little organization at first glance, but little by little the structure becomes clear." and "His organization of the work force was very efficient."?': 'No',
'Does the word "record" mean the same thing in sentences "The track record shows that he will be a good president." and "The lawyer has a good record."?': 'Yes',
'Does the word "immunity" mean the same thing in sentences "Some people have better immunity to diseases than others." and "Immunity to criticism."?': 'No',
'Does the word "shrink" mean the same thing in sentences "Hot water will shrink the sweater." and "Can you shrink this image ?"?': 'Yes',
'Does the word "harvest" mean the same thing in sentences "The Chinese are said to harvest organs from executed criminals." and "Harvest the grapes."?': 'No',
'Does the word "washout" mean the same thing in sentences "It was several days after the storm before they could repair the washout and open the road." and "As an actor, he was a complete washout, so he went back to accounting."?': 'No',
'Does the word "land" mean the same thing in sentences "The land had never been plowed." and "Good or bad land for growing potatoes."?': 'Yes',
'Does the word "day" mean the same thing in sentences "Every dog has his day." and "He deserves his day in court."?': 'Yes',
'Does the word "salvation" mean the same thing in sentences "In Islam, salvation is achieved by praying in the mosque to Allah and following the sunnah of Muhammad." and "The salvation of his party was the president\'s major concern."?': 'Yes',
'Does the word "service" mean the same thing in sentences "The service happened yesterday." and "He accepted service of the subpoena."?': 'Yes',
'Does the word "throw" mean the same thing in sentences "Throw accusations at someone." and "She threw the blanket around the child."?': 'No',
'Does the word "clap" mean the same thing in sentences "Clap two boards together." and "The wings of the birds clapped loudly."?': 'No',
'Does the word "denier" mean the same thing in sentences "Global warming denier ( see : Wikipedia : Global warming denial )." and "Holocaust denier ( see : Wikipedia : Holocaust denial )."?': 'Yes',
'Does the word "carry" mean the same thing in sentences "To carry the war from Greece into Asia." and "This crime carries a penalty of five years in prison."?': 'No',
'Does the word "separatism" mean the same thing in sentences "Demands for some form of separatism on grounds of religion have been perceived as a threat to mainstream education." and "Separatism is a serious problem in Quebec."?': 'Yes',
'Does the word "good" mean the same thing in sentences "Weigh the good against the bad." and "Among the highest goods of all are happiness and self-realization."?': 'Yes',
'Does the word "bail" mean the same thing in sentences "He is out on bail." and "The judge set bail at $ 10,000."?': 'No',
'Does the word "circuit" mean the same thing in sentences "A ten-day coach circuit of the island." and "We took a quick circuit of the park."?': 'Yes',
'Does the word "spreader" mean the same thing in sentences "Ellen used a spreader to butter her bagel." and "He used his knife as a spreader."?': 'Yes',
'Does the word "setup" mean the same thing in sentences "It takes time to learn the setup around here." and "The classroom setup was simple and efficient."?': 'Yes',
'Does the word "product" mean the same thing in sentences "This is a product of lime and nitric acid." and "A product of lime and nitric acid."?': 'Yes',
'Does the word "trail" mean the same thing in sentences "You\'ll get your coat all muddy if you trail it around like that." and "The bride\'s veiled trailed along the ground."?': 'Yes',
'Does the word "walk" mean the same thing in sentences "He had a funny walk." and "The pitcher now has two walks in this inning alone."?': 'No',
'Does the word "gang" mean the same thing in sentences "Do a drop for the telephone gang through the ceiling of the wiring closet." and "A gang of wires."?': 'Yes',
'Does the word "claimant" mean the same thing in sentences "He was a claimant to the throne." and "Claimants of unemployment compensation."?': 'Yes',
'Does the word "avenue" mean the same thing in sentences "They explored every avenue they could think of." and "It promises to open new avenues to understanding."?': 'Yes',
'Does the word "mellow" mean the same thing in sentences "These apples need to mellow a bit more." and "The sun mellowed the fruit."?': 'Yes',
'Does the word "draw" mean the same thing in sentences "He played only draw and stud." and "John only had four cards of the same suit ; he had a drawing hand."?': 'Yes',
'Does the word "control" mean the same thing in sentences "Measures for the control of disease." and "They instituted controls over drinking on campus."?': 'Yes',
'Does the word "variation" mean the same thing in sentences "An emery wheel is the modern variation of a grindstone." and "Any variation in his routine was immediately reported."?': 'No',
'Does the word "amount" mean the same thing in sentences "The amount he had in cash was insufficient." and "The dogs need different amounts of food."?': 'Yes',
'Does the word "man" mean the same thing in sentences "We can not man all the desks." and "Students were manning the booths."?': 'Yes',
'Does the word "residence" mean the same thing in sentences "He refused to live in the governor\'s residence." and "A person can have several residences."?': 'No',
'Does the word "compel" mean the same thing in sentences "We compel all students to fill out this form." and "Logic compels the wise, while fools feel compelled by emotions."?': 'Yes',
'Does the word "eclat" mean the same thing in sentences "The eclat of a great achievement." and "They gave him more eclat than he really deserved."?': 'No',
'Does the word "slave" mean the same thing in sentences "A slave to fashion." and "A slave to cocaine."?': 'Yes',
'Does the word "overreact" mean the same thing in sentences "Please try not to overreact if she drives badly when she is first learning." and "Do n\'t overreact to the bad news -- take it easy."?': 'Yes',
'Does the word "zinger" mean the same thing in sentences "She tried to think of some killer of an argument, a real zinger that would disarm all opposition." and "He always greeted me with a new zinger."?': 'Yes',
'Does the word "tread" mean the same thing in sentences "To tread land when too light." and "For fools rush in where angels fear to tread."?': 'Yes',
'Does the word "lay" mean the same thing in sentences "To lay a body in the grave." and "Lay the books on the table."?': 'Yes',
'Does the word "softening" mean the same thing in sentences "Refrigeration delayed the softening of the fruit." and "He observed the softening of iron by heat."?': 'Yes',
'Does the word "drink" mean the same thing in sentences "You can lead a horse to water but you ca n\'t make him drink." and "Cynthia drank the wine."?': 'Yes',
'Does the word "squeamishness" mean the same thing in sentences "The program was withdrawn because of the squeamishness of some viewers." and "He refused to allow squeamishness to deter him from his duty."?': 'Yes',
'Does the word "date" mean the same thing in sentences "You may need that at a later date." and "They hoped to get together at an early date."?': 'Yes',
'Does the word "oat" mean the same thing in sentences "The main forms of oat are meal and bran." and "The oat stalks made good straw."?': 'Yes',
'Does the word "community" mean the same thing in sentences "They shared a community of possessions." and "A community is infinitely more brutalised by the habitual employment of punishment than it is by the occasional occurrence of crime ( Oscar Wilde )."?': 'No',
'Does the word "period" mean the same thing in sentences "A change soon put a period to my tranquility." and "Death put a period to his endeavors."?': 'Yes',
'Does the word "unlock" mean the same thing in sentences "Unlock the door." and "I unlocked the door and walked in."?': 'Yes',
'Does the word "fit" mean the same thing in sentences "This piece wo n\'t fit into the puzzle." and "I wanted to borrow my little sister\'s jeans, but they did n\'t fit."?': 'Yes',
'Does the word "bed" mean the same thing in sentences "The room had only a bed and chair." and "He sat on the edge of the bed."?': 'Yes',
'Does the word "have" mean the same thing in sentences "Have a postdoc." and "She has $ 1,000 in the bank."?': 'No',
'Does the word "resolve" mean the same thing in sentences "I\'ll have to resolve the equation with the new values." and "Can this image be resolved ?"?': 'No',
'Does the word "reading" mean the same thing in sentences "He bought some reading material at the airport." and "The teacher assigned new readings."?': 'Yes',
'Does the word "hope" mean the same thing in sentences "In spite of his troubles he never gave up hope." and "He was their best hope for a victory."?': 'No',
'Does the word "digress" mean the same thing in sentences "Do n\'t digress when you give a lecture." and "She always digresses when telling a story."?': 'Yes',
'Does the word "audience" mean the same thing in sentences "Someone in the audience began to cough." and "We joined the audience just as the lights went down."?': 'Yes',
'Does the word "care" mean the same thing in sentences "Do you care to try this dish ?" and "The nurse was caring for the wounded."?': 'No',
'Does the word "head" mean the same thing in sentences "Head the fish." and "The salmon are first headed and then scaled."?': 'Yes',
'Does the word "tease" mean the same thing in sentences "Tease tissue for microscopic examinations." and "She has a way of teasing men with her flirtatious behavior."?': 'No',
'Does the word "file" mean the same thing in sentences "File one\'s fingernails." and "File these bills, please."?': 'No',
'Does the word "smoking" mean the same thing in sentences "Smoking stinks." and "Smoking is permitted."?': 'Yes',
'Does the word "entrance" mean the same thing in sentences "Her entrance attracted no attention whatsoever." and "She made a grand entrance."?': 'Yes',
'Does the word "game" mean the same thing in sentences "He is serving for the game." and "The game lasted two hours."?': 'No',
'Does the word "gravity" mean the same thing in sentences "The more remote the body the less the gravity." and "Gravity is weaker than electromagnetism, but affects all matter and energy."?': 'Yes',
'Does the word "scoop" mean the same thing in sentences "He got a scoop on the bribery of city officials." and "He used a scoop to serve the ice cream."?': 'No',
'Does the word "determination" mean the same thing in sentences "The determination of molecular structures." and "His determination showed in his every movement."?': 'No',
'Does the word "stay" mean the same thing in sentences "You can stay with me while you are in town." and "Stay a bit longer -- the day is still young."?': 'Yes',
'Does the word "sustain" mean the same thing in sentences "The money will sustain our good cause." and "She alone sustained her family."?': 'Yes',
'Does the word "contour" mean the same thing in sentences "It defines a major contour of this administration." and "The contours of the melody."?': 'Yes',
'Does the word "child" mean the same thing in sentences "My youngest child is forty-three." and "They had three children."?': 'Yes',
'Does the word "elaborate" mean the same thing in sentences "Elaborate a plan." and "Could you elaborate on the plot for your novel for me ?"?': 'Yes',
'Does the word "cradle" mean the same thing in sentences "Cradle gold." and "He cradled the infant in his arms."?': 'No',
'Does the word "endorsement" mean the same thing in sentences "Companies sometimes pay millions for product endorsement by celebrities." and "A star athlete can make a lot of money from endorsements."?': 'Yes',
'Does the word "vascularity" mean the same thing in sentences "A prominent vascularity." and "Reduced bodyfat brought out the vascularity in his arms."?': 'Yes',
'Does the word "grey" mean the same thing in sentences "He was dressed in grey." and "The Confederate army was a vast grey."?': 'No',
'Does the word "interest" mean the same thing in sentences "He has a lot of interest in vintage cars." and "How much interest do you pay on your mortgage ?"?': 'No',
'Does the word "inhale" mean the same thing in sentences "Inhale the fresh mountain air." and "Inhale deeply."?': 'Yes',
'Does the word "symbol" mean the same thing in sentences "The eagle is a symbol of the United States." and "Chinese people use word symbols for writing."?': 'No',
'Does the word "mound" mean the same thing in sentences "They have a southpaw on the mound." and "They built small mounds to hide behind."?': 'No',
'Does the word "genre" mean the same thing in sentences "The still-life has been a popular genre in painting since the 17th century." and "The computer game Half-Life redefined the first-person shooter genre."?': 'Yes',
'Does the word "train" mean the same thing in sentences "Train of mourners." and "Express trains do n\'t stop at Princeton Junction."?': 'No',
'Does the word "focus" mean the same thing in sentences "The controversy brought clearly into focus an important difference of opinion." and "I believe I can bring the high degree of focus required for this important job."?': 'No',
'Does the word "procurement" mean the same thing in sentences "He was responsible for the procurement of materials and supplies." and "I have a lot of experience in the procurement of construction materials and sub-contracts."?': 'Yes',
'Does the word "mediate" mean the same thing in sentences "Mediate between the old and the new." and "He mediated a settlement."?': 'No',
'Does the word "bring" mean the same thing in sentences "Bring comments." and "Bring water to the boiling point."?': 'No',
'Does the word "stage" mean the same thing in sentences "All the world\'s a stage -- Shakespeare." and "The stage pulled into town carrying the payroll for the mill and three ladies."?': 'No',
'Does the word "account" mean the same thing in sentences "Passing grades account for half of the grades given in this exam." and "I ca n\'t account for the missing money."?': 'No',
'Does the word "pluck" mean the same thing in sentences "To pluck at somebody\'s sleeve." and "He plucked the strings of his mandolin."?': 'Yes',
'Does the word "lighten" mean the same thing in sentences "Lighten the burden of caring for her elderly parents." and "She lightened the load on the tired donkey."?': 'No',
'Does the word "chip" mean the same thing in sentences "This cup has a chip in it." and "Potato chip, tortilla chip."?': 'No',
'Does the word "evidence" mean the same thing in sentences "His trembling was evidence of his fear." and "There is no evidence that anyone was here earlier."?': 'No',
'Does the word "yogi" mean the same thing in sentences "Gurdjieff connects this type of breathing with yogi breathing." and "It\'s a yogi trick of some sort."?': 'Yes',
'Does the word "wonder" mean the same thing in sentences "I wonder whether this was the right thing to do." and "She wondered whether it would snow tonight."?': 'Yes',
'Does the word "colony" mean the same thing in sentences "The American colony in Paris." and "An artists\' colony."?': 'No',
'Does the word "grain" mean the same thing in sentences "A grain of sugar." and "Wheat is a grain that is grown in Kansas."?': 'No',
'Does the word "rudeness" mean the same thing in sentences "His rudeness was inexcusable." and "The rudeness of the cabin gave it a rustic charm but little comfort."?': 'Yes',
'Does the word "stick" mean the same thing in sentences "The policy so far is all stick and no carrot." and "Cigarettes are taxed at one dollar per stick."?': 'No',
'Does the word "bench" mean the same thing in sentences "Injuries have shortened the bench." and "She placed the workpiece on the bench, inspected it closely, and opened the cover."?': 'No',
'Does the word "cap" mean the same thing in sentences "We should put a cap on the salaries, to keep them under control." and "They established a cap for prices."?': 'Yes',
'Does the word "umbrella" mean the same thing in sentences "Quick, grab that umbrella before you get rained on !" and "An air umbrella over England."?': 'No',
'Does the word "seal" mean the same thing in sentences "A coat of seal." and "Close the lid tightly to get a good seal."?': 'No',
'Does the word "stir" mean the same thing in sentences "Stir the soup." and "Stir the soil."?': 'Yes',
'Does the word "issue" mean the same thing in sentences "Issue a new uniform to the children." and "Water issued from the hole in the wall."?': 'No',
'Does the word "subject" mean the same thing in sentences "He did not want to subject himself to the judgments of his superiors." and "The sergeant subjected the new recruits to many drills."?': 'No',
'Does the word "repression" mean the same thing in sentences "His goal was the repression of insolence." and "The long repression of Christian sects."?': 'No',
'Does the word "property" mean the same thing in sentences "Self-confidence is not an endearing property." and "The president was concerned about the property across from the White House."?': 'No',
'Does the word "flip" mean the same thing in sentences "I do n\'t know what to do -- I may as well flip a coin !" and "Flip me the beachball."?': 'No',
'Does the word "support" mean the same thing in sentences "The foundation provided support for the experiment." and "The policy found little public support."?': 'No',
'Does the word "charge" mean the same thing in sentences "He was arrested on a charge of larceny." and "That\'s a slanderous charge of abuse of trust."?': 'Yes',
'Does the word "absorption" mean the same thing in sentences "The absorption of light, heat, electricity, etc." and "The absorption of photons by atoms or molecules."?': 'Yes',
'Does the word "puddle" mean the same thing in sentences "His chair sat in a puddle of books and magazines." and "There were puddles of muddy water in the road after the rain."?': 'No',
'Does the word "push" mean the same thing in sentences "Give the door a hard push if it sticks." and "Let\'s give one last push on our advertising campaign."?': 'No',
'Does the word "ply" mean the same thing in sentences "Ships ply the waters near the coast." and "He plied his ax with bloody results."?': 'No',
'Does the word "slot" mean the same thing in sentences "He followed the deer\'s slot over the soft turf to the edge of the trees." and "The TV program has a new time slot."?': 'No',
'Does the word "recapitulate" mean the same thing in sentences "Let\'s recapitulate the main ideas." and "Ontogeny recapitulates phylogeny."?': 'No',
'Does the word "slope" mean the same thing in sentences "He climbed the steep slope." and "The slope of a parabola increases linearly with x."?': 'No',
'Does the word "tap" mean the same thing in sentences "Tap beer in a bar." and "He was tapping his fingers on the table impatiently."?': 'No',
'Does the word "simmer" mean the same thing in sentences "Simmer the sauce." and "Simmering water."?': 'Yes',
'Does the word "past" mean the same thing in sentences "Past tense." and "Reporters dug into the candidate\'s past."?': 'No',
'Does the word "mark" mean the same thing in sentences "Mark homework." and "Mark my words."?': 'No',
'Does the word "hide" mean the same thing in sentences "Hide the money." and "Muslim women hide their faces."?': 'Yes',
'Does the word "suggestion" mean the same thing in sentences "It was a suggestion we could n\'t refuse." and "Suggestion often works better than explicit demand."?': 'No',
'Does the word "consolidate" mean the same thing in sentences "Consolidate one\'s hold on first place." and "Consolidate one\'s gains."?': 'Yes',
'Does the word "remember" mean the same thing in sentences "After the shelling, many people lost the ability to remember." and "Some remember better than others."?': 'Yes',
'Does the word "instill" mean the same thing in sentences "Instill medication into my eye." and "Her presence instilled faith into the children."?': 'No',
'Does the word "curl" mean the same thing in sentences "Curl my hair, please." and "She curled farther down under the covers."?': 'No',
'Does the word "holder" mean the same thing in sentences "Put your umbrella in the umbrella holder." and "A towel holder."?': 'No',
'Does the word "news" mean the same thing in sentences "We watch the 7 o\'clock news every night." and "Did you listen to the news tonight ?"?': 'Yes',
'Does the word "acknowledge" mean the same thing in sentences "In all thy ways acknowledge Him. - Proverbs 3:6." and "She acknowledged his complement with a smile."?': 'No',
'Does the word "confidence" mean the same thing in sentences "He took me into his confidence." and "Confidence is always borrowed, never owned."?': 'No',
'Does the word "unload" mean the same thing in sentences "To unload a ship." and "Unload the truck."?': 'Yes',
'Does the word "gentrify" mean the same thing in sentences "Gentrify the old center of town." and "Gentrify a row of old houses."?': 'Yes',
'Does the word "pastime" mean the same thing in sentences "Sailing is her favorite pastime." and "His main pastime is gambling."?': 'Yes',
'Does the word "loop" mean the same thing in sentences "Loop a rope." and "The bicycle looped around the tree."?': 'No',
'Does the word "grind" mean the same thing in sentences "A coarse grind of coffee." and "This bag contains espresso grind."?': 'Yes',
'Does the word "blow" mean the same thing in sentences "Certain insects are said to blow." and "He blew a lot of money on his new home theater."?': 'No',
'Does the word "function" mean the same thing in sentences "The function of a teacher." and "The function of an auger is to bore holes."?': 'No',
'Does the word "shape" mean the same thing in sentences "He cut a square shape out of the cake." and "The vet checked to see what kind of shape the animal was in."?': 'No',
'Does the word "overcome" mean the same thing in sentences "You must overcome all difficulties." and "He overcame his infirmity."?': 'Yes',
'Does the word "prime" mean the same thing in sentences "I need to prime these handrails before we can apply the finish coat." and "Prime a cannon."?': 'No',
'Does the word "figure" mean the same thing in sentences "He did n\'t figure her." and "I ca n\'t figure if he\'s telling the truth or lying."?': 'Yes',
'Does the word "fall" mean the same thing in sentences "The rise and fall of the tides." and "A fall from virtue."?': 'No',
'Does the word "intimidate" mean the same thing in sentences "He\'s trying to intimidate you. If you ignore him, hopefully he\'ll stop." and "Her boss intimidates her."?': 'Yes',
'Does the word "chairperson" mean the same thing in sentences "She was the chairperson of the board and she presided over the meeting." and "Address your remarks to the chairperson."?': 'Yes',
'Does the word "pour" mean the same thing in sentences "Pour water over the floor." and "Water poured onto the plants."?': 'Yes',
'Does the word "play" mean the same thing in sentences "Play a hooked fish." and "She played nervously with her wedding ring."?': 'No',
'Does the word "bootleg" mean the same thing in sentences "Bootleg tapes of the diva\'s singing." and "They were bootlegging whiskey."?': 'No',
'Does the word "drip" mean the same thing in sentences "I put a drip of vanilla extract in my hot cocoa." and "The constant sound of dripping irritated him."?': 'Yes',
'Does the word "plan" mean the same thing in sentences "Plan the new wing of the museum." and "The rebels had planned turmoil and confusion."?': 'No',
'Does the word "knead" mean the same thing in sentences "Knead dough." and "I kneaded the dough into a loaf."?': 'Yes',
'Does the word "commitment" mean the same thing in sentences "A man of energy and commitment." and "His business commitments took him to London."?': 'No',
'Does the word "stop" mean the same thing in sentences "It came to a stop at the bottom of the hill." and "The third baseman made some remarkable stops."?': 'No',
'Does the word "conduct" mean the same thing in sentences "Conduct an orchestra." and "He conducted us to the palace."?': 'No',
'Does the word "cord" mean the same thing in sentences "The bundle was tied with a cord." and "The burglar tied up the victim with a cord."?': 'Yes',
'Does the word "complement" mean the same thing in sentences "The complement of 01100101_2 is 10011011_2." and "A full complement of workers."?': 'Yes',
'Does the word "pig" mean the same thing in sentences "He was a pig and his apartment a pigpen ; take-away containers and pizza boxes in a long, moldy stream lined his counter tops." and "The farmer kept a pen with two pigs that he fed from table scraps and field waste."?': 'No',
'Does the word "complaint" mean the same thing in sentences "Do n\'t come too close, I\'ve got this nasty complaint." and "I have no complaints about the quality of his work, but I do n\'t enjoy his company."?': 'No',
'Does the word "house" mean the same thing in sentences "The house gets a percentage of every bet." and "They had a large carriage house."?': 'No',
'Does the word "trial" mean the same thing in sentences "He had a fair trial and the jury found him guilty." and "Most of these complaints are settled before they go to trial."?': 'Yes',
'Does the word "pull" mean the same thing in sentences "The pull up the hill had him breathing harder." and "She took a pull on her cigarette."?': 'No',
'Does the word "finger" mean the same thing in sentences "Human hands have five fingers : the thumb, the forefinger ( or index finger ), the middle finger, the ring finger and the little finger." and "Her fingers were long and thin."?': 'Yes',
'Does the word "record" mean the same thing in sentences "Chicago set the homicide record." and "Coffee production last year broke all previous records."?': 'Yes',
'Does the word "moderate" mean the same thing in sentences "Moderate your alcohol intake." and "Moderate your speed."?': 'No',
'Does the word "curve" mean the same thing in sentences "She scribbled a curve on the paper." and "You should slow down when approaching a curve."?': 'No',
'Does the word "potentiate" mean the same thing in sentences "These anti-depressants potentiate the analgesic action of opiates." and "Potentiate the drug."?': 'Yes',
'Does the word "circus" mean the same thing in sentences "They used the elephants to help put up the circus." and "The children always love to go to the circus."?': 'No',
'Does the word "matter" mean the same thing in sentences "It does not matter much." and "It matters that they left."?': 'Yes',
'Does the word "remount" mean the same thing in sentences "Remount a regiment." and "He remounted his horse."?': 'No',
'Does the word "existence" mean the same thing in sentences "The biggest tree in existence." and "Laws in existence for centuries."?': 'No',
'Does the word "offspring" mean the same thing in sentences "She was the mother of many offspring." and "Industrialism prepared the way for acceptance of the French Revolution\'s various socialistic offspring."?': 'No',
'Does the word "do" mean the same thing in sentences "Do research." and "Do nothing."?': 'Yes',
'Does the word "crispness" mean the same thing in sentences "The crispness of his reply." and "Crispness of new dollar bills."?': 'No',
'Does the word "eligibility" mean the same thing in sentences "Eligibility for a loan." and "Eligibility of a candidate for office."?': 'Yes',
'Does the word "possession" mean the same thing in sentences "The scoreboard shows a little football symbol next to the name of the team that has possession." and "Back then, people with psychiatric disorders were sometimes thought to be victims of demonic possession."?': 'No',
'Does the word "asthmatic" mean the same thing in sentences "The painful gasps of a dying asthmatic." and "She is a chronic asthmatic."?': 'Yes',
'Does the word "talk" mean the same thing in sentences "Actions talk louder than words." and "Susan talked with Rachel."?': 'Yes',
'Does the word "trust" mean the same thing in sentences "The experience destroyed his trust and personal dignity." and "He put more trust in his own two legs than in the gun."?': 'No',
'Does the word "refinement" mean the same thing in sentences "He described a refinement of this technique." and "A man of intellectual refinement."?': 'No',
'Does the word "bang" mean the same thing in sentences "The party went with a bang." and "Tiffany has long hair and bangs."?': 'No',
'Does the word "look" mean the same thing in sentences "Do n\'t look in the closet." and "Look at my new car !"?': 'Yes',
'Does the word "appeal" mean the same thing in sentences "An appeal for help." and "An appeal to the public to keep calm."?': 'Yes',
'Does the word "faddist" mean the same thing in sentences "He is such a fashion faddist." and "The faddist magazines have been hyping that lately."?': 'Yes',
'Does the word "feeling" mean the same thing in sentences "I had a feeling that she was lying." and "I disliked him and the feeling was mutual."?': 'No',
'Does the word "growth" mean the same thing in sentences "The growth of population." and "Growth was dampened by a softening of the global economy in 2001, but picked up in the subsequent years due to strong growth in China."?': 'Yes',
'Does the word "bother" mean the same thing in sentences "A spot of bother." and "A bit of a bother."?': 'No',
'Does the word "proportion" mean the same thing in sentences "An inordinate proportion of the book is given over to quotations." and "A building of vast proportions."?': 'No',
'Does the word "swing" mean the same thing in sentences "It took time to get into the swing of things." and "The party went with a swing."?': 'Yes',
'Does the word "insufficiency" mean the same thing in sentences "The troops went hungry because of the insufficiency of their supplies." and "Pointed out the insufficiencies in my report."?': 'No',
'Does the word "zap" mean the same thing in sentences "Zap the enemy." and "If it\'s not warm in the middle, zap it some more."?': 'No',
'Does the word "processing" mean the same thing in sentences "The processing of ore to obtain minerals." and "The processing of newly arrived immigrants."?': 'Yes',
'Does the word "mind" mean the same thing in sentences "I would n\'t mind an ice cream right now." and "I do n\'t mind your behavior."?': 'Yes',
'Does the word "resemble" mean the same thing in sentences "The twins resemble each other." and "This paper resembles my own work."?': 'Yes',
'Does the word "post" mean the same thing in sentences "A soldier manned the entrance post." and "In England they call mail ` the post\'."?': 'No',
'Does the word "couple" mean the same thing in sentences "He\'s coming for a couple of days." and "The engaged couple."?': 'No',
'Does the word "narrow" mean the same thing in sentences "I can not narrow down the rules for this game." and "The road narrowed."?': 'No',
'Does the word "tread" mean the same thing in sentences "To tread a path." and "She trod with care as the ground was slippery."?': 'Yes',
'Does the word "divest" mean the same thing in sentences "The company decided to divest." and "The board of trustees divested $ 20 million in real estate property."?': 'Yes',
'Does the word "cultivate" mean the same thing in sentences "Cultivate the land." and "Cultivate your musical taste."?': 'No',
'Does the word "communicate" mean the same thing in sentences "Please communicate this message to all employees." and "The disease was mainly communicated via rats and other vermin."?': 'No',
'Does the word "lash" mean the same thing in sentences "Lash the horse." and "To lash something to a spar."?': 'Yes',
'Does the word "carpet" mean the same thing in sentences "The larvae of some moths spin a web that resembles a carpet." and "A carpet of flowers."?': 'Yes',
'Does the word "gravitate" mean the same thing in sentences "Boys gravitate towards girls at that age." and "The stars gravitate towards each other."?': 'No',
'Does the word "knot" mean the same thing in sentences "The bird had a knot of feathers forming a crest." and "Their muscles stood out in knots."?': 'No',
'Does the word "wipe" mean the same thing in sentences "Wipe the blackboard." and "Brian wiped at the counter."?': 'Yes',
'Does the word "street" mean the same thing in sentences "He worked both sides of the street." and "They walked the streets of the small town."?': 'No',
'Does the word "look" mean the same thing in sentences "He gave me a dirty look." and "If looks could kill..."?': 'Yes',
'Does the word "presidency" mean the same thing in sentences "After many years as a party leader, she finally ascended to the presidency." and "Andrew Jackson expanded the power of the presidency beyond what was customary before his time."?': 'Yes',
'Does the word "den" mean the same thing in sentences "A den of vice." and "An opium den."?': 'Yes',
'Does the word "vision" mean the same thing in sentences "He had a vision of the Virgin Mary." and "He had a vision of his own death."?': 'No',
'Does the word "expedite" mean the same thing in sentences "This should expedite the process." and "He expedited the search by alphabetizing the papers."?': 'Yes',
'Does the word "form" mean the same thing in sentences "When icicles start to form on the eaves you know the roads will be icy." and "The water formed little beads."?': 'Yes',
'Does the word "freeze" mean the same thing in sentences "Do n\'t go outside wearing just a t-shirt ; you\'ll freeze !" and "It\'s freezing in here !"?': 'Yes',
'Does the word "circulate" mean the same thing in sentences "The air here does not circulate." and "Blood circulates in my veins."?': 'Yes',
'Does the word "revamp" mean the same thing in sentences "They plan to revamp the historical theater in the old downtown." and "They revamped their old house before selling it."?': 'Yes',
'Does the word "toke" mean the same thing in sentences "The artist took a thoughtful toke off the joint, then passed it along." and "The boys took a few tokes on a joint."?': 'Yes',
'Does the word "agree" mean the same thing in sentences "I ca n\'t agree with you !" and "They agreed about what should be done."?': 'Yes',
'Does the word "device" mean the same thing in sentences "The device is small enough to wear on your wrist." and "He would stoop to any device to win a point."?': 'No',
'Does the word "consensus" mean the same thing in sentences "The lack of consensus reflected differences in theoretical positions." and "Those rights and obligations are based on an unstated consensus."?': 'Yes',
'Does the word "aggravate" mean the same thing in sentences "To aggravate my woes. —Alexander Pope." and "The defense made by the prisoner\'s counsel did rather aggravate than extenuate his crime. —Addison."?': 'Yes',
'Does the word "knitting" mean the same thing in sentences "She put down her knitting and went to answer the phone." and "I find knitting very relaxing."?': 'No',
'Does the word "adhere" mean the same thing in sentences "Will this wallpaper adhere to the wall ?" and "The residents of this village adhered to Catholicism."?': 'No',
'Does the word "retain" mean the same thing in sentences "I retain this drug for a long time." and "The dam retains the water."?': 'Yes',
'Does the word "cry" mean the same thing in sentences "A cry to arms." and "I heard a cry from afar."?': 'Yes',
'Does the word "round" mean the same thing in sentences "Round the edges." and "Round the bend."?': 'No',
'Does the word "spring" mean the same thing in sentences "The spring was broken." and "The spring of a bow."?': 'No',
'Does the word "sprinkling" mean the same thing in sentences "A sprinkling of grey at his temples." and "Baptized with a sprinkling of holy water."?': 'No',
'Does the word "generation" mean the same thing in sentences "They had to wait a generation for that prejudice to fade." and "Dams were built for the generation of electricity."?': 'No',
'Does the word "model" mean the same thing in sentences "Model an airplane." and "Model the latest fashion."?': 'No',
'Does the word "pride" mean the same thing in sentences "He had pride of ownership in his department." and "He took pride in his work."?': 'Yes',
'Does the word "occupy" mean the same thing in sentences "The young prince will soon occupy the throne." and "He occupies the position of manager."?': 'Yes',
'Does the word "view" mean the same thing in sentences "If there are any rabbits in this park, they keep carefully out of our view." and "He took the computer with a view to pawning it."?': 'No',
'Does the word "baseline" mean the same thing in sentences "The ref missed the call. The ball hit the baseline." and "The established a baseline for the budget."?': 'No',
'Does the word "program" mean the same thing in sentences "He proposed an elaborate program of public works." and "The program lasted more than two hours."?': 'No',
'Does the word "manner" mean the same thing in sentences "All manner of persons participate." and "His rapid manner of talking."?': 'No',
'Does the word "crack" mean the same thing in sentences "The crack of the bat hitting the ball." and "The crack of a whip."?': 'Yes',
'Does the word "sweep" mean the same thing in sentences "Please sweep the floor." and "Her new show dog swept all championships."?': 'No',
'Does the word "overdrive" mean the same thing in sentences "Troops are ready to go into overdrive as soon as the signal is given." and "Melissa\'s brain was in overdrive."?': 'Yes',
'Does the word "moon" mean the same thing in sentences "The clock had a moon that showed various phases." and "He made a moon lamp that he used as a night light."?': 'Yes',
'Does the word "zero" mean the same thing in sentences "In unary and k - adic notation in general, zero is the empty string." and "Write 0.0 to indicate a floating point number rather than the integer zero."?': 'Yes',
'Does the word "deceleration" mean the same thing in sentences "The rocket is now in deceleration." and "He initiated deceleration by braking."?': 'Yes',
'Does the word "concoction" mean the same thing in sentences "He volunteered to taste her latest concoction." and "It suddenly spewed out a thick green concoction."?': 'No',
'Does the word "lifeline" mean the same thing in sentences "She offered me a lifeline in my time of grief." and "The airlift provided a lifeline for Berlin."?': 'Yes',
'Does the word "gospel" mean the same thing in sentences "His word was gospel." and "Newton\'s writings were gospel for those who followed."?': 'No',
'Does the word "failure" mean the same thing in sentences "The mechanic\'s failure to check the brakes." and "He resented my failure to return his call."?': 'Yes',
'Does the word "course" mean the same thing in sentences "The course of the river." and "We offer seafood as the first course."?': 'No',
'Does the word "note" mean the same thing in sentences "The singer held the note too long." and "I had to co-sign his note at the bank."?': 'No',
'Does the word "prepare" mean the same thing in sentences "Prepare a speech." and "Prepare a report."?': 'Yes',
'Does the word "array" mean the same thing in sentences "An array of troops in battle order." and "Drawn up in battle array."?': 'Yes',
'Does the word "buster" mean the same thing in sentences "Dam buster." and "Crime buster."?': 'Yes',
'Does the word "sell" mean the same thing in sentences "I\'ll sell you all three for a hundred dollars." and "He sold his house in January."?': 'Yes',
'Does the word "point" mean the same thing in sentences "A point is defined by its coordinates." and "The point of the arrow was due north."?': 'No',
'Does the word "nick" mean the same thing in sentences "Nick horses." and "I nicked myself while I was shaving."?': 'No',
'Does the word "butter" mean the same thing in sentences "Lora buttered the toast with unsalted butter." and "Lora buttered the toast."?': 'Yes',
'Does the word "print" mean the same thing in sentences "Using a crayon, the girl made a print of the leaf under the page." and "Paw prints were everywhere."?': 'Yes',
'Does the word "post" mean the same thing in sentences "Two of the receivers ran post patterns." and "He set a row of posts in the ground and strung barbwire between them."?': 'Yes',
'Does the word "crutch" mean the same thing in sentences "He uses drugs as a psychological crutch." and "He walked on crutches for a month until the cast was removed from his leg."?': 'No',
'Does the word "skip" mean the same thing in sentences "Skip town." and "He skipped a row in the text and so the sentence was incomprehensible."?': 'No',
'Does the word "sweat" mean the same thing in sentences "He\'s in a sweat about exams." and "Sweat poured off his brow."?': 'No',
'Does the word "lock" mean the same thing in sentences "Lock the bike to the fence." and "He was locked in a laughing fit."?': 'No',
'Does the word "blade" mean the same thing in sentences "A blade of lint on his suit." and "Gay young blades bragged of their amorous adventures."?': 'No',
'Does the word "grasp" mean the same thing in sentences "He has a good grasp of accounting practices." and "A terrible power had her in its grasp."?': 'No',
'Does the word "trade" mean the same thing in sentences "As they say in the trade." and "The skilled trades were the first to organize modern labor unions."?': 'Yes',
'Does the word "acquire" mean the same thing in sentences "Children acquire language at an amazing rate." and "I acquired a strong aversion to television."?': 'No',
'Does the word "segregate" mean the same thing in sentences "We do n\'t segregate in this county." and "This neighborhood is segregated."?': 'Yes',
'Does the word "dean" mean the same thing in sentences "He is the dean of foreign correspondents." and "Dean of the diplomatic corps - a country\'s most senior ambassador."?': 'Yes',
'Does the word "series" mean the same thing in sentences "The third series of Friends aired from 1996 to 1997." and "The harmonic series has been much studied."?': 'No',
'Does the word "flee" mean the same thing in sentences "The prisoner tried to flee, but was caught by the guards." and "He threw down his gun and fled."?': 'Yes',
'Does the word "race" mean the same thing in sentences "The race is to the swift." and "Some biologists doubt that there are important genetic differences between races of human beings."?': 'No',
'Does the word "crisscross" mean the same thing in sentences "Crisscross the sheet of paper." and "Wrinkles crisscrossed her face."?': 'No',
'Does the word "web" mean the same thing in sentences "The trees cast a delicate web of shadows over the lawn." and "Some of that content is now only available on the Web."?': 'No',
'Does the word "steep" mean the same thing in sentences "Steep the blossoms in oil." and "Steep the fruit in alcohol."?': 'Yes',
'Does the word "stalk" mean the same thing in sentences "Stalk the woods for deer." and "I stalked the woods for game."?': 'Yes',
'Does the word "retraction" mean the same thing in sentences "The retraction of the landing gear." and "Retraction of the foreskin."?': 'Yes',
'Does the word "bandy" mean the same thing in sentences "To bandy words ( with somebody )." and "We bandied around these difficult questions."?': 'No',
'Does the word "transportation" mean the same thing in sentences "Mulligan\'s sentence was commuted from death to transportation." and "The sentence was one of transportation for life."?': 'Yes',
'Does the word "center" mean the same thing in sentences "It is in the center of town." and "They were raising money to build a new center for research."?': 'No',
'Does the word "access" mean the same thing in sentences "He took a wrong turn on the access to the bridge." and "He gained access to the building."?': 'No',
'Does the word "tack" mean the same thing in sentences "Tack the notice on the board." and "He tacked together some verses."?': 'No',
'Does the word "cybernate" mean the same thing in sentences "Cybernate a factory." and "We live in a cybernated age."?': 'Yes',
'Does the word "redefinition" mean the same thing in sentences "Words like ` conservative\' require periodic redefinition." and "She provided a redefinition of his duties."?': 'Yes',
'Does the word "vice" mean the same thing in sentences "Vice offends the moral standards of the community." and "Smoking is a vice, not a virtue."?': 'Yes',
'Does the word "land" mean the same thing in sentences "Maybe that\'s how it works in TV - land, bu not in the real world." and "He made it the law of the land."?': 'No',
'Does the word "bar" mean the same thing in sentences "There was no bar against leaving." and "The window was protected by steel bars."?': 'No',
'Does the word "real" mean the same thing in sentences "Every integer is a real number, but not vice versa." and "A real number can be regarded abstractly as an equivalence class of Cauchy sequences of real numbers."?': 'Yes',
'Does the word "solve" mean the same thing in sentences "Solve for x." and "Solve an equation."?': 'Yes',
'Does the word "superlative" mean the same thing in sentences "The superlative of " big " is " biggest "." and "` most surely\' is the superlative of the adverb ` surely\'."?': 'Yes',
'Does the word "exposure" mean the same thing in sentences "The studio had a northern exposure." and "The exposure of his anger was shocking."?': 'No',
'Does the word "kind" mean the same thing in sentences "This is a strange kind of tobacco." and "What kinds of desserts are there ?"?': 'Yes',
'Does the word "vengeance" mean the same thing in sentences "He swore vengeance on the man who betrayed him." and "Vengeance is mine."?': 'Yes',
'Does the word "hack" mean the same thing in sentences "I ca n\'t hack it anymore." and "He hacked his way through the forest."?': 'No',
'Does the word "file" mean the same thing in sentences "File for divorce." and "File a complaint."?': 'Yes',
'Does the word "justify" mean the same thing in sentences "Justify the margins." and "The end justifies the means."?': 'No',
'Does the word "suspect" mean the same thing in sentences "I suspect him of being the thief." and "The U.S. suspected Bin Laden as the mastermind behind the terrorist attacks."?': 'Yes',
'Does the word "claim" mean the same thing in sentences "They struck in support of their claim for a shorter work day." and "His claim asked for damages."?': 'No',
'Does the word "arrangement" mean the same thing in sentences "He changed the arrangement of the topics." and "The arrangement of the furniture."?': 'No',
'Does the word "action" mean the same thing in sentences "A man of action." and "He is out of action."?': 'No',
'Does the word "contact" mean the same thing in sentences "They kept in daily contact." and "They claimed that they had been in contact with extraterrestrial beings."?': 'Yes',
'Does the word "capture" mean the same thing in sentences "Capture the essence of Spring." and "Capture an idea."?': 'Yes',
'Does the word "count" mean the same thing in sentences "Your views do n\'t count here." and "Count your change."?': 'No',
'Does the word "sensitivity" mean the same thing in sentences "Sensitivity to pain." and "A galvanometer of extreme sensitivity."?': 'No',
'Does the word "intrude" mean the same thing in sentences "The colors do n\'t intrude on the viewer." and "They intruded on our dinner party."?': 'No',
'Does the word "ball" mean the same thing in sentences "The children were playing ball on the beach." and "She kicked him in the balls and got away."?': 'No',
'Does the word "principal" mean the same thing in sentences "She sent unruly pupils to see the principal." and "A portion of your mortgage payment goes to reduce the principal, and the rest covers interest."?': 'No',
'Does the word "row" mean the same thing in sentences "Row the boat across the lake." and "They rowed the canals of Venice."?': 'Yes',
'Does the word "strain" mean the same thing in sentences "His responsibilities were a constant strain." and "He created a new strain of sheep."?': 'No',
'Does the word "sleeper" mean the same thing in sentences "The winner was a true sleeper -- no one expected him to get it." and "The British call a railroad tie a sleeper."?': 'No',
'Does the word "correct" mean the same thing in sentences "Correct the alignment of the front wheels." and "The stock market corrected."?': 'No',
'Does the word "excogitate" mean the same thing in sentences "The scientist must stop to observe and start to excogitate." and "Excogitate a way to measure the speed of light."?': 'No',
'Does the word "glimpse" mean the same thing in sentences "I have only begun to glimpse the magnitude of the problem." and "We glimpsed the Queen as she got into her limousine."?': 'Yes',
'Does the word "explode" mean the same thing in sentences "They sought to explode the myth." and "The enemy exploded the bridge."?': 'Yes',
'Does the word "interest" mean the same thing in sentences "They said nothing of great interest." and "Primary colors can add interest to a room."?': 'Yes',
'Does the word "matter" mean the same thing in sentences "Is anything the matter ?" and "He always took some reading matter with him on the plane."?': 'No',
'Does the word "observe" mean the same thing in sentences "Please observe the reaction of these two chemicals." and "She observed that his presentation took up too much time."?': 'No',
'Does the word "heart" mean the same thing in sentences "Buddhists believe that suffering is right at the heart of all life." and "The wood at the heart of a tree is the oldest."?': 'Yes',
'Does the word "extend" mean the same thing in sentences "Extend a hand." and "Extend your backyard."?': 'No',
'Does the word "rawness" mean the same thing in sentences "Their poor behavior was due to the rawness of the troops." and "After taking a cold, rawness of the larynx and trachea come on."?': 'No',
'Does the word "inspire" mean the same thing in sentences "The lung cancer patient can not inspire air very well." and "The patient has trouble inspiring."?': 'Yes',
'Does the word "favor" mean the same thing in sentences "The outcome was in his favor." and "That style is in favor this season."?': 'No',
'Does the word "coarsen" mean the same thing in sentences "Coarsen the surface." and "Because the wool is poor quality it will coarsen the fabric."?': 'Yes',
'Does the word "reek" mean the same thing in sentences "You reek of perfume." and "Your fridge reeks of egg."?': 'Yes',
'Does the word "second" mean the same thing in sentences "He came in a close second." and "The treasure is 2 minutes and 45 seconds south of here."?': 'No',
'Does the word "disappearance" mean the same thing in sentences "What was responsible for the disappearance of the rainforest ?" and "He regretted the disappearance of Greek from school curricula."?': 'Yes',
'Does the word "shock" mean the same thing in sentences "He was numb with shock." and "Whole fields of wheat in shock."?': 'No',
'Does the word "basis" mean the same thing in sentences "The whole argument rested on a basis of conjecture." and "He worked on an interim basis."?': 'No',
'Does the word "quarter" mean the same thing in sentences "A quarter of a pound." and "The Latin Quarter."?': 'No',
'Does the word "sickness" mean the same thing in sentences "Drugs have become a sickness they can not cure." and "A great sickness of his judgment."?': 'Yes',
'Does the word "importance" mean the same thing in sentences "A person of importance." and "The importance of a well-balanced diet."?': 'No',
'Does the word "convert" mean the same thing in sentences "Could you convert my dollars into pounds ?" and "Convert centimeters into inches."?': 'Yes',
'Does the word "line" mean the same thing in sentences "That salesman must have practiced his fast line of talk." and "` let me show you my etchings\' is a rather worn line."?': 'Yes',
'Does the word "provide" mean the same thing in sentences "We aim to provide the local community with more green spaces." and "The hostess provided lunch for all the guests."?': 'Yes',
'Does the word "score" mean the same thing in sentences "Score the clay before firing it." and "He scored a 200."?': 'No',
'Does the word "reach" mean the same thing in sentences "To reach one a book." and "This car can reach a speed of 140 miles per hour."?': 'No',
'Does the word "balance" mean the same thing in sentences "He took what he wanted and I got the balance." and "The balance of the agreement remains in effect."?': 'Yes',
'Does the word "gasp" mean the same thing in sentences "She gave a gasp and fainted." and "I\'m popping out for a gasp."?': 'Yes',
'Does the word "void" mean the same thing in sentences "Nobody has crossed the void since one man died trying three hundred years ago ; it\'s high time we had another go." and "The huge desert voids."?': 'Yes',
'Does the word "high" mean the same thing in sentences "I\'m on a permanent high these days." and "Summer temperatures reached an all-time high."?': 'No',
'Does the word "campaign" mean the same thing in sentences "I managed his campaign for governor." and "An election campaign."?': 'No',
'Does the word "notice" mean the same thing in sentences "His hard work soon attracted the teacher\'s notice." and "He gave notice two months before he moved."?': 'No',
'Does the word "flux" mean the same thing in sentences "That high a neutron flux would be lethal in seconds." and "The newness and flux of the computer industry."?': 'No',
'Does the word "love" mean the same thing in sentences "Their love left them indifferent to their surroundings." and "She was his first love."?': 'Yes',
'Does the word "treatment" mean the same thing in sentences "The right to equal treatment in the criminal justice system." and "The treatment of water sewage."?': 'Yes',
'Does the word "addition" mean the same thing in sentences "The addition of flowers created a pleasing effect." and "The addition of five more items to the agenda will make the meeting unbearably long."?': 'Yes',
'Does the word "harvest" mean the same thing in sentences "A harvest of love." and "They gathered a harvest of examples."?': 'Yes',
'Does the word "divaricate" mean the same thing in sentences "Divaricate one\'s fingers." and "The road divaricates here."?': 'No',
'Does the word "resign" mean the same thing in sentences "Resign a claim to the throne." and "He resigned the crown to follow his heart."?': 'No',
'Does the word "know" mean the same thing in sentences "I know Latin." and "This student knows her irregular verbs."?': 'Yes',
'Does the word "grace" mean the same thing in sentences "Their youngest son said grace." and "It has become less common to say grace before having dinner."?': 'Yes',
'Does the word "read" mean the same thing in sentences "The King will read the proclamation at noon." and "Have you read this book ?"?': 'Yes',
'Does the word "cross" mean the same thing in sentences "Cross a horse and a donkey." and "The trains crossed."?': 'No',
'Does the word "eyeful" mean the same thing in sentences "They wanted to see violence and they got an eyeful." and "She was a statuesque redheaded eyeful."?': 'No',
'Does the word "superlative" mean the same thing in sentences "` least famous\' is the superlative degree of the adjective ` famous\'." and "` fastest\' is the superlative of the adjective ` fast\'."?': 'Yes',
'Does the word "edit" mean the same thing in sentences "Edit film." and "The same family has been editing the influential newspaper for almost 100 years."?': 'No',
'Does the word "represent" mean the same thing in sentences "I represent the silent majority." and "The flower represents a young girl."?': 'No',
'Does the word "endorsement" mean the same thing in sentences "A Democrat usually gets the union\'s endorsement." and "The association announced its endorsement of the policy."?': 'No',
'Does the word "vacate" mean the same thing in sentences "You must vacate your office by tonight." and "Vacate a death sentence."?': 'No',
'Does the word "chokehold" mean the same thing in sentences "He grabbed the woman in a chokehold, demanded her cash and jewelry, and then fled." and "The president applied a chokehold to labor disputes that inconvenienced the public."?': 'No',
'Does the word "rent" mean the same thing in sentences "Let\'s rent a car." and "We rented our apartment to friends while we were abroad."?': 'No',
'Does the word "sell" mean the same thing in sentences "The French try to sell us their image as great lovers." and "The new idea sold well in certain circles."?': 'No',
'Does the word "express" mean the same thing in sentences "Can you express this distance in kilometers ?" and "Italians express coffee rather than filter it."?': 'No',
'Does the word "juice" mean the same thing in sentences "Squeeze the orange and some juice will come out." and "Digestive juices."?': 'No',
'Does the word "cast" mean the same thing in sentences "Cast a warm light." and "Cast a ballot."?': 'No',
'Does the word "matter" mean the same thing in sentences "What\'s the matter ?" and "Several matters to attend to."?': 'Yes',
'Does the word "content" mean the same thing in sentences "The two groups were similar in content." and "They could read to their heart\'s content."?': 'No',
'Does the word "rusticate" mean the same thing in sentences "Rusticate the house in the country." and "He was rusticated for his bad behavior."?': 'No',
'Does the word "converge" mean the same thing in sentences "The lines converge at this point." and "Social forces converged to bring the Fascists back to power."?': 'No',
'Does the word "bang" mean the same thing in sentences "When he struck it with a hammer, there was a loud bang." and "My bangs keep getting in my eyes when I swim."?': 'No',
'Does the word "carry" mean the same thing in sentences "He can not carry a tune." and "She carried herself well."?': 'No',
'Does the word "wind" mean the same thing in sentences "That\'s a lot of wind." and "As they accelerated onto the motorway, the wind tore the plywood off the car\'s roof-rack."?': 'No',
'Does the word "bury" mean the same thing in sentences "Bury a bone." and "They buried the stolen goods."?': 'Yes',
'Does the word "edition" mean the same thing in sentences "It was too late for the morning edition." and "The first edition appeared in 1920."?': 'Yes',
'Does the word "rank" mean the same thing in sentences "Based on your test scores, you have a rank of 23." and "The fancy hotel was of the first rank."?': 'Yes',
'Does the word "train" mean the same thing in sentences "A train of thought." and "It led to a train of disasters."?': 'No',
'Does the word "trough" mean the same thing in sentences "The neurologist pointed to a troubling trough in the pattern of his brain-waves." and "The buoy bobbed between the crests and troughs of the waves moving across the bay."?': 'Yes',
'Does the word "level" mean the same thing in sentences "Level the ground." and "The hunter levels the gun before taking a shot."?': 'No',
'Does the word "day" mean the same thing in sentences "Every dog has its day." and "Two days later they left."?': 'Yes',
'Does the word "stoop" mean the same thing in sentences "I wo n\'t stoop to reading other people\'s mail." and "He stooped to tie his shoe-laces."?': 'No',
'Does the word "feeling" mean the same thing in sentences "I had a strange feeling in my leg." and "He had a queasy feeling."?': 'Yes',
'Does the word "custody" mean the same thing in sentences "Your guests are now in my custody." and "The mother was awarded custody of the children."?': 'Yes',
'Does the word "erupt" mean the same thing in sentences "Erupt in anger." and "Unrest erupted in the country."?': 'No',
'Does the word "differ" mean the same thing in sentences "I beg to differ !" and "The winter schedule differed from the spring schedule."?': 'No',
'Does the word "fear" mean the same thing in sentences "I fear the winters in Moscow." and "We should not fear the Communists !"?': 'Yes',
'Does the word "ligate" mean the same thing in sentences "Ligate the artery." and "The enzyme ligated."?': 'No',
'Does the word "sign" mean the same thing in sentences "Do n\'t forget the minus sign." and "It was a sign from God."?': 'No',
'Does the word "presidency" mean the same thing in sentences "Ernest was a historian specializing in the presidency of Herbert Hoover." and "In France, a presidency lasts for five years."?': 'Yes',
'Does the word "shuffle" mean the same thing in sentences "Do n\'t forget to shuffle the cards." and "The data packets are shuffled before transmission."?': 'Yes',
'Does the word "aim" mean the same thing in sentences "It was created with the conscious aim of answering immediate needs." and "He took aim and fired."?': 'No',
'Does the word "wounded" mean the same thing in sentences "The wounded lay on stretchers waiting for surgery." and "They had to leave the wounded where they fell."?': 'Yes',
'Does the word "boil" mean the same thing in sentences "Boil this liquid until it evaporates." and "Boil some water in a pan."?': 'Yes',
'Does the word "victory" mean the same thing in sentences "Clinched a victory." and "The general always gets credit for his army\'s victory."?': 'Yes',
'Does the word "understanding" mean the same thing in sentences "He has virtually no understanding of social cause and effect." and "There was an understanding between management and the workers."?': 'No',
'Does the word "quarry" mean the same thing in sentences "A British term for ` quarry\' is ` stone pit\'." and "Michelangelo personally quarried marble from the world-famous quarry at Carrara."?': 'Yes',
'Does the word "game" mean the same thing in sentences "For actors, memorizing lines is no game." and "For him, life is all fun and games."?': 'Yes',
'Does the word "holder" mean the same thing in sentences "He\'s been an account holder with us since 2004." and "In 2012, there were 28 living holders of the Victoria Cross or the George Cross."?': 'Yes',
'Does the word "chapter" mean the same thing in sentences "He read a chapter every night before falling asleep." and "He joined the Atlanta chapter."?': 'No',
'Does the word "put" mean the same thing in sentences "We put the time of arrival at 8 P.M." and "Put your things here."?': 'No',
'Does the word "deflate" mean the same thing in sentences "Deflate the air mattress." and "The balloons deflated."?': 'No',
'Does the word "love" mean the same thing in sentences "A mother\'s love is not easily shaken." and "Children need a lot of love."?': 'Yes',
'Does the word "feed" mean the same thing in sentences "Feed carrots into a food processor." and "Her success feeds her vanity."?': 'No',
'Does the word "age" mean the same thing in sentences "We age every day -- what a depressing thought !" and "She aged gracefully."?': 'Yes',
'Does the word "counsel" mean the same thing in sentences "Psychiatrists, psychologists, social workers and other mental health professionals counsel clients." and "The lawyer counselled his client to remain silent."?': 'Yes',
'Does the word "leak" mean the same thing in sentences "The leak gained on the ship\'s pumps." and "I have to take a leak."?': 'No',
'Does the word "repair" mean the same thing in sentences "To repair a house, a road, a shoe, or a ship." and "To repair a shattered fortune."?': 'Yes',
'Does the word "wall" mean the same thing in sentences "He ducked behind the garden wall and waited." and "The wall followed the road."?': 'Yes',
'Does the word "agree" mean the same thing in sentences "All parties agree in the expediency of the law." and "They agreed about what to do."?': 'Yes',
'Does the word "place" mean the same thing in sentences "Place a phone call." and "Place emphasis on a certain point."?': 'No',
'Does the word "nursing" mean the same thing in sentences "She went into nursing as a career." and "Study nursing."?': 'Yes',
'Does the word "naught" mean the same thing in sentences "It was all for naught." and "All my efforts led to naught."?': 'No',
'Does the word "onrush" mean the same thing in sentences "From the bow she stared at the mesmerising onrush of the sea where it split and foamed." and "The explosion interrupted the wild onrush of her thoughts."?': 'Yes',
'Does the word "correction" mean the same thing in sentences "Market runups are invariably followed by a correction." and "The correction of his vision with eye glasses."?': 'No',
'Does the word "fumble" mean the same thing in sentences "Fumble a grounder." and "He fumbled the key into the lock."?': 'No',
'Does the word "extent" mean the same thing in sentences "An orchard of considerable extent." and "The vast extent of the desert."?': 'Yes',
'Does the word "stress" mean the same thing in sentences "Stress is a vasoconstrictor." and "He put the stress on the wrong syllable."?': 'No',
'Does the word "build" mean the same thing in sentences "Build a modern nation." and "Build a million-dollar business."?': 'Yes',
'Does the word "exchange" mean the same thing in sentences "Exchange employees between branches of the company." and "Exchange prisoners."?': 'Yes',
}

In [4]:
human_readable_reversed = {
    r'Does the word "defeat" mean the same thing in sentences "The army \'s only defeat." and "It was a narrow defeat."?': 'Yes',
r'Does the word "groom" mean the same thing in sentences "Sheila groomed the horse." and "Groom the dogs."?': 'Yes',
r'Does the word "penetration" mean the same thing in sentences "Any penetration, however slight, is sufficient to complete the offense." and "The penetration of upper management by women."?': 'Yes',
r'Does the word "hit" mean the same thing in sentences "An interesting idea hit her." and "We hit Detroit at one in the morning but kept driving through the night."?': 'No',
r'Does the word "deliberation" mean the same thing in sentences "A little deliberation would have deterred them." and "He was a man of judicial deliberation."?': 'No',
r'Does the word "navel" mean the same thing in sentences "You were not supposed to show your navel on television." and "They argued whether or not Adam had a navel."?': 'Yes',
r'Does the word "afforest" mean the same thing in sentences "Afforest the mountains." and "After we leave the quarry, we intend to afforest the land and turn it into a nature reserve."?': 'Yes',
r'Does the word "solve" mean the same thing in sentences "Did you solve the problem ?" and "Solve an old debt."?': 'No',
r'Does the word "purchase" mean the same thing in sentences "They closed the purchase with a handshake." and "They offer a free hamburger with the purchase of a drink."?': 'Yes',
r'Does the word "software" mean the same thing in sentences "The market for software is expected to expand." and "Did you test the software package to ensure completeness ?"?': 'Yes',
r'Does the word "push" mean the same thing in sentences "The army made a push toward the sea." and "Some details got lost in the push to get the project done."?': 'Yes',
r'Does the word "bake" mean the same thing in sentences "This oven bakes potatoes." and "Idaho potatoes bake beautifully."?': 'Yes',
r'Does the word "relieve" mean the same thing in sentences "This pill will relieve your headaches." and "Relieve the pressure and the stress."?': 'No',
r'Does the word "style" mean the same thing in sentences "This style of shoe is in demand." and "In the characteristic New York style."?': 'No',
r'Does the word "crumb" mean the same thing in sentences "Crumb a cutlet." and "Crumb the table."?': 'No',
r'Does the word "include" mean the same thing in sentences "The list includes the names of many famous writers." and "I include you in the list of culprits."?': 'No',
r'Does the word "companion" mean the same thing in sentences "Drinking companions." and "His dog has been his trusted companion for the last five years."?': 'Yes',
r'Does the word "reveal" mean the same thing in sentences "He revealed the children found." and "The actress wo n\'t reveal how old she is."?': 'Yes',
r'Does the word "presence" mean the same thing in sentences "She blushed in his presence." and "I \'m convinced that there was a presence in that building that I ca n\'t explain, which led to my heroic actions."?': 'No',
r'Does the word "relax" mean the same thing in sentences "The rules relaxed after the new director arrived." and "Do n\'t relax your efforts now."?': 'No',
r'Does the word "parity" mean the same thing in sentences "The parity of the mother must be considered." and "Parity is often used to check the integrity of transmitted data."?': 'No',
r'Does the word "raise" mean the same thing in sentences "Raise a barn." and "To raise a wall, or a heap of stones."?': 'Yes',
r'Does the word "suspend" mean the same thing in sentences "The prison sentence was suspended." and "Suspend the particles."?': 'No',
r'Does the word "amass" mean the same thing in sentences "She is amassing a lot of data for her thesis." and "To amass a treasure or a fortune."?': 'Yes',
r'Does the word "term" mean the same thing in sentences "He learned many medical terms." and "A healthy baby born at full term."?': 'No',
r'Does the word "leash" mean the same thing in sentences "Kept a tight leash on his emotions." and "He \'s always gotten a long leash."?': 'Yes',
r'Does the word "conversion" mean the same thing in sentences "His conversion to the Catholic faith." and "The conversion of equations."?': 'No',
r'Does the word "making" mean the same thing in sentences "It was already in the making." and "The making of measurements."?': 'Yes',
r'Does the word "set" mean the same thing in sentences "They played two sets of tennis after dinner." and "Before the set of sun."?': 'No',
r'Does the word "mate" mean the same thing in sentences "Camels hate leaving their mates." and "He lost the mate to his shoe."?': 'No',
r'Does the word "expression" mean the same thing in sentences "His manner of expression showed how much he cared." and "They stared at the newcomer with a puzzled expression."?': 'No',
r'Does the word "rim" mean the same thing in sentences "Sugar rimmed the dessert plate." and "Rim a hat."?': 'No',
r'Does the word "cure" mean the same thing in sentences "Cure hay." and "Cure meats."?': 'Yes',
r'Does the word "rift" mean the same thing in sentences "The Grand Canyon is a rift in the Earth \'s surface, but is smaller than some of the undersea ones." and "My marriage is in trouble, the fight created a rift between us and we ca n\'t reconnect."?': 'Yes',
r'Does the word "swim" mean the same thing in sentences "A big fish was swimming in the tank." and "We had to swim for 20 minutes to reach the shore."?': 'Yes',
r'Does the word "quiet" mean the same thing in sentences "When you quiet, we can start talking." and "The teacher did her best to quiet the children down."?': 'No',
r'Does the word "top" mean the same thing in sentences "To be at the top of one \'s class, or at the top of the school." and "At the top of his profession."?': 'Yes',
r'Does the word "consultation" mean the same thing in sentences "Frequent consultations with his lawyer." and "A consultation of several medical specialists."?': 'Yes',
r'Does the word "chiromance" mean the same thing in sentences "The Gypsies chiromanced." and "She refused to chiromance my fate."?': 'Yes',
r'Does the word "bank" mean the same thing in sentences "The pilot had to bank the aircraft." and "Bank on your good education."?': 'No',
r'Does the word "rag" mean the same thing in sentences "Rag ore." and "Rag that old tune."?': 'No',
r'Does the word "work" mean the same thing in sentences "Work is done against friction to drag a bag along the ground." and "Work equals force times distance."?': 'Yes',
r'Does the word "allowance" mean the same thing in sentences "A child \'s allowance should not be too generous." and "He objected to the allowance of smoking in the dining room."?': 'No',
r'Does the word "contact" mean the same thing in sentences "He used his business contacts to get an introduction to the governor." and "Litmus paper turns red on contact with an acid."?': 'No',
r'Does the word "virus" mean the same thing in sentences "He caught a virus and had to stay home from school." and "The virus of jealousy is latent in everyone."?': 'No',
r'Does the word "humour" mean the same thing in sentences "She has a great sense of humour, and I always laugh a lot whenever we get together." and "The sensitive subject was treated with humour, but in such way that no one was offended."?': 'Yes',
r'Does the word "neighbor" mean the same thing in sentences "Fort Worth is a neighbor of Dallas." and "What is the closest neighbor to the Earth ?"?': 'Yes',
r'Does the word "sinking" mean the same thing in sentences "After several hours of sinking an unexpected rally rescued the market." and "He could not control the sinking of his legs."?': 'Yes',
r'Does the word "sneak" mean the same thing in sentences "Sneak a cigarette." and "Sneak a look."?': 'Yes',
r'Does the word "fix" mean the same thing in sentences "Fix a race." and "Fix your eyes on this spot."?': 'No',
r'Does the word "impulse" mean the same thing in sentences "The total impulse from the impact will depend on the kinetic energy of the bullet." and "The impulse knocked him over."?': 'No',
r'Does the word "fetish" mean the same thing in sentences "Common male fetishes are breasts, legs, hair, shoes, and underwear." and "I know a guy who has a foot fetish."?': 'Yes',
r'Does the word "seizure" mean the same thing in sentences "The search warrant permitted the seizure of evidence." and "The seizure of a thief, a property, a throne, etc."?': 'Yes',
r'Does the word "pattern" mean the same thing in sentences "They changed their dietary pattern." and "The American constitution has provided a pattern for many republics."?': 'No',
r'Does the word "conscience" mean the same thing in sentences "He has no conscience about his cruelty." and "A person of unflagging conscience."?': 'No',
r'Does the word "demystify" mean the same thing in sentences "Let \'s demystify the event by explaining what it is all about." and "The article was written to demystify the mechanics of the internal combustion engine."?': 'Yes',
r'Does the word "team" mean the same thing in sentences "The IT manager leads a team of three software developers." and "We need more volunteers for the netball team."?': 'Yes',
r'Does the word "conduct" mean the same thing in sentences "To conduct the affairs of a kingdom." and "You can not conduct business like this."?': 'Yes',
r'Does the word "administer" mean the same thing in sentences "She administers the funds." and "Administer critical remarks to everyone present."?': 'No',
r'Does the word "abort" mean the same thing in sentences "He sent a short message requesting an abort due to extreme winds in the area." and "I wasted a year of my life working on an abort."?': 'Yes',
r'Does the word "glass" mean the same thing in sentences "We collected art glass." and "She collected old glass."?': 'Yes',
r'Does the word "oppose" mean the same thing in sentences "The board opposed his motion." and "The senator said he would oppose the bill."?': 'No',
r'Does the word "married" mean the same thing in sentences "We invited several young marrieds." and "My plan is to wait to have sex until i \'m married."?': 'Yes',
r'Does the word "utility" mean the same thing in sentences "I \'ve bought a new disk utility that can recover deleted files." and "A computer system provides utility programs to perform the tasks needed by most users."?': 'Yes',
r'Does the word "passive" mean the same thing in sentences "` The ball was thrown \' is an abbreviated passive." and "` The ball was thrown by the boy \' uses the passive voice."?': 'Yes',
r'Does the word "hold" mean the same thing in sentences "She always held herself as a lady." and "Hold in place."?': 'Yes',
r'Does the word "bit" mean the same thing in sentences "A bit of rock caught him in the eye." and "A bit of paper."?': 'No',
r'Does the word "induce" mean the same thing in sentences "The ads induced me to buy a VCR." and "Induce a crisis."?': 'No',
r'Does the word "center" mean the same thing in sentences "They received messages from several centers." and "Convention center."?': 'Yes',
r'Does the word "morale" mean the same thing in sentences "After the layoffs morale was at an all time low, they were so dispirited nothing was getting done." and "Morale is an important quality in soldiers. With good morale they \'ll charge into a hail of bullets ; without it they wo n\'t even cross a street."?': 'Yes',
r'Does the word "security" mean the same thing in sentences "The head of security was a former policeman." and "Military security has been stepped up since the recent uprising."?': 'No',
r'Does the word "picture" mean the same thing in sentences "Too often the narrative was interrupted by long word pictures." and "The author gives a depressing picture of life in Poland."?': 'Yes',
r'Does the word "judge" mean the same thing in sentences "I can not judge some works of modern art." and "I judge a man \'s character by the cut of his suit."?': 'Yes',
r'Does the word "cast" mean the same thing in sentences "The cast was praised for a fine performance." and "The cast would need a great deal of machining to become a recognizable finished part."?': 'No',
r'Does the word "fire" mean the same thing in sentences "The gun fired." and "Do n\'t fire until you see the whites of their eyes."?': 'Yes',
r'Does the word "grab" mean the same thing in sentences "Grab power." and "This story will grab you."?': 'No',
r'Does the word "passion" mean the same thing in sentences "He has a passion for cock fighting." and "We share a passion for books."?': 'No',
r'Does the word "read" mean the same thing in sentences "The gauge read ` empty \'." and "Can you read Greek ?"?': 'No',
r'Does the word "roast" mean the same thing in sentences "To roast meat on a spit." and "Roast the turkey."?': 'Yes',
r'Does the word "dress" mean the same thing in sentences "Fastidious about his dress." and "He came to the party in formal dress."?': 'Yes',
r'Does the word "miss" mean the same thing in sentences "The workers on the conveyor belt miss one out of ten." and "How could I miss that typo ?"?': 'Yes',
r'Does the word "rerun" mean the same thing in sentences "Bush wants to rerun in 1996." and "We have to rerun the subjects -- they misunderstood the instructions."?': 'No',
r'Does the word "chip" mean the same thing in sentences "Chip a tooth." and "Be careful not to chip the paint."?': 'Yes',
r'Does the word "share" mean the same thing in sentences "To share a shelter with another." and "They share a language."?': 'Yes',
r'Does the word "stone" mean the same thing in sentences "She had jewels made of all the rarest stones." and "A heavy chap who must have weighed more than twenty stone."?': 'No',
r'Does the word "approach" mean the same thing in sentences "He approached the age of manhood." and "Would counsel please approach the bench ? asked the judge."?': 'Yes',
r'Does the word "restore" mean the same thing in sentences "To restore harmony among those who are at variance." and "Restore the emperor to the throne."?': 'Yes',
r'Does the word "election" mean the same thing in sentences "They celebrated his election." and "The results of the election will be announced tonight."?': 'No',
r'Does the word "bell" mean the same thing in sentences "She heard the distant toll of church bells." and "Saved by the bell."?': 'Yes',
r'Does the word "logic" mean the same thing in sentences "By the logic of war." and "Economic logic requires it."?': 'Yes',
r'Does the word "drip" mean the same thing in sentences "My broken pen dripped ink onto the table." and "After putting oil on the side of the salad, the chef should drip a little vinegar in the oil."?': 'Yes',
r'Does the word "event" mean the same thing in sentences "In that event, the first possibility is excluded." and "He acted very wise after the event."?': 'No',
r'Does the word "approval" mean the same thing in sentences "Although she fussed at them, she secretly viewed all her children with approval." and "He bought it on approval."?': 'No',
r'Does the word "marmalade" mean the same thing in sentences "Lime marmalade." and "Thick cut marmalade."?': 'Yes',
r'Does the word "dress" mean the same thing in sentences "I rose and dressed before daybreak." and "Parents must feed and dress their child."?': 'Yes',
r'Does the word "square" mean the same thing in sentences "A checkerboard has 64 squares." and "You can compute the area of a square if you know the length of its sides."?': 'No',
r'Does the word "trip" mean the same thing in sentences "We made a trip to the beach." and "He took a trip to the shopping center."?': 'Yes',
r'Does the word "restraint" mean the same thing in sentences "The unlawful restraint of trade." and "He was a model of polite restraint."?': 'No',
r'Does the word "seating" mean the same thing in sentences "There is seating for 40 students in this classroom." and "There is plenty of comfortable seating."?': 'Yes',
r'Does the word "irony" mean the same thing in sentences "Irony is wasted on the stupid." and "The irony of Ireland \'s copying the nation she most hated."?': 'No',
r'Does the word "flare" mean the same thing in sentences "She could not control her flare of rage." and "The skirt had a wide flare."?': 'No',
r'Does the word "internationalize" mean the same thing in sentences "We internationalized the committee." and "Internationalize trade of certain drugs."?': 'No',
r'Does the word "favor" mean the same thing in sentences "To fall out of favor." and "She enjoyed the queen \'s favor."?': 'Yes',
r'Does the word "sit" mean the same thing in sentences "She sat on the jury." and "I asked him to sit."?': 'No',
r'Does the word "shore" mean the same thing in sentences "The river was shored by trees." and "Shore and buttress an old building."?': 'No',
r'Does the word "snorter" mean the same thing in sentences "A snorter of a sermon." and "The storm was n\'t long but it was a snorter."?': 'Yes',
r'Does the word "government" mean the same thing in sentences "He had considerable experience of government." and "Tyrannical government."?': 'No',
r'Does the word "producer" mean the same thing in sentences "This microorganism is a producer of disease." and "Maine is a leading producer of potatoes."?': 'Yes',
r'Does the word "step" mean the same thing in sentences "She is always stepping on others to get ahead." and "The architect wants to step the terrace."?': 'No',
r'Does the word "lock" mean the same thing in sentences "She locked her jewels in the safe." and "If you put the brakes on too hard, the wheels will lock."?': 'Yes',
r'Does the word "interview" mean the same thing in sentences "My interviews with teenagers revealed a weakening of religious bonds." and "The reporter gave the witness an interview."?': 'Yes',
r'Does the word "estimation" mean the same thing in sentences "They had a high estimation of his ability." and "In my estimation the boy is innocent."?': 'No',
r'Does the word "backup" mean the same thing in sentences "He discovered a backup in the toilet." and "A traffic backup on the main street."?': 'Yes',
r'Does the word "lubricate" mean the same thing in sentences "The liquid in this can lubricates well." and "Lubricate my car."?': 'No',
r'Does the word "wash" mean the same thing in sentences "Wash the towels, please !" and "Dishwashers wash dishes much more efficiently than most humans."?': 'No',
r'Does the word "nose" mean the same thing in sentences "The nose of the rocket heated up on reentry." and "The hound has a good nose."?': 'No',
r'Does the word "presence" mean the same thing in sentences "He sensed the presence of danger." and "He felt the presence of an evil force."?': 'No',
r'Does the word "ceiling" mean the same thing in sentences "He put a ceiling on the number of women who worked for him." and "The dining room had an ornate ceiling."?': 'No',
r'Does the word "ruin" mean the same thing in sentences "They explored several Roman ruins." and "You have brought ruin on this entire family."?': 'No',
r'Does the word "bandage" mean the same thing in sentences "The nurse bandaged a sprained ankle." and "Bandage an incision."?': 'Yes',
r'Does the word "move" mean the same thing in sentences "We moved from our previous position." and "I was sitting on the sofa for a long time, I was too lazy to move."?': 'Yes',
r'Does the word "conflict" mean the same thing in sentences "This form of conflict is essential to Mann \'s writing." and "He noticed a conflict in the dates of the two meetings."?': 'No',
r'Does the word "cookery" mean the same thing in sentences "Henry was not very good at cookery and most of his meals ended up burned." and "People are needed who have experience in cookery."?': 'Yes',
r'Does the word "image" mean the same thing in sentences "A movie is a series of images projected so rapidly that the eye integrates them." and "A public image is as fragile as Humpty Dumpty."?': 'No',
r'Does the word "beat" mean the same thing in sentences "The conductor set the beat." and "He heard the beat of a drum."?': 'No',
r'Does the word "pitch" mean the same thing in sentences "Pitch the tent over there." and "Pitch a tent."?': 'Yes',
r'Does the word "marginalization" mean the same thing in sentences "The marginalization of literature." and "The marginalization of the underclass."?': 'Yes',
r'Does the word "plus" mean the same thing in sentences "Four plus three equals seven." and "Two plus two equals four."?': 'Yes',
r'Does the word "god" mean the same thing in sentences "Money was his god." and "He was a god among men."?': 'No',
r'Does the word "defang" mean the same thing in sentences "The snake was defanged." and "Defang the poisonous snake."?': 'Yes',
r'Does the word "school" mean the same thing in sentences "When the school day was done we would walk home together." and "The school was founded in 1900."?': 'No',
r'Does the word "enrich" mean the same thing in sentences "The oil boom enriched a lot of local people." and "Hobbies enrich lives."?': 'No',
r'Does the word "visibility" mean the same thing in sentences "That candidate does not have sufficient visibility to win an election." and "A windshield with good visibility."?': 'No',
r'Does the word "post" mean the same thing in sentences "I \'ll post the news on the bulletin board." and "Post a sign."?': 'No',
r'Does the word "adhesion" mean the same thing in sentences "A heated hydraulic press was required for adhesion." and "The adhesion of Seville was decisive."?': 'No',
r'Does the word "sister" mean the same thing in sentences "My sister married a musician." and "My sister is always driving me crazy."?': 'Yes',
r'Does the word "expose" mean the same thing in sentences "Expose your students to art." and "Expose the blanket to sunshine."?': 'Yes',
r'Does the word "sound" mean the same thing in sentences "The beautiful sound of music." and "The sound of rain on the roof."?': 'Yes',
r'Does the word "jolt" mean the same thing in sentences "All the jars and jolts were smoothed out by the shock absorbers." and "The door closed with a jolt."?': 'Yes',
r'Does the word "sketch" mean the same thing in sentences "Sketch the outline of the book." and "Sketch the building."?': 'No',
r'Does the word "fill" mean the same thing in sentences "The mixer returned to the plant for another fill." and "There was not enough fill for the trench."?': 'Yes',
r'Does the word "depth" mean the same thing in sentences "Depth of a closet." and "The depth of the water."?': 'Yes',
r'Does the word "organize" mean the same thing in sentences "Organize a strike." and "Can you help me organize my files ?"?': 'No',
r'Does the word "gas" mean the same thing in sentences "The atmosphere is made up of a number of different gases." and "He stepped on the gas."?': 'No',
r'Does the word "nonproliferation" mean the same thing in sentences "They protested that the nonproliferation treaty was just a plot to maintain the hegemony of those who already had nuclear weapons." and "Nuclear disarmament and nonproliferation are closely related goals."?': 'Yes',
r'Does the word "rig" mean the same thing in sentences "They rigged the bomb to the ignition." and "Rig a ship."?': 'No',
r'Does the word "peace" mean the same thing in sentences "Peace came on November 11th." and "The roommates lived in peace together."?': 'No',
r'Does the word "drag" mean the same thing in sentences "Taxation is a drag on the economy." and "Too many laws are a drag on the use of new land."?': 'Yes',
r'Does the word "introduce" mean the same thing in sentences "A new word processor was introduced." and "Introduce the new neighbors to the community."?': 'No',
r'Does the word "shtik" mean the same thing in sentences "His shtik made us laugh." and "Give him a shtik cake."?': 'No',
r'Does the word "history" mean the same thing in sentences "History takes the long view." and "All of human history."?': 'No',
r'Does the word "harness" mean the same thing in sentences "Harness the horse." and "Harness natural forces and resources."?': 'No',
r'Does the word "clatter" mean the same thing in sentences "Clatter of shutters." and "The clatter of iron wheels on cobblestones."?': 'Yes',
r'Does the word "religion" mean the same thing in sentences "Eckankar is a new religion but Zoroastrianism is an old religion." and "Islam is a major religion in parts of Asia and Africa."?': 'Yes',
r'Does the word "squash" mean the same thing in sentences "Somehow, she squashed all her books into her backpack, which was now too heavy to carry." and "We all managed to squash into Mum \'s tiny car."?': 'Yes',
r'Does the word "sex" mean the same thing in sentences "Slime molds are sometimes erroneously said to have thirteen sexes." and "She did n\'t want to know the sex of the foetus."?': 'Yes',
r'Does the word "estimate" mean the same thing in sentences "He estimated what would be likely." and "I estimate this chicken to weigh three pounds."?': 'Yes',
r'Does the word "burst" mean the same thing in sentences "I blew the balloon up too much, and it burst." and "The bubble burst."?': 'Yes',
r'Does the word "relish" mean the same thing in sentences "He relishes their time together." and "I do n\'t relish the idea of going out tonight."?': 'Yes',
r'Does the word "twist" mean the same thing in sentences "Twist the strips of dough." and "Twist one \'s head."?': 'No',
r'Does the word "yacht" mean the same thing in sentences "You are a true yachtsman ! Are you a member of the local yacht club ?" and "Would you like to go sailing on my uncle \'s yacht ?"?': 'Yes',
r'Does the word "poison" mean the same thing in sentences "Her husband poisoned her drink in order to kill her." and "This mushrooms can poison."?': 'No',
r'Does the word "guard" mean the same thing in sentences "Guards must be good blockers." and "The left guard was injured on the play."?': 'No',
r'Does the word "unleash" mean the same thing in sentences "He unleashed his dog in the park." and "Unleash the dogs in the park."?': 'Yes',
r'Does the word "glorify" mean the same thing in sentences "You are glorifying a rather mediocre building." and "Glorify one \'s spouse \'s cooking."?': 'No',
r'Does the word "storage" mean the same thing in sentences "There \'s a lot of storage space in the loft." and "I′d recommend backing up these files to storage before reinstalling the operating system."?': 'No',
r'Does the word "region" mean the same thing in sentences "The equatorial regions." and "Here we enter the region of opinion."?': 'No',
r'Does the word "rock" mean the same thing in sentences "Thou art Peter, and upon this rock I will build my church -- Gospel According to Matthew." and "There were rocks all over the garden."?': 'Yes',
r'Does the word "patch" mean the same thing in sentences "When ice skating, be sure to stay away from reeds, there \'s always thin patches of ice there and you could fall through." and "A patch of bad weather."?': 'Yes',
r'Does the word "zone" mean the same thing in sentences "There is a no-smoking zone that extends 25 feet outside of each entrance." and "The white zone is for loading and unloading only."?': 'Yes',
r'Does the word "score" mean the same thing in sentences "She scored high on the SAT." and "Score the SAT essays."?': 'No',
r'Does the word "scopolamine" mean the same thing in sentences "Someone sedated with scopolamine has difficulty lying." and "Transdermal scopolamine is used to treat motion sickness."?': 'Yes',
r'Does the word "rake" mean the same thing in sentences "The enemy machine guns raked the roadway." and "Rake leaves."?': 'No',
r'Does the word "emphasis" mean the same thing in sentences "He used a yellow highlighter to indicate where to give emphasis in his speech." and "His emphasis on civil rights."?': 'No',
r'Does the word "explosion" mean the same thing in sentences "An explosion of laughter." and "The information explosion."?': 'No',
r'Does the word "assume" mean the same thing in sentences "She assumed strange manners." and "Mr. Jones will assume the position of a lifeguard until a proper replacement is found."?': 'Yes',
r'Does the word "nightclub" mean the same thing in sentences "The gossip columnist got his information by visiting nightclubs every night." and "There are both gays and heteros in this nightclub."?': 'Yes',
r'Does the word "flex" mean the same thing in sentences "Flex a muscle." and "Flex your wrists."?': 'No',
r'Does the word "nail" mean the same thing in sentences "The chemists could not nail the identity of the chromosome." and "Nail the board onto the wall."?': 'No',
r'Does the word "form" mean the same thing in sentences "They formed a company." and "Social groups form everywhere."?': 'Yes',
r'Does the word "hoist" mean the same thing in sentences "Hoist a sail." and "Hoist the flags."?': 'Yes',
r'Does the word "sacrifice" mean the same thing in sentences "The animals were sacrificed after the experiment." and "The general had to sacrifice several soldiers to save the regiment."?': 'Yes',
r'Does the word "summon" mean the same thing in sentences "Summon all your courage." and "Summon a lawyer."?': 'No',
r'Does the word "bond" mean the same thing in sentences "Organic chemistry primarily consists of the study of carbon bonds, in their many variations." and "A $ 10,000 bond was furnished by an alderman."?': 'No',
r'Does the word "hem" mean the same thing in sentences "Let down the hem." and "It seeped along the hem of his jacket."?': 'Yes',
r'Does the word "right" mean the same thing in sentences "Mineral rights." and "We \'re on the side of right in this contest."?': 'Yes',
r'Does the word "feel" mean the same thing in sentences "The runner felt her pulse." and "I can feel the sadness in his poems."?': 'No',
r'Does the word "understand" mean the same thing in sentences "I understand him to be a good doctor." and "She did not understand her husband."?': 'Yes',
r'Does the word "have" mean the same thing in sentences "I \'ve got a house in the country." and "I have a house and a car."?': 'Yes',
r'Does the word "chop" mean the same thing in sentences "Chop meat." and "Chop a hole in the ground."?': 'No',
r'Does the word "massacre" mean the same thing in sentences "The massacre on St. Bartholomew \'s Day." and "St. Valentine \'s Day massacre."?': 'Yes',
r'Does the word "neck" mean the same thing in sentences "He admired her long graceful neck." and "The horse won by a neck."?': 'Yes',
r'Does the word "belt" mean the same thing in sentences "The rotund man had difficulty belting his pants, and generally wore suspenders to avoid the issue." and "Belt your trousers."?': 'Yes',
r'Does the word "nature" mean the same thing in sentences "The true nature of jealousy." and "Matters of a personal nature."?': 'No',
r'Does the word "pedal" mean the same thing in sentences "There are three pedals on manual cars, two on automatics." and "The pedal of a loom."?': 'Yes',
r'Does the word "cinema" mean the same thing in sentences "This story would be good cinema." and "The cinema is right across the street from the restaurant."?': 'No',
r'Does the word "stampede" mean the same thing in sentences "When he shouted ` fire \' there was a stampede to the exits." and "She and her husband would join in the general stampede. -W. Black."?': 'No',
r'Does the word "blind" mean the same thing in sentences "They had just moved in and had not put up blinds yet." and "He spent hours reading to the blind."?': 'No',
r'Does the word "totality" mean the same thing in sentences "The totality of war and its consequences." and "Appalled by the totality of the destruction."?': 'No',
r'Does the word "course" mean the same thing in sentences "The course was less than a mile." and "The course had only nine holes."?': 'Yes',
r'Does the word "offing" mean the same thing in sentences "There was a ship in the offing." and "There was a wedding in the offing."?': 'No',
r'Does the word "awareness" mean the same thing in sentences "I gradually passed from sleep to full awareness." and "The crash intruded on his awareness."?': 'Yes',
r'Does the word "remove" mean the same thing in sentences "He removed his children to the countryside." and "Remove a case to another court."?': 'Yes',
r'Does the word "snap" mean the same thing in sentences "He snapped his stick in anger." and "If you bend it too much, it will snap."?': 'Yes',
r'Does the word "style" mean the same thing in sentences "Style my hair." and "Style the dress."?': 'Yes',
r'Does the word "dignity" mean the same thing in sentences "Showed his true dignity when under pressure." and "It was beneath his dignity to cheat."?': 'Yes',
r'Does the word "constellate" mean the same thing in sentences "Hills constellated with lights." and "The poets constellate in this town every summer."?': 'No',
r'Does the word "roll" mean the same thing in sentences "The ball rolled." and "To roll a wheel, a ball, or a barrel."?': 'Yes',
r'Does the word "footing" mean the same thing in sentences "They were on a friendly footing." and "On a friendly footing."?': 'No',
r'Does the word "conjecture" mean the same thing in sentences "I explained it, but it is pure conjecture whether he understood, or not." and "The physicist used his conjecture about subatomic particles to design an experiment."?': 'No',
r'Does the word "mark" mean the same thing in sentences "His arrow hit the mark." and "I filled the bottle up to the 500ml mark."?': 'No',
r'Does the word "acknowledge" mean the same thing in sentences "The Crown Prince was acknowledged as the true heir to the throne." and "It is important to acknowledge the work of others in one \'s own writing."?': 'No',
r'Does the word "fall" mean the same thing in sentences "In the fall of 1973." and "When that became known the price of their stock went into free fall."?': 'No',
r'Does the word "pressure" mean the same thing in sentences "The public brought pressure to bear on the government." and "She has felt pressure lately because her boss expects her to get the job done by the first."?': 'No',
r'Does the word "identity" mean the same thing in sentences "She guessed the identity of his lover." and "The identity under numerical multiplication is 1."?': 'No',
r'Does the word "clear" mean the same thing in sentences "Clear your desk." and "Clear a debt."?': 'No',
r'Does the word "tantrum" mean the same thing in sentences "Many parents become embarrassed by their children throwing tantrums in public places." and "She threw a tantrum."?': 'Yes',
r'Does the word "theme" mean the same thing in sentences "It was the usual ` boy gets girl \' theme." and "The theme is announced in the first measures."?': 'No',
r'Does the word "kettle" mean the same thing in sentences "To cook pasta, you first need to put the kettle on." and "There \'s a hot kettle of soup on the stove."?': 'Yes',
r'Does the word "hedge" mean the same thing in sentences "Hedge the property." and "To hedge a field or garden."?': 'Yes',
r'Does the word "river" mean the same thing in sentences "Occasionally rivers overflow their banks and cause floods." and "The river was navigable for 50 miles."?': 'Yes',
r'Does the word "ingenuity" mean the same thing in sentences "Ingenuity is one of the characteristics of a beaver." and "Poverty is the mother of ingenuity."?': 'Yes',
r'Does the word "suit" mean the same thing in sentences "A flush is five cards in the same suit." and "In bridge you must follow suit."?': 'Yes',
r'Does the word "security" mean the same thing in sentences "He held several valuable securities." and "We support the armed services in the name of national security."?': 'No',
r'Does the word "preferment" mean the same thing in sentences "The preferment went to the younger candidate." and "Preferment of charges."?': 'No',
r'Does the word "share" mean the same thing in sentences "We shared the cake." and "I \'d like to share this idea with you."?': 'No',
r'Does the word "follow" mean the same thing in sentences "She followed dinner with a brandy." and "These people still follow the laws of their ancient religion."?': 'No',
r'Does the word "close" mean the same thing in sentences "Close a wound." and "Close a book."?': 'Yes',
r'Does the word "override" mean the same thing in sentences "The Congress promptly overrode the president \'s veto, passing the bill into law." and "Health considerations override financial concerns."?': 'No',
r'Does the word "bump" mean the same thing in sentences "My car bumped into the tree." and "Bump and grind."?': 'No',
r'Does the word "nest" mean the same thing in sentences "A nest of snipers." and "A nest of thieves."?': 'No',
r'Does the word "partition" mean the same thing in sentences "The Arab peninsula was partitioned by the British." and "Partition a room off."?': 'No',
r'Does the word "degree" mean the same thing in sentences "Murder in the second degree." and "A second degree burn."?': 'Yes',
r'Does the word "establishment" mean the same thing in sentences "The firm celebrated twenty years since their establishment by updating their look." and "He quickly became recognized as a member of the establishment."?': 'Yes',
r'Does the word "shock" mean the same thing in sentences "Corn is bound in small sheaves and several sheaves are set up together in shocks." and "Subjects received a small electric shock when they made the wrong response."?': 'No',
r'Does the word "march" mean the same thing in sentences "The march of science." and "The march of time."?': 'Yes',
r'Does the word "derivative" mean the same thing in sentences "` electricity \' is a derivative of ` electric \'." and "The derivative of f : f(x ) = x^2 is f\':f\'(x ) = 2x."?': 'No',
r'Does the word "reproach" mean the same thing in sentences "He brought reproach upon his family." and "Words of reproach."?': 'No',
r'Does the word "fortune" mean the same thing in sentences "Whatever my fortune may be." and "It was as if fortune guided his hand."?': 'No',
r'Does the word "cascade" mean the same thing in sentences "Progressing in severity as though a cascade of genetic damage was occurring." and "Separation of isotopes by a cascade of processes."?': 'Yes',
r'Does the word "enterprise" mean the same thing in sentences "A growing enterprise must have a bold leader." and "A micro enterprise is defined as a business having 5 or fewer employees and a low seed capital."?': 'Yes',
r'Does the word "entertain" mean the same thing in sentences "He entertained the notion of moving to South America." and "Entertain interesting notions."?': 'No',
r'Does the word "exile" mean the same thing in sentences "They chose exile rather than assimilation." and "Men in exile dream of hope."?': 'Yes',
r'Does the word "union" mean the same thing in sentences "Lightning produced an unusual union of the metals." and "There is strength in union."?': 'No',
r'Does the word "topic" mean the same thing in sentences "He kept drifting off the topic." and "It was a very sensitive topic."?': 'No',
r'Does the word "reading" mean the same thing in sentences "His main reading was detective stories." and "Suggestions for further reading."?': 'Yes',
r'Does the word "comply" mean the same thing in sentences "He complied with my instructions." and "You must comply or else !"?': 'Yes',
r'Does the word "immunity" mean the same thing in sentences "He was granted immunity from prosecution." and "The prosecutor offered the lieutenant immunity for all the crimes he would testify having known to be planned by the elusive drug baron."?': 'Yes',
r'Does the word "cloud" mean the same thing in sentences "Clouds of blossoms." and "He opened the door and was greeted by a cloud of bats."?': 'Yes',
r'Does the word "marshal" mean the same thing in sentences "Marshal facts or arguments." and "Marshal the troops."?': 'No',
r'Does the word "smokescreen" mean the same thing in sentences "Requesting new powers of surveillance is just a smokescreen to hide their failures." and "All that talk is just a smokescreen to disguise the fact that he has nothing to say."?': 'Yes',
r'Does the word "color" mean the same thing in sentences "Color has been a sensitive issue in many societies." and "She used a different color for the trim."?': 'No',
r'Does the word "channel" mean the same thing in sentences "Gutters carried off the rainwater into a series of channels under the street." and "Poison is released through a channel in the snake \'s fangs."?': 'No',
r'Does the word "climax" mean the same thing in sentences "The climax of the artist \'s career." and "The deathbed scene is the climax of the play."?': 'No',
r'Does the word "marry" mean the same thing in sentences "The minister married us on Saturday." and "A justice of the peace will marry Jones and Smith."?': 'Yes',
r'Does the word "penetrate" mean the same thing in sentences "She was penetrated with sorrow." and "The hikers did not manage to penetrate the dense forest."?': 'No',
r'Does the word "darkness" mean the same thing in sentences "His lectures dispelled the darkness." and "He moved off into the darkness."?': 'No',
r'Does the word "legitimation" mean the same thing in sentences "His parents \' subsequent marriage resulted in his legitimation." and "He has filial rights because he obtained letters of legitimation from the king."?': 'Yes',
r'Does the word "sex" mean the same thing in sentences "The film contained no sex or violence." and "He wanted a better sex life."?': 'Yes',
r'Does the word "theology" mean the same thing in sentences "He studied theology at Oxford." and "Jewish theology."?': 'No',
r'Does the word "production" mean the same thing in sentences "The widget making machine is being used for production now." and "They export most of their agricultural production."?': 'No',
r'Does the word "void" mean the same thing in sentences "He voided the check and returned it." and "Void a plea."?': 'Yes',
r'Does the word "question" mean the same thing in sentences "There was a question about my training." and "There is no question about the validity of the enterprise."?': 'No',
r'Does the word "fault" mean the same thing in sentences "It was John \'s fault." and "The fault lies with you."?': 'Yes',
r'Does the word "rounding" mean the same thing in sentences "The error in the calculation was attributable to rounding." and "Taxes are rounded off to the nearest dollar but the rounding error is surprisingly small."?': 'Yes',
r'Does the word "devotee" mean the same thing in sentences "He was a devotee of Arnold Schwarzenegger." and "A devotee of classical music."?': 'Yes',
r'Does the word "cancel" mean the same thing in sentences "The corrective feedback mechanism cancels out the noise." and "Cancel cheques or tickets."?': 'No',
r'Does the word "captain" mean the same thing in sentences "John Henry said to the captain, " A man ai n\'t nothing but a man. "." and "A captain of industry."?': 'Yes',
r'Does the word "signage" mean the same thing in sentences "The poor signage at Heathrow is only in English." and "The signage in the airport is designed to point the way clearly to important locations."?': 'Yes',
r'Does the word "despair" mean the same thing in sentences "One harsh word would send her into the depths of despair." and "He turned around in despair, aware that he was not going to survive."?': 'No',
r'Does the word "air" mean the same thing in sentences "Air travel involves too much waiting in airports." and "If you \'ve time to spare go by air."?': 'Yes',
r'Does the word "excursion" mean the same thing in sentences "Many summer excursions to the shore." and "While driving home I took an excursion and saw some deer."?': 'Yes',
r'Does the word "scan" mean the same thing in sentences "He made a thorough scan of the beach with his binoculars." and "You could see the tumor in the CAT scan."?': 'No',
r'Does the word "distress" mean the same thing in sentences "The patient appeared to be in distress." and "Originally distress was a landlord \'s remedy against a tenant for unpaid rents or property damage but now the landlord is given a landlord \'s lien."?': 'No',
r'Does the word "suppress" mean the same thing in sentences "This drug can suppress the hemorrhage." and "The rescue team managed to suppress the flow of oil by blasting the drilling hole."?': 'Yes',
r'Does the word "leakiness" mean the same thing in sentences "The leakiness of the boat made it dangerous to use." and "The leakiness of the roof."?': 'Yes',
r'Does the word "support" mean the same thing in sentences "She supported him during the illness." and "I support France in the World Cup."?': 'Yes',
r'Does the word "meter" mean the same thing in sentences "Meter the mail." and "Meter the flow of water."?': 'No',
r'Does the word "popularity" mean the same thing in sentences "His charm soon won him affection and popularity." and "This gallant laboring to avoid popularity falls into a habit of affectation. — Ben Jonson."?': 'Yes',
r'Does the word "face" mean the same thing in sentences "When he returned to work he met many new faces." and "An angry face."?': 'No',
r'Does the word "upset" mean the same thing in sentences "Everyone gets stomach upsets from time to time." and "The biggest upset since David beat Goliath."?': 'No',
r'Does the word "shopping" mean the same thing in sentences "Went shopping for a reliable plumber." and "Does her shopping at the mall rather than down town."?': 'Yes',
r'Does the word "wear" mean the same thing in sentences "Maternity wear." and "The tires showed uneven wear."?': 'No',
r'Does the word "substitute" mean the same thing in sentences "I had no shallots so I substituted onion." and "They tend to substitute for more mundane things."?': 'Yes',
r'Does the word "occasion" mean the same thing in sentences "On the occasion of his 60th birthday." and "At this point, she seized the occasion to make her own observation."?': 'Yes',
r'Does the word "gutter" mean the same thing in sentences "The cooling lava continued to gutter toward lower ground." and "Gutter the buildings."?': 'No',
r'Does the word "homer" mean the same thing in sentences "Each of the pigeon fanciers released a homer at the same time." and "The first baseman hit a homer to lead off the ninth."?': 'No',
r'Does the word "break" mean the same thing in sentences "He broke the men \'s 100-meter record." and "Break a record."?': 'Yes',
r'Does the word "suit" mean the same thing in sentences "If you take my advice, you \'ll file suit against him immediately." and "The family brought suit against the landlord."?': 'Yes',
r'Does the word "pretension" mean the same thing in sentences "The town still puts forward pretensions as a famous resort." and "His pretension to the crown."?': 'Yes',
r'Does the word "scream" mean the same thing in sentences "Her screaming attracted the neighbors." and "He heard the scream of the brakes."?': 'No',
r'Does the word "taker" mean the same thing in sentences "The study could not confirm the real percentage of drug takers in the country." and "She is known as quite a risk taker."?': 'Yes',
r'Does the word "corner" mean the same thing in sentences "The liquor store on the corner also sold lottery tickets." and "Standing on the corner watching all the girls go by."?': 'Yes',
r'Does the word "zero" mean the same thing in sentences "The probability is 1 in 1,000,000,000,000,000 — a number having five ciphers of zeros." and "The zero sign in American Sign Language is considered rude in some cultures."?': 'Yes',
r'Does the word "roominess" mean the same thing in sentences "Roominess in this size car is always a compromise." and "His unselfishness gave him great intellectual roominess."?': 'No',
r'Does the word "jump" mean the same thing in sentences "Every year, hundreds of people jump off the Golden Gate bridge." and "The parachutist did n\'t want to jump."?': 'Yes',
r'Does the word "drive" mean the same thing in sentences "He drives me mad." and "Can you drive this four-wheel truck ?"?': 'No',
r'Does the word "consist" mean the same thing in sentences "The payment consists in food." and "What does love consist in ?"?': 'Yes',
r'Does the word "bleed" mean the same thing in sentences "I bled him." and "Bleed the radiators."?': 'No',
r'Does the word "opinion" mean the same thing in sentences "My opinion differs from yours." and "Truth, in matters of religion, is simply the opinion that has survived. - Oscar Wilde."?': 'Yes',
r'Does the word "blood" mean the same thing in sentences "Blood carries oxygen and nutrients to the tissues and carries away waste products." and "The ancients believed that blood was the seat of the emotions."?': 'Yes',
r'Does the word "satisfaction" mean the same thing in sentences "The chef tasted the sauce with great satisfaction." and "The satisfaction of their demand for better services."?': 'No',
r'Does the word "adjoin" mean the same thing in sentences "I adjoin a copy of your my lawyer \'s letter." and "The living room and dining room adjoin each other."?': 'No',
r'Does the word "element" mean the same thing in sentences "To be in one \'s own element." and "Water is the element of fishes."?': 'No',
r'Does the word "bandwagon" mean the same thing in sentences "When they saw how things were going everybody jumped on the bandwagon." and "The gaudy bandwagon led the circus parade."?': 'No',
r'Does the word "setter" mean the same thing in sentences "Some crossword setters work for various newspapers under different pseudonyms." and "She has a spaniel and a red setter."?': 'No',
r'Does the word "palpitate" mean the same thing in sentences "His heart palpitated." and "When he just looks at me, my heart begins to palpitate with excitement."?': 'Yes',
r'Does the word "cry" mean the same thing in sentences "Animal cries filled the night." and "A cry of rage."?': 'No',
r'Does the word "roll" mean the same thing in sentences "They rolled their eyes at his words." and "Roll a cigarette."?': 'No',
r'Does the word "wade" mean the same thing in sentences "Wade the pond." and "Can we wade across the river to the other side ?"?': 'Yes',
r'Does the word "snap" mean the same thing in sentences "A cold snap in the middle of May." and "He can hear the snap of a twig."?': 'No',
r'Does the word "haul" mean the same thing in sentences "Haul stones from the quarry in a truck." and "Haul vegetables to the market."?': 'Yes',
r'Does the word "keep" mean the same thing in sentences "He is keeping three women in the guest cottage." and "Keep boarders."?': 'Yes',
r'Does the word "running" mean the same thing in sentences "The running of the engine is smooth." and "He has responsibility for the running of two companies at the same time."?': 'No',
r'Does the word "ingenuity" mean the same thing in sentences "The pyramids demonstrate the ingenuity of the ancient Egyptians." and "A plot of great ingenuity."?': 'No',
r'Does the word "tighten" mean the same thing in sentences "Tighten the wire." and "Please tighten that screw a quarter-turn."?': 'Yes',
r'Does the word "size" mean the same thing in sentences "The size of the building seemed to have increased since I was last there." and "Size gives body to a fabric."?': 'No',
r'Does the word "bombshell" mean the same thing in sentences "News of the attack came like a bombshell." and "She was a blonde bombshell."?': 'No',
r'Does the word "stay" mean the same thing in sentences "To stay a flat sheet in a steam boiler." and "Some of her friends decided it was time to go home, but she wanted to stay on until sunrise."?': 'No',
r'Does the word "miss" mean the same thing in sentences "He misses his mother." and "I miss you ! Come home soon !"?': 'Yes',
r'Does the word "press" mean the same thing in sentences "Put the ironing in the linen press." and "The press of business matters."?': 'No',
r'Does the word "dash" mean the same thing in sentences "He wooed her with the confident dash of a cavalry officer." and "He is preparing for the 100-yard dash."?': 'No',
r'Does the word "theory" mean the same thing in sentences "The architect has a theory that more is less." and "They killed him on the theory that dead men tell no tales."?': 'Yes',
r'Does the word "need" mean the same thing in sentences "God has no need of men to accomplish His work." and "She satisfied his need for affection."?': 'Yes',
r'Does the word "trade" mean the same thing in sentences "The bells traded places." and "Trade in an old car for a new one."?': 'Yes',
r'Does the word "rendition" mean the same thing in sentences "They heard a live rendition of three pieces by Schubert." and "Often imitations are extended to provide a more accurate rendition of the child \'s intended meaning."?': 'No',
r'Does the word "celebrate" mean the same thing in sentences "To celebrate a birthday." and "Celebrate Christmas."?': 'Yes',
r'Does the word "account" mean the same thing in sentences "The paper was rejected on account of its length." and "Do n\'t do it on my account."?': 'Yes',
r'Does the word "answer" mean the same thing in sentences "Their answer was to sue me." and "His answer to any problem was to get drunk."?': 'Yes',
r'Does the word "movement" mean the same thing in sentences "The second movement is slow and melodic." and "Politicians have to respect a mass movement."?': 'No',
r'Does the word "impulse" mean the same thing in sentences "Profound religious impulses." and "He bought it on an impulse."?': 'No',
r'Does the word "pause" mean the same thing in sentences "When telling the scary story, he paused for effect." and "We pause for station identification."?': 'No',
r'Does the word "intend" mean the same thing in sentences "She was intended to become the director." and "What do his words intend ?"?': 'No',
r'Does the word "efflux" mean the same thing in sentences "The efflux of matter from a boil can be painful." and "We all age through the efflux of time."?': 'Yes',
r'Does the word "devour" mean the same thing in sentences "She devoured his novels." and "She intended to devour the book."?': 'Yes',
r'Does the word "underlay" mean the same thing in sentences "Underlay the plate." and "To underlay a cut, plate, or the like, for printing."?': 'No',
r'Does the word "retie" mean the same thing in sentences "She stopped walking for a moment to retie her shoe." and "Retie the string and make it strong enough now."?': 'Yes',
r'Does the word "retroversion" mean the same thing in sentences "Retroversion of the uterus." and "The teacher translated Latin texts into English which he gave to his students for retroversion."?': 'No',
r'Does the word "drive" mean the same thing in sentences "Drive a car." and "Drive a car or bus."?': 'Yes',
r'Does the word "truncate" mean the same thing in sentences "Truncate a word." and "Truncate a series."?': 'No',
r'Does the word "conserve" mean the same thing in sentences "Energy is conserved in this process." and "Children must be taught to conserve our national heritage."?': 'No',
r'Does the word "manifestation" mean the same thing in sentences "A manifestation of great emotion." and "A manifestation of disease."?': 'No',
r'Does the word "major" mean the same thing in sentences "She is a linguistics major." and "She is a math major."?': 'Yes',
r'Does the word "pocketbook" mean the same thing in sentences "The publishers brought out small format pocketbooks of the whole of their nature series \'." and "That car is too expensive for my pocketbook."?': 'No',
r'Does the word "site" mean the same thing in sentences "A good site for the school." and "The Israeli web site was damaged by hostile hackers."?': 'No',
r'Does the word "folly" mean the same thing in sentences "This is a war of folly." and "Trying to drive through a blizzard is the height of folly."?': 'Yes',
r'Does the word "deliver" mean the same thing in sentences "Deliver a captive from the prison." and "Render a verdict ( i.e., deliver a judgment )."?': 'No',
r'Does the word "heating" mean the same thing in sentences "The heating system was n\'t working." and "They have radiant heating."?': 'Yes',
r'Does the word "chemistry" mean the same thing in sentences "Their chemistry was wrong from the beginning -- they hated each other." and "The chemistry of soil."?': 'No',
r'Does the word "bank" mean the same thing in sentences "That bank holds the mortgage on my home." and "He sat on the bank of the river and watched the currents."?': 'No',
r'Does the word "spin" mean the same thing in sentences "Spiders spin a fine web." and "Spin silk."?': 'No',
r'Does the word "blur" mean the same thing in sentences "The fog blurs my vision." and "To blur a photograph by moving the camera while taking it."?': 'Yes',
r'Does the word "martini" mean the same thing in sentences "Will you join us at six o\'clock for martinis ?" and "My boss is out on another of his three martini lunches."?': 'Yes',
r'Does the word "consumption" mean the same thing in sentences "The fire \'s consumption of the forest caused ecological changes." and "The consumption of energy has increased steadily."?': 'No',
r'Does the word "recognition" mean the same thing in sentences "A politician whose recall of names was as remarkable as his recognition of faces." and "Experimental psychologists measure the elapsed time from the onset of the stimulus to its recognition by the observer."?': 'Yes',
r'Does the word "assume" mean the same thing in sentences "He assumed to himself the right to fill all positions in the town." and "When will the new President assume office ?"?': 'No',
r'Does the word "stretch" mean the same thing in sentences "Stretch the limits." and "Stretch out that piece of cloth."?': 'No',
r'Does the word "violence" mean the same thing in sentences "The violence of the storm, fortunately, was more awesome than destructive." and "The storm \'s violence."?': 'Yes',
r'Does the word "nose" mean the same thing in sentences "Keep your nose out of it." and "She has a cold in the nose."?': 'No',
r'Does the word "distance" mean the same thing in sentences "There is a long distance between Moscow and Vladivostok." and "The distance to Petersborough is thirty miles."?': 'Yes',
r'Does the word "reserve" mean the same thing in sentences "Reserve me a seat on a flight." and "We managed to reserve a table at Maxim \'s."?': 'No',
r'Does the word "character" mean the same thing in sentences "A real character." and "The radical character of our demands."?': 'No',
r'Does the word "habit" mean the same thing in sentences "Owls have nocturnal habits." and "She had a habit twirling the ends of her hair."?': 'Yes',
r'Does the word "flight" mean the same thing in sentences "Birds are capable of flight." and "Take flight."?': 'No',
r'Does the word "coolant" mean the same thing in sentences "The atomic reactor used a gas coolant." and "He added more coolant to the car \'s radiator."?': 'Yes',
r'Does the word "beat" mean the same thing in sentences "I got beaten up by thugs on my way home." and "Beat the drum."?': 'No',
r'Does the word "bravura" mean the same thing in sentences "The music ends with a display of bravura." and "In a final bravura the ballerina appeared to be floating in water."?': 'Yes',
r'Does the word "type" mean the same thing in sentences "The word ` element \' contains five different types of character." and "The capable type."?': 'No',
r'Does the word "station" mean the same thing in sentences "The responsibilities of a man in his station." and "Married above her station."?': 'Yes',
r'Does the word "die" mean the same thing in sentences "Their anger died." and "My secret will die with me !"?': 'Yes',
r'Does the word "sound" mean the same thing in sentences "Sound a certain note." and "Sound the bell."?': 'Yes',
r'Does the word "engage" mean the same thing in sentences "Make sure the gear is engaged." and "The teeth of one cogwheel engage those of another."?': 'Yes',
r'Does the word "yoke" mean the same thing in sentences "They threw off the yoke of domination." and "Under the yoke of a tyrant."?': 'Yes',
r'Does the word "excavate" mean the same thing in sentences "Carnegie had a lake excavated for Princeton University \'s rowing team." and "Excavate gold."?': 'No',
r'Does the word "hearing" mean the same thing in sentences "They condemned him without a hearing." and "They make good music -- you should give them a hearing."?': 'No',
r'Does the word "complexion" mean the same thing in sentences "Diphthongs are complexions of vowels." and "A rugged complexion."?': 'No',
r'Does the word "rein" mean the same thing in sentences "He reined in his horses in front of the post office." and "Rein a horse."?': 'No',
r'Does the word "stretch" mean the same thing in sentences "A stretch of clear water." and "Beyond any stretch of his understanding."?': 'No',
r'Does the word "vagina" mean the same thing in sentences "The vagina is elastic enough to allow the passage of a fetus." and "The vagina receives the penis during coitus."?': 'Yes',
r'Does the word "unclothe" mean the same thing in sentences "She unclothed her innermost feelings." and "Unclothe your heart of envy."?': 'No',
r'Does the word "tiger" mean the same thing in sentences "It aroused the tiger in me." and "He \'s a tiger on the tennis court."?': 'Yes',
r'Does the word "population" mean the same thing in sentences "They hired hunters to keep down the deer population." and "The population of New Jersey will not stand for this !"?': 'Yes',
r'Does the word "smoke" mean the same thing in sentences "He \'s smoking his pipe." and "Do you smoke ?"?': 'Yes',
r'Does the word "spark" mean the same thing in sentences "A spark of decency." and "A spark of interest."?': 'Yes',
r'Does the word "notice" mean the same thing in sentences "He escaped the notice of the police." and "We received a notice to vacate the premises."?': 'No',
r'Does the word "clean" mean the same thing in sentences "Clean up before you see your grandparents." and "Clean your fingernails before dinner."?': 'Yes',
r'Does the word "population" mean the same thing in sentences "It is an estimate of the mean of the population." and "He deplored the population of colonies with convicted criminals."?': 'No',
r'Does the word "lot" mean the same thing in sentences "A lot of money." and "They were an angry lot."?': 'No',
r'Does the word "close" mean the same thing in sentences "Close up an umbrella." and "Close the circuit."?': 'Yes',
r'Does the word "concord" mean the same thing in sentences "Their ideas concorded." and "Both philosophers concord on this point."?': 'No',
r'Does the word "plant" mean the same thing in sentences "Plant a thought in the students \' minds." and "Plant fish."?': 'No',
r'Does the word "wind" mean the same thing in sentences "He put the key in the old clock and gave it a good wind." and "When there is no wind, row."?': 'No',
r'Does the word "plus" mean the same thing in sentences "A water molecule is made up of two hydrogen atoms plus one of oxygen." and "He is a real plus to the team."?': 'No',
r'Does the word "creation" mean the same thing in sentences "The restructure resulted in the creation of a number of shared services." and "From its creation the plan was doomed to failure."?': 'No',
r'Does the word "association" mean the same thing in sentences "His association of his father with being beaten was too strong to break." and "The host is not always injured by association with a parasite."?': 'No',
r'Does the word "evacuate" mean the same thing in sentences "After the earthquake, residents were evacuated." and "Evacuate the bottle."?': 'No',
r'Does the word "park" mean the same thing in sentences "The put the car in park and got out." and "There are laws that protect the wildlife in this park."?': 'No',
r'Does the word "grip" mean the same thing in sentences "A good grip on French history." and "He was in the grip of a powerful emotion."?': 'Yes',
r'Does the word "despair" mean the same thing in sentences "In the depths of despair." and "They were rescued from despair at the last minute."?': 'Yes',
r'Does the word "grow" mean the same thing in sentences "The child grew fast." and "We grow wheat here."?': 'No',
r'Does the word "man" mean the same thing in sentences "He taught me to set up the men on the chess board." and "He awaited word from his man in Havana."?': 'No',
r'Does the word "map" mean the same thing in sentences "Sorrow was mapped on the mother \'s face." and "We have n\'t even begun to map the many galaxies that we know exist."?': 'No',
r'Does the word "perceive" mean the same thing in sentences "She finally perceived the futility of her protest." and "I could perceive the ship coming over the horizon."?': 'No',
r'Does the word "people" mean the same thing in sentences "There were at least 200 people in the audience." and "Old people."?': 'Yes',
r'Does the word "performance" mean the same thing in sentences "An inspired performance of Mozart \'s C minor concerto." and "We congratulated him on his performance at the rehearsal."?': 'Yes',
r'Does the word "definition" mean the same thing in sentences "Exercise had given his muscles superior definition." and "The definition of a telescope."?': 'Yes',
r'Does the word "pleasure" mean the same thing in sentences "He was tingling with pleasure." and "The pleasure of his company."?': 'No',
r'Does the word "refer" mean the same thing in sentences "Refer to your notes." and "Refer a bill to a committee."?': 'No',
r'Does the word "voice" mean the same thing in sentences "A singer takes good care of his voice." and "After the fire a still small voice. — 1 Kings 19:12."?': 'Yes',
r'Does the word "leviathan" mean the same thing in sentences "They were assigned the leviathan of textbooks." and "It was a leviathan among redwoods."?': 'Yes',
r'Does the word "endorse" mean the same thing in sentences "Endorse a new project." and "Endorse cheques."?': 'No',
r'Does the word "security" mean the same thing in sentences "Bankers are reluctant to lend without good security." and "The watch dog gave her a feeling of security."?': 'No',
r'Does the word "grace" mean the same thing in sentences "It was debated whether saving grace could be obtained outside the membership of the church." and "The Virgin lived in a state of grace."?': 'Yes',
r'Does the word "hiccup" mean the same thing in sentences "He has the hiccups - give him a glass of water." and "There was a loud hiccup from the back of the room and the class erupted in laughter."?': 'Yes',
r'Does the word "deliverable" mean the same thing in sentences "Under this contract the deliverables include both software and hardware." and "We packaged the deliverable, a program called FLOOD.EXE, in an installer file."?': 'Yes',
r'Does the word "chance" mean the same thing in sentences "There is a 30 % chance of rain tomorrow." and "We ran into each other by pure chance."?': 'No',
r'Does the word "surrender" mean the same thing in sentences "The last Taleban fighters finally surrendered." and "The squatters had to surrender the building after the police moved in."?': 'No',
r'Does the word "twin" mean the same thing in sentences "The polished surface twinned his face and chest in reverse." and "Twin crystals."?': 'No',
r'Does the word "submit" mean the same thing in sentences "The lawyers submitted the material to the court." and "I submit these plans for your approval."?': 'Yes',
r'Does the word "selection" mean the same thing in sentences "I \'ve brought a selection of fine cheeses to go with your wine." and "The store carried a large selection of shoes."?': 'No',
r'Does the word "patent" mean the same thing in sentences "I patented my idea." and "Should I patent this invention ?"?': 'Yes',
r'Does the word "enjoy" mean the same thing in sentences "The industry enjoyed a boom." and "Enjoy your holidays !"?': 'No',
r'Does the word "overstate" mean the same thing in sentences "I think it is overstating matters to say that an hour online is spending all night on the computer." and "He was suggested not to overstate at the interview."?': 'Yes',
r'Does the word "salute" mean the same thing in sentences "A terrible stench saluted our nostrils." and "When the officers show up, the soldiers have to salute."?': 'No',
r'Does the word "execute" mean the same thing in sentences "The President executed the treaty." and "To execute a turn in ballet."?': 'No',
r'Does the word "end" mean the same thing in sentences "Both ends wrote at the same time." and "The phone rang at the other end."?': 'Yes',
r'Does the word "principle" mean the same thing in sentences "He will not violate his principles." and "A man of principle."?': 'Yes',
r'Does the word "baste" mean the same thing in sentences "Baste a hem." and "Baste a roast."?': 'No',
r'Does the word "calve" mean the same thing in sentences "The icebergs and glaciers calve." and "The whales calve at this time of year."?': 'No',
r'Does the word "construction" mean the same thing in sentences "During the construction we had to take a detour." and "She wore her hair in an amazing construction of whirls and ribbons."?': 'No',
r'Does the word "fill" mean the same thing in sentences "Fill the child with pride." and "The bucket filled with rain."?': 'Yes',
r'Does the word "replay" mean the same thing in sentences "We replayed the game." and "Replay a point."?': 'Yes',
r'Does the word "fence" mean the same thing in sentences "We fenced in our yard." and "If we fence in that field, it will be a good pasture for a horse."?': 'Yes',
r'Does the word "efficiency" mean the same thing in sentences "She did the work with great efficiency." and "The efficiency of the planning department is deplorable."?': 'Yes',
r'Does the word "radiate" mean the same thing in sentences "Spokes radiate from the hub of the wheel." and "This plants radiate spines in all directions."?': 'Yes',
r'Does the word "back" mean the same thing in sentences "The car backed up and hit the tree." and "To back the oars."?': 'Yes',
r'Does the word "seize" mean the same thing in sentences "The movie seized my imagination." and "To seize a ship after libeling."?': 'No',
r'Does the word "test" mean the same thing in sentences "When the test was stolen the professor had to make a new set of questions." and "The test was standardized on a large sample of students."?': 'No',
r'Does the word "steamroller" mean the same thing in sentences "The Senator steamrollered the bill to defeat." and "Steamroller the road."?': 'No',
r'Does the word "produce" mean the same thing in sentences "Produce a new play." and "The tree would not produce fruit."?': 'No',
r'Does the word "leg" mean the same thing in sentences "Dan wo n\'t be able to come to the party, since he broke his leg last week and is now on crutches." and "After six days, we \'re finally in the last leg of our cross-country trip."?': 'No',
r'Does the word "attest" mean the same thing in sentences "I attest this signature." and "When will the appraiser attest the date of the painting ?"?': 'Yes',
r'Does the word "attention" mean the same thing in sentences "The old car needs constant attention." and "She was the center of attention."?': 'No',
r'Does the word "richness" mean the same thing in sentences "The cut of her clothes and the richness of the fabric were distinctive." and "They studied forerunners of richness or poverty."?': 'No',
r'Does the word "kid" mean the same thing in sentences "They were able to send their kids to college." and "Kid gloves."?': 'No',
r'Does the word "purse" mean the same thing in sentences "He made the contribution out of his own purse." and "He and his wife shared a common purse."?': 'Yes',
r'Does the word "park" mean the same thing in sentences "Can you park right here ?" and "Park the children with the in-laws."?': 'No',
r'Does the word "month" mean the same thing in sentences "He paid the bill last month." and "July is my favourite month."?': 'Yes',
r'Does the word "rotation" mean the same thing in sentences "The rotation of the dancer kept time with the music." and "Crop rotation."?': 'No',
r'Does the word "turnout" mean the same thing in sentences "The turnout for the rally." and "A large turnout for the meeting."?': 'No',
r'Does the word "silence" mean the same thing in sentences "When the motor stopped, the silence was almost deafening." and "He needed silence in order to sleep."?': 'Yes',
r'Does the word "shade" mean the same thing in sentences "They used umbrellas as shades." and "As the sun moved he readjusted the shade."?': 'Yes',
r'Does the word "patient" mean the same thing in sentences "The number of emergency patients has grown rapidly." and "The subject of a passive verb is usually a patient."?': 'No',
r'Does the word "probe" mean the same thing in sentences "If you probe further, you may discover different reasons." and "Probe an anthill."?': 'No',
r'Does the word "effect" mean the same thing in sentences "The effect of the hurricane was a devastated landscape." and "The magnetic effect was greater when the rod was lengthwise."?': 'Yes',
r'Does the word "add" mean the same thing in sentences "This adds a light note to the program." and "To add numbers."?': 'No',
r'Does the word "apply" mean the same thing in sentences "He applied this racial slur to me !" and "The same laws apply to you !"?': 'No',
r'Does the word "local" mean the same thing in sentences "The local seemed to take forever to get to New York." and "The expresses skipped my station, so I had to take a local."?': 'Yes',
r'Does the word "blow" mean the same thing in sentences "To blow a horse." and "Blow a horse."?': 'Yes',
r'Does the word "back" mean the same thing in sentences "The titles are printed on the backs of the books." and "The fall broke his back."?': 'No',
r'Does the word "swarm" mean the same thing in sentences "The garden was swarming with bees." and "There swarm bees in the garden."?': 'Yes',
r'Does the word "truth" mean the same thing in sentences "He was famous for the truth of his portraits." and "The situation brought home to us the blunt truth of the military threat."?': 'Yes',
r'Does the word "heat" mean the same thing in sentences "The massage heated her up." and "I \'ll heat up the water."?': 'No',
r'Does the word "progress" mean the same thing in sentences "Science has made extraordinary progress in the last fifty years." and "Great progress in the arts."?': 'Yes',
r'Does the word "rascal" mean the same thing in sentences "If you have deer in the area, you may have to put a fence around your garden to keep the rascals out." and "That little rascal bit me !"?': 'Yes',
r'Does the word "party" mean the same thing in sentences "I \'m throwing a huge party for my 21st birthday." and "The contract requires that the party of the first part pay the fee."?': 'No',
r'Does the word "affinity" mean the same thing in sentences "In anatomical structure prehistoric man shows close affinity with modern humans." and "The mysterious affinity between them."?': 'No',
r'Does the word "model" mean the same thing in sentences "The sculptor modelled the clay into the form of a dolphin." and "Model a head with clay."?': 'Yes',
r'Does the word "suppress" mean the same thing in sentences "Hot blackcurrant juice mixed with honey may suppress cough." and "Suppress a yawn."?': 'Yes',
r'Does the word "give" mean the same thing in sentences "Give one \'s talents to a good cause." and "Give a divorce."?': 'No',
r'Does the word "defensibility" mean the same thing in sentences "They built their castles with an eye to their defensibility." and "Client complaints create a felt need for the defensibility of individual actions."?': 'Yes',
r'Does the word "afterthought" mean the same thing in sentences "This whole remodeling was an afterthought, we were only going to paint and call it good." and "The garage was an afterthought."?': 'Yes',
r'Does the word "wave" mean the same thing in sentences "A heat wave." and "A wave of emotion swept over him."?': 'No',
r'Does the word "take" mean the same thing in sentences "Act seven, scene three, take two." and "The take must be repeated."?': 'Yes',
r'Does the word "rail" mean the same thing in sentences "She railed against the bad social policies." and "Rail off the crowds from the Presidential palace."?': 'No',
r'Does the word "begin" mean the same thing in sentences "This economic measure does n\'t even begin to deal with the problem of inflation." and "You can not even begin to understand the problem we had to deal with during the war."?': 'Yes',
r'Does the word "exploit" mean the same thing in sentences "He is exploiting the students." and "We must exploit the resources we are given wisely."?': 'No',
r'Does the word "bitch" mean the same thing in sentences "Ann gossiped about me and mocked my work ; sometimes she can be a real bitch !" and "She said her son thought Hillary was a bitch."?': 'Yes',
r'Does the word "frost" mean the same thing in sentences "She frosts her hair." and "Frost the glass."?': 'Yes',
r'Does the word "kindling" mean the same thing in sentences "The kindlings of love." and "Go and collect some kindling."?': 'No',
r'Does the word "treat" mean the same thing in sentences "Treat him with caution, please." and "Treat the water so it can be drunk."?': 'No',
r'Does the word "screen" mean the same thing in sentences "Screen the suitcases." and "Screen the job applicants."?': 'No',
r'Does the word "magnitude" mean the same thing in sentences "They tried to predict the magnitude of the explosion." and "About the magnitude of a small pea."?': 'Yes',
r'Does the word "hand" mean the same thing in sentences "He handed them the letter." and "Hand me the spoon, please."?': 'Yes',
r'Does the word "poor" mean the same thing in sentences "The poor are always with us." and "The urban poor need assistance."?': 'Yes',
r'Does the word "mistake" mean the same thing in sentences "Make no mistake about his intentions." and "He made a bad mistake."?': 'No',
r'Does the word "sleep" mean the same thing in sentences "He did n\'t get enough sleep last night." and "We need to conduct an overnight sleep test to diagnose your sleep problem."?': 'Yes',
r'Does the word "impose" mean the same thing in sentences "I can not impose my opinion." and "I can not impose that I get what I want."?': 'Yes',
r'Does the word "roughness" mean the same thing in sentences "Men associate the roughness of nonstandard working-class speech with masculinity." and "The roughness of her voice was a signal to keep quiet."?': 'Yes',
r'Does the word "feedstock" mean the same thing in sentences "Bio-materials may be substituted for petrochemical feedstocks as petroleum prices rise." and "The filtered extract of the mash is the feedstock for the fermenter."?': 'Yes',
r'Does the word "round" mean the same thing in sentences "Hospital rounds." and "We enjoyed our round of the local bars."?': 'No',
r'Does the word "deuce" mean the same thing in sentences "The deuce with it." and "Love is a bodily infirmity... which breaks out the deuce knows how or why ( Thackeray )."?': 'Yes',
r'Does the word "lodge" mean the same thing in sentences "We are lodging three foreign students this semester." and "Lodge a bullet in the table."?': 'No',
r'Does the word "pillar" mean the same thing in sentences "A thin pillar of smoke betrayed their campsite." and "A pillar of smoke."?': 'No',
r'Does the word "pluck" mean the same thing in sentences "She plucked the phone from her bag and dialled." and "Pluck the flowers off the bush."?': 'Yes',
r'Does the word "tense" mean the same thing in sentences "Tense the rope manually before tensing the spring." and "Alternately relax and tense your calf muscle."?': 'Yes',
r'Does the word "inexperience" mean the same thing in sentences "Procedural inexperience created difficulties." and "I failed to get the job due to inexperience."?': 'Yes',
r'Does the word "show" mean the same thing in sentences "She shows her dogs frequently." and "All he had to show for four years of attendance at college was a framed piece of paper."?': 'Yes',
r'Does the word "narcolepsy" mean the same thing in sentences "Excitement induced narcolepsy caused him to sleep through the most important events of his life." and "He believes that narcolepsy is attributable to an inability to suppress REM sleep during waking."?': 'Yes',
r'Does the word "cast" mean the same thing in sentences "Cast a spell." and "Cast lots."?': 'No',
r'Does the word "deck" mean the same thing in sentences "Deck the halls with holly." and "Wow, did you see her deck that guy who pinched her ?"?': 'No',
r'Does the word "heartbeat" mean the same thing in sentences "Music is the heartbeat of the people." and "New York is the commercial heartbeat of America."?': 'Yes',
r'Does the word "cradle" mean the same thing in sentences "The cradle of liberty." and "He was taught from the cradle never to cry."?': 'No',
r'Does the word "path" mean the same thing in sentences "The path of a meteor, of a caravan, or of a storm." and "The hurricane demolished houses in its path."?': 'Yes',
r'Does the word "craft" mean the same thing in sentences "He learned his craft as an apprentice." and "He represented the craft of brewers."?': 'Yes',
r'Does the word "change" mean the same thing in sentences "It is a refreshing change to meet a woman mechanic." and "He attributed the change to their marriage."?': 'No',
r'Does the word "bar" mean the same thing in sentences "The street was lined with all-night bars." and "He drowned his sorrows in whiskey at the bar."?': 'Yes',
r'Does the word "fact" mean the same thing in sentences "How much of the story is fact and how much fiction is hard to tell." and "Your fears have no basis in fact."?': 'Yes',
r'Does the word "header" mean the same thing in sentences "This wall has four header courses." and "Your header is too long ; " Local Cannibals " will suffice."?': 'No',
r'Does the word "host" mean the same thing in sentences "Viruses depend on the host that they infect in order to be able to reproduce." and "Atlanta was chosen to be host for the Olympic Games."?': 'No',
r'Does the word "set" mean the same thing in sentences "We set up the sprinkler." and "Set a broken bone."?': 'No',
r'Does the word "disentangle" mean the same thing in sentences "How can I disentangle myself from her personal affairs ?" and "Can you disentangle the cord ?"?': 'No',
r'Does the word "debt" mean the same thing in sentences "I am in your debt." and "He is badly in debt."?': 'Yes',
r'Does the word "call" mean the same thing in sentences "The unions called a general strike for Sunday." and "I would not call her beautiful."?': 'No',
r'Does the word "raise" mean the same thing in sentences "Raise an army." and "We raise hogs here."?': 'No',
r'Does the word "ply" mean the same thing in sentences "Ply the seven seas." and "Ply an axe."?': 'No',
r'Does the word "help" mean the same thing in sentences "I \'ve printed out a list of math helps." and "He was a great help to me when I was moving house."?': 'Yes',
r'Does the word "underbelly" mean the same thing in sentences "The area by the canal is the underbelly of town - not something you \'d see in the guidebooks." and "The soft underbelly of the Axis- Winston Churchill."?': 'No',
r'Does the word "personality" mean the same thing in sentences "She is a Hollywood personality." and "The president has a unique personality."?': 'Yes',
r'Does the word "release" mean the same thing in sentences "A new release from the London Symphony Orchestra." and "The release of iodine from the thyroid gland."?': 'No',
r'Does the word "obstruction" mean the same thing in sentences "The filibuster was a major obstruction to the success of their plan." and "Obstruction of justice."?': 'No',
r'Does the word "detest" mean the same thing in sentences "She detests politicians." and "I detest snakes."?': 'Yes',
r'Does the word "triumph" mean the same thing in sentences "The agreement was a triumph for common sense." and "The triumph of knowledge."?': 'Yes',
r'Does the word "number" mean the same thing in sentences "Zero, one, -1, 2.5, and pi are all numbers." and "The word “ fifteen ”, the Arabic numeral “ 15 ”, and the Roman numeral “ XV ” are all numerals representing the number fifteen."?': 'Yes',
r'Does the word "stock" mean the same thing in sentences "They will cut round stock to 1-inch diameter." and "He owns a controlling share of the company \'s stock."?': 'No',
r'Does the word "condition" mean the same thing in sentences "He conditions old cars." and "I condition my hair after washing it."?': 'No',
r'Does the word "anesthetize" mean the same thing in sentences "The patient must be anesthetized before the operation." and "Anesthetize the gum before extracting the teeth."?': 'Yes',
r'Does the word "active" mean the same thing in sentences "` The boy threw the ball \' uses the active voice." and "The ball was kicked by Fred. would, in active voice, be " Fred kicked the ball. "."?': 'Yes',
r'Does the word "lid" mean the same thing in sentences "His lids would stay open no longer." and "He raised the piano lid."?': 'No',
r'Does the word "raise" mean the same thing in sentences "He got a 3 % raise." and "The boss gave me a raise."?': 'Yes',
r'Does the word "scare" mean the same thing in sentences "A war scare." and "A bomb scare led them to evacuate the building."?': 'Yes',
r'Does the word "drive" mean the same thing in sentences "His drive and energy exhausted his co-workers." and "He sliced his drive out of bounds."?': 'No',
r'Does the word "step" mean the same thing in sentences "The police took steps to reduce crime." and "He paused on the bottom step."?': 'No',
r'Does the word "value" mean the same thing in sentences "I establish the colors and principal values by organizing the painting into three values -- dark, medium... and light - Joe Hing Lowe." and "The value assigned was 16 milliseconds."?': 'No',
r'Does the word "standing" mean the same thing in sentences "Of equal standing." and "A member in good standing."?': 'Yes',
r'Does the word "confront" mean the same thing in sentences "John confronted it with emergency measures." and "You must confront your opponent."?': 'Yes',
r'Does the word "assent" mean the same thing in sentences "He gave his assent eagerly." and "I will give this act my assent."?': 'Yes',
r'Does the word "refocus" mean the same thing in sentences "Refocus the image until it is very sharp." and "The group needs to refocus its goals."?': 'No',
r'Does the word "precedence" mean the same thing in sentences "Family takes precedence over work, in an emergency." and "Its precedence as the world \'s leading manufacturer of pharmaceuticals."?': 'Yes',
r'Does the word "bank" mean the same thing in sentences "He banked with Barclays." and "Where do you bank in this town ?"?': 'Yes',
r'Does the word "squeeze" mean the same thing in sentences "Increased expenses put a squeeze on profits." and "A gentle squeeze on the arm."?': 'No',
r'Does the word "light" mean the same thing in sentences "There was a light at every corner." and "They saw the light of the beacon."?': 'Yes',
r'Does the word "projection" mean the same thing in sentences "Our ukuleles have been designed to have superior sound and projection." and "A prime ingredient of public speaking is projection of the voice."?': 'Yes',
r'Does the word "lover" mean the same thing in sentences "A lover of his country." and "A lover of fine wines."?': 'Yes',
r'Does the word "comparison" mean the same thing in sentences "To bring a thing into comparison with another." and "They made a comparison of noise levels."?': 'Yes',
r'Does the word "pack" mean the same thing in sentences "The nurse packed gauze in the wound." and "You had better pack your swollen ankle with ice."?': 'Yes',
r'Does the word "screen" mean the same thing in sentences "They put screens in the windows for protection against insects." and "A metal screen protected the observers."?': 'Yes',
r'Does the word "press" mean the same thing in sentences "He pressed down on the boards." and "Press your thumb on this spot."?': 'Yes',
r'Does the word "irritate" mean the same thing in sentences "Aspirin irritates my stomach." and "Irritate the glands of a leaf."?': 'No',
r'Does the word "unemployed" mean the same thing in sentences "The unemployed are a growing portion of the population." and "The long-term unemployed need assistance."?': 'Yes',
r'Does the word "deflate" mean the same thing in sentences "The bad review of his work deflated his self-confidence." and "Deflate a balloon."?': 'No',
r'Does the word "mold" mean the same thing in sentences "The furniture molded in the old house." and "Mold public opinion."?': 'No',
r'Does the word "top" mean the same thing in sentences "Top trees and bushes." and "I do n\'t want to be bald, so just top my hair."?': 'Yes',
r'Does the word "freshness" mean the same thing in sentences "She loved the freshness of newly baked bread." and "The freshness of the air revived him."?': 'Yes',
r'Does the word "dawn" mean the same thing in sentences "It was the dawn of the Roman Empire." and "The dawn of civilization."?': 'No',
r'Does the word "slave" mean the same thing in sentences "Art thou the slave that with thy breath hast kill\'d/ Mine innocent child ? Shakespeare. Much Ado About Nothing." and "His mother was his abject slave."?': 'Yes',
r'Does the word "march" mean the same thing in sentences "The march went up Fifth Avenue." and "It was a long march."?': 'No',
r'Does the word "trim" mean the same thing in sentences "Trim lumber." and "They traditionally trim the tree on Christmas Eve."?': 'No',
r'Does the word "overwinter" mean the same thing in sentences "Shackleton \'s men overwintered on Elephant Island." and "Insects may overwinter in fallen fruit if it is not removed."?': 'Yes',
r'Does the word "analogy" mean the same thing in sentences "The operation of a computer presents and interesting analogy to the working of the brain." and "The models show by analogy how matter is built up."?': 'Yes',
r'Does the word "wanton" mean the same thing in sentences "The young man wantoned away his inheritance." and "Wanton one \'s money away."?': 'Yes',
r'Does the word "escape" mean the same thing in sentences "They installed a second hatch as an escape." and "Their escape route."?': 'Yes',
r'Does the word "section" mean the same thing in sentences "He always turns first to the business section." and "No section of the nation is more ardent than the South."?': 'No',
r'Does the word "playbook" mean the same thing in sentences "The 1963 playbook leaves out the whole first scene." and "They borrowed a page from the playbook of the opposition."?': 'No',
r'Does the word "spin" mean the same thing in sentences "The car went into a spin." and "One of the planet \'s moons has a slower spin than the others."?': 'Yes',
r'Does the word "cover" mean the same thing in sentences "This farm covers some 200 acres." and "When the pot comes to a boil, cover it and reduce the heat to medium."?': 'No',
r'Does the word "truth" mean the same thing in sentences "Hunger and jealousy are just eternal truths of human existence." and "He thought of answering with the truth but he knew they would n\'t believe it."?': 'Yes',
r'Does the word "splash" mean the same thing in sentences "Just a splash of whiskey." and "Her red hat gave her outfit a splash of color."?': 'No',
r'Does the word "match" mean the same thing in sentences "These two copies are supposed to be identical, but they do n\'t match." and "Their interests did n\'t match, so it took a long time to agree what to do together."?': 'Yes',
r'Does the word "unearth" mean the same thing in sentences "The CIA unearthed a plot to kill the President." and "To unearth a secret."?': 'Yes',
r'Does the word "bend" mean the same thing in sentences "The road bends to the right." and "Copper rods bend easily."?': 'No',
r'Does the word "reason" mean the same thing in sentences "We are told that man is endowed with reason and capable of distinguishing good from evil." and "The reason I robbed the bank was that I needed the money."?': 'No',
r'Does the word "earth" mean the same thing in sentences "She sighed when the plane \'s wheels finally touched earth." and "The earth shook for several minutes."?': 'No',
r'Does the word "shower" mean the same thing in sentences "Would male strippers be appropriate for the divorce shower ?" and "Her church group has planned an adoption shower."?': 'Yes',
r'Does the word "warm" mean the same thing in sentences "The soup warmed slowly on the stove." and "He came inside to warm up."?': 'Yes',
r'Does the word "render" mean the same thing in sentences "She rendered the French poem into English." and "To render Latin into English."?': 'Yes',
r'Does the word "reunite" mean the same thing in sentences "Two of the members tried several times, but failed to reunite the band." and "After ten years apart, the band will reunite."?': 'Yes',
r'Does the word "decree" mean the same thing in sentences "The King decreed that all firstborn males should be killed." and "The King only can decree."?': 'No',
r'Does the word "planting" mean the same thing in sentences "He ordered the planting of policemen outside every doorway." and "The landscape architect suggested a small planting in the northwest corner."?': 'No',
r'Does the word "slick" mean the same thing in sentences "The oil slick has now spread to cover the entire bay, critically endangering the sea life." and "Careful in turn three — there \'s an oil slick on the road."?': 'Yes',
r'Does the word "heartland" mean the same thing in sentences "The home counties are the Conservative heartland." and "It was in the heartland of the French wine growing areas."?': 'Yes',
r'Does the word "gum" mean the same thing in sentences "These trees gum in the Spring." and "If you gum the tape it is stronger."?': 'No',
r'Does the word "handwriting" mean the same thing in sentences "She recognized his handwriting." and "Handwriting can be slow and painful for one with arthritis."?': 'No',
r'Does the word "mail" mean the same thing in sentences "I \'ll mail you the check tomorrow." and "I \'ll mail you the paper when it \'s written."?': 'No',
r'Does the word "instinct" mean the same thing in sentences "Altruistic instincts in social animals." and "Many animals fear fire by instinct."?': 'Yes',
r'Does the word "recourse" mean the same thing in sentences "His only recourse was the police." and "Have recourse to the courts."?': 'No',
r'Does the word "exchange" mean the same thing in sentences "Deductible losses on sales or exchanges of property are allowable." and "All in all, it was an even exchange."?': 'No',
r'Does the word "tube" mean the same thing in sentences "No mate, I am taking the tube !" and "In Paris the subway system is called the ` metro \' and in London it is called the ` tube \' or the ` underground \'."?': 'Yes',
r'Does the word "nature" mean the same thing in sentences "It is his nature to help others." and "They tried to preserve nature as they found it."?': 'No',
r'Does the word "play" mean the same thing in sentences "He is playing his cards close to his chest." and "The Democrats still have some cards to play before they will concede the electoral victory."?': 'Yes',
r'Does the word "alleviator" mean the same thing in sentences "Aspirin is an alleviator of headaches." and "Blessed is he who is an alleviator of suffering."?': 'Yes',
r'Does the word "involvement" mean the same thing in sentences "There was additional involvement of the liver and spleen." and "He escaped involvement in the accident."?': 'Yes',
r'Does the word "catch" mean the same thing in sentences "Did you catch that allusion ?" and "If you leave now you might catch him."?': 'No',
r'Does the word "communication" mean the same thing in sentences "A secret passageway provided communication between the two rooms." and "The node had established communication with the network, but had as yet sent no data."?': 'No',
r'Does the word "walk" mean the same thing in sentences "Walk with God." and "The patient can not walk yet."?': 'No',
r'Does the word "approach" mean the same thing in sentences "The approach of winter." and "He lost the hole when his approach rolled over the green."?': 'No',
r'Does the word "truth" mean the same thing in sentences "They debated the truth of the proposition." and "He told the truth."?': 'No',
r'Does the word "constituent" mean the same thing in sentences "Needs continued support by constituents to be re-elected." and "The animal constituent of plankton."?': 'No',
r'Does the word "demand" mean the same thing in sentences "The demand exceeded the supply." and "The automobile reduced the demand for buggywhips."?': 'Yes',
r'Does the word "smolder" mean the same thing in sentences "A smoldering fire." and "The remains of the bonfire were left to smolder for hours."?': 'Yes',
r'Does the word "place" mean the same thing in sentences "He placed the glass on the table." and "Place a bet."?': 'No',
r'Does the word "manage" mean the same thing in sentences "She managed her parents \' affairs after they got too old." and "The young violinist did n\'t manage her bow very well."?': 'No',
r'Does the word "iceberg" mean the same thing in sentences "The Titanic hit an iceberg and sank." and "Iceberg is still the most popular lettuce."?': 'No',
r'Does the word "demand" mean the same thing in sentences "The boss demanded that he be fired immediately." and "I demand to see the manager."?': 'Yes',
r'Does the word "react" mean the same thing in sentences "She reacts negatively to everything I say." and "The hydrogen and the oxygen react."?': 'No',
r'Does the word "symbol" mean the same thing in sentences "The lion is the symbol of courage." and "The lamb is the symbol of meekness or patience."?': 'Yes',
r'Does the word "identification" mean the same thing in sentences "The thief \'s identification was followed quickly by his arrest." and "Identification of the gun was an important clue."?': 'Yes',
r'Does the word "pig" mean the same thing in sentences "The conveyor carried the pigs from the smelter to the freight cars." and "After the ill-advised trade, the investor was stuck with worthless options for 10,000 tons of iron pig."?': 'Yes',
r'Does the word "slice" mean the same thing in sentences "A slice of bread." and "A slice of the company \'s revenue."?': 'No',
r'Does the word "work" mean the same thing in sentences "He is working his servants hard." and "Work the body onto the flatbed truck."?': 'No',
r'Does the word "pick" mean the same thing in sentences "Pick mushrooms." and "Pick flowers."?': 'Yes',
r'Does the word "smear" mean the same thing in sentences "She smeared her lips with lipstick." and "Smear the wall with paint."?': 'Yes',
r'Does the word "gesture" mean the same thing in sentences "This Web browser can be controlled with mouse gestures." and "The middle-finger gesture is really a nonverbal swear."?': 'Yes',
r'Does the word "charge" mean the same thing in sentences "The battle began with a cavalry charge." and "The wrestler \'s charge carried him past his adversary."?': 'Yes',
r'Does the word "discovery" mean the same thing in sentences "The purpose of the voyage was discovery." and "The defense argued that the plaintiff \'s discovery was inadequate."?': 'No',
r'Does the word "contribution" mean the same thing in sentences "He expected his contribution to be repaid with interest." and "I am proud of my contribution in advancing the project."?': 'No',
r'Does the word "manner" mean the same thing in sentences "Her dignified manner." and "What manner of man are you ?"?': 'No',
r'Does the word "value" mean the same thing in sentences "The dealer valued the book." and "Value the jewelry and art work in the estate."?': 'Yes',
r'Does the word "accept" mean the same thing in sentences "The cow accepted the bull." and "To accept the report of a committee."?': 'No',
r'Does the word "fend" mean the same thing in sentences "The mountain climbers had to fend against the ice and snow." and "The youngsters had to fend for themselves after their parents died."?': 'No',
r'Does the word "hour" mean the same thing in sentences "The hour is getting late." and "The hour grows late and I must go home."?': 'Yes',
r'Does the word "whiff" mean the same thing in sentences "The gust of air whiffed away the clouds." and "Whiff a pipe."?': 'No',
r'Does the word "runoff" mean the same thing in sentences "When there is very heavy rainfall on California hillsides over a prolonged period, there is almost always ensuing surface runoff." and "The runoff of nitrates is poisoning the lake."?': 'Yes',
r'Does the word "trouble" mean the same thing in sentences "The troubles in Northern Ireland." and "What is the trouble ?"?': 'Yes',
r'Does the word "hear" mean the same thing in sentences "I heard a sound from outside the window." and "We must hear the expert before we make a decision."?': 'Yes',
r'Does the word "demonstration" mean the same thing in sentences "The lecture was accompanied by dramatic demonstrations." and "The lecturer shot off a pistol as a demonstration of the startle response."?': 'Yes',
r'Does the word "compass" mean the same thing in sentences "We compassed the earth." and "This writer attempts more than his talents can compass."?': 'No',
r'Does the word "see" mean the same thing in sentences "See that the curtains are closed." and "Do you see what I mean ?"?': 'No',
r'Does the word "arrange" mean the same thing in sentences "Arrange the books on the shelves in chronological order." and "Arrange my schedule."?': 'No',
r'Does the word "contact" mean the same thing in sentences "The surfaces contact at this point." and "The wire must not contact the metal cover."?': 'Yes',
r'Does the word "quest" mean the same thing in sentences "A quest for wealth." and "A quest for diamonds."?': 'No',
r'Does the word "edition" mean the same thing in sentences "The boy is a younger edition of his father." and "They issued a limited edition of Bach recordings."?': 'No',
r'Does the word "fundus" mean the same thing in sentences "The uterine fundus." and "The fundus of the stomach."?': 'Yes',
r'Does the word "grind" mean the same thing in sentences "Grind a lens." and "Grind the spices in a mortar."?': 'No',
r'Does the word "trim" mean the same thing in sentences "Trim a shop window." and "Trim the christmas tree."?': 'Yes',
r'Does the word "overhaul" mean the same thing in sentences "Overhaul the health care system." and "You should overhaul your car engine."?': 'Yes',
r'Does the word "change" mean the same thing in sentences "After a brisk walk, I washed up and changed my shirt." and "Ask the janitor to come and change the lightbulb."?': 'Yes',
r'Does the word "populate" mean the same thing in sentences "Deer are populating the woods." and "Populate the forest with deer and wild boar for hunting."?': 'No',
r'Does the word "function" mean the same thing in sentences "Height is a function of age." and "Price is a function of supply and demand."?': 'Yes',
r'Does the word "brandish" mean the same thing in sentences "He brandished his sword at the pirates." and "Brandish a sword."?': 'No',
r'Does the word "outdoors" mean the same thing in sentences "She loves the outdoors." and "He wanted to get outdoors a little."?': 'Yes',
r'Does the word "line" mean the same thing in sentences "Trees line the riverbank." and "To line a copy book."?': 'No',
r'Does the word "satellite" mean the same thing in sentences "The Moon is a natural satellite of the Earth." and "A spent upper stage is a derelict satellite."?': 'Yes',
r'Does the word "reform" mean the same thing in sentences "The Church reformed me." and "Reform your conduct."?': 'Yes',
r'Does the word "study" mean the same thing in sentences "Anthropology is the study of human beings." and "He knocked lightly on the closed door of the study."?': 'No',
r'Does the word "age" mean the same thing in sentences "Old age is not for sissies." and "Age has n\'t slowed him down at all."?': 'Yes',
r'Does the word "version" mean the same thing in sentences "An experimental version of the night fighter." and "His version of the fight was different from mine."?': 'No',
r'Does the word "pump" mean the same thing in sentences "I \'ve pumped over 1000 gallons of water in the last ten minutes." and "Pump the gas pedal."?': 'Yes',
r'Does the word "cattle" mean the same thing in sentences "Do you want to raise cattle ?" and "So many head of cattle."?': 'Yes',
r'Does the word "fly" mean the same thing in sentences "The pilot flew to Cuba." and "All nations fly their flags in front of the U.N."?': 'No',
r'Does the word "shipment" mean the same thing in sentences "Shipment of hazardous waste on this route is strictly prohibited." and "We \'re expecting another shipment of oranges tomorrow."?': 'No',
r'Does the word "shake" mean the same thing in sentences "These stories shook the community." and "Do n\'t shake your fist at me !"?': 'No',
r'Does the word "bone" mean the same thing in sentences "Bone the turkey before roasting it." and "I had to bone up on my Latin verbs before the final exam."?': 'No',
r'Does the word "start" mean the same thing in sentences "We simply could not start the engine." and "To start a stream of water."?': 'Yes',
r'Does the word "pulse" mean the same thing in sentences "A transmitter pulsed by an electronic tube." and "Pulse waves."?': 'Yes',
r'Does the word "foundation" mean the same thing in sentences "There is little foundation for his objections." and "The foundation of a new scientific society."?': 'No',
r'Does the word "material" mean the same thing in sentences "Coal is a hard black material." and "The archives provided rich material for a definitive biography."?': 'No',
r'Does the word "foul" mean the same thing in sentences "She \'s fouled her diaper." and "To foul the face or hands with mire."?': 'Yes',
r'Does the word "buy" mean the same thing in sentences "She buys for the big department store." and "I \'m going to buy my father something nice for his birthday."?': 'Yes',
r'Does the word "coat" mean the same thing in sentences "When the dog shed its coat, it left hair all over the furniture and the carpet." and "A second coat of paint."?': 'No',
r'Does the word "resume" mean the same thing in sentences "Resume one \'s duties." and "Resume a title."?': 'Yes',
r'Does the word "answer" mean the same thing in sentences "The answers were in the back of the book." and "There is no simple answer to corruption."?': 'Yes',
r'Does the word "prize" mean the same thing in sentences "I prize these old photographs." and "We prize his creativity."?': 'No',
r'Does the word "deposit" mean the same thing in sentences "She deposits her paycheck every month." and "Deposit the suitcase on the bench."?': 'No',
r'Does the word "graduate" mean the same thing in sentences "This school graduates 2,000 students each year." and "Graduate a cylinder."?': 'No',
r'Does the word "shade" mean the same thing in sentences "Shade your eyes when you step out into the bright sunlight." and "The butterfly wings shade to yellow."?': 'No',
r'Does the word "point" mean the same thing in sentences "He pointed to the empty parking space." and "Point a chimney."?': 'No',
r'Does the word "lesson" mean the same thing in sentences "I hope this accident taught you a lesson !" and "The accident was a good lesson to me."?': 'Yes',
r'Does the word "grace" mean the same thing in sentences "The conception of grace developed alongside the conception of sin." and "A beautiful figure which she used in subtle movements of unparalleled grace."?': 'No',
r'Does the word "negociate" mean the same thing in sentences "The parties negociated all night." and "Negociate securities."?': 'No',
r'Does the word "toehold" mean the same thing in sentences "His father gave him a toehold in the oil business." and "American diplomacy provided a toehold on which to proceed toward peace talks."?': 'Yes',
r'Does the word "garden" mean the same thing in sentences "This house has a swimming pool, a tent, a swing set and a fountain in the garden." and "Our garden is overgrown with weeds."?': 'Yes',
r'Does the word "protection" mean the same thing in sentences "The witnesses demanded police protection." and "They had no protection from the fallout."?': 'No',
r'Does the word "loosen" mean the same thing in sentences "Loosen the soil." and "Loosen the tension on a rope."?': 'No',
r'Does the word "discipline" mean the same thing in sentences "The offenders deserved the harsh discipline they received." and "He insisted on discipline among the troops."?': 'No',
r'Does the word "waste" mean the same thing in sentences "Waste a joke on an unappreciative audience." and "Waste heat."?': 'Yes',
r'Does the word "act" mean the same thing in sentences "He \'s acting strangely - I think there \'s something wrong with him." and "You should act like an adult."?': 'Yes',
r'Does the word "card" mean the same thing in sentences "I heard you do n\'t get carded at the other liquor store." and "They have to card anybody who looks 21 or younger."?': 'Yes',
r'Does the word "evoke" mean the same thing in sentences "This remark evoked sadness." and "Evoke sympathy."?': 'No',
r'Does the word "blanket" mean the same thing in sentences "There was a blanket of snow." and "The city woke under a thick blanket of fog."?': 'No',
r'Does the word "inundate" mean the same thing in sentences "The basement was inundated after the storm." and "The Dutch would sometimes inundate the land to hinder the Spanish army."?': 'No',
r'Does the word "lump" mean the same thing in sentences "Do you want one lump or two with your coffee ?" and "A lump of coal."?': 'No',
r'Does the word "snag" mean the same thing in sentences "I snagged my stocking." and "Snag a bargain."?': 'No',
r'Does the word "hollow" mean the same thing in sentences "Hunger had caused the hollows in their cheeks." and "He built himself a cabin in a hollow high up in the Appalachians."?': 'No',
r'Does the word "do" mean the same thing in sentences "Do n\'t behave like a fool." and "What makes her do this way ?"?': 'Yes',
r'Does the word "network" mean the same thing in sentences "Retirement meant dropping out of a whole network of people who had been part of my life." and "A network of canals."?': 'No',
r'Does the word "tad" mean the same thing in sentences "Not a tad of difference." and "A tad too expensive."?': 'Yes',
r'Does the word "avert" mean the same thing in sentences "How can the danger be averted ?" and "Avert a strike."?': 'Yes',
r'Does the word "receive" mean the same thing in sentences "His movie received a good review." and "These aspects of civilization do not find expression or receive an interpretation."?': 'Yes',
r'Does the word "inwardness" mean the same thing in sentences "Inwardness is what an Englishman quite simply has, painlessly, as a birthright." and "The sensitiveness of James \'s characters, their seeming inwardness."?': 'Yes',
r'Does the word "cure" mean the same thing in sentences "The smoke and heat cures the meat." and "Cure pickles."?': 'Yes',
r'Does the word "speculation" mean the same thing in sentences "Speculations about the outcome of the election." and "He knew the stock was a speculation when he bought it."?': 'No',
r'Does the word "minute" mean the same thing in sentences "You have twenty minutes to complete the test." and "He ran a 4 minute mile."?': 'Yes',
r'Does the word "dereliction" mean the same thing in sentences "What he did was a terrible dereliction of duty." and "The new soldier did not clean his cabin and was scolded for dereliction and disobedience."?': 'Yes',
r'Does the word "interpretation" mean the same thing in sentences "The edict was subject to many interpretations." and "The report included his interpretation of the forensic evidence."?': 'No',
r'Does the word "administer" mean the same thing in sentences "Administer an exam." and "Administer the last unction."?': 'No',
r'Does the word "juncture" mean the same thing in sentences "At that juncture he had no idea what to do." and "We \'re at a crucial juncture in our relationship."?': 'Yes',
r'Does the word "vow" mean the same thing in sentences "They took vows of poverty." and "The old hermit, up in the mountains, took a vow of silence."?': 'Yes',
r'Does the word "boil" mean the same thing in sentences "They brought the water to a boil." and "Add the noodles when the water comes to the boil."?': 'Yes',
r'Does the word "spike" mean the same thing in sentences "The second baseman sharpened his spikes before every game." and "The seismograph showed a sharp spike in response to the temblor."?': 'No',
r'Does the word "puff" mean the same thing in sentences "He huffed and puffed as he made his way up the mountain." and "Puff a cigar."?': 'No',
r'Does the word "demand" mean the same thing in sentences "I demand an immediate explanation." and "I demand an explanation."?': 'Yes',
r'Does the word "meet" mean the same thing in sentences "My proposal met with much opposition." and "Meet a need."?': 'No',
r'Does the word "synthesize" mean the same thing in sentences "The liver synthesizes vitamins." and "His operas synthesize music and drama in perfect harmony."?': 'Yes',
r'Does the word "flagship" mean the same thing in sentences "The retail company \'s store in London is their flagship." and "It is their flagship newspaper."?': 'Yes',
r'Does the word "stringer" mean the same thing in sentences "A first stringer." and "A second stringer."?': 'Yes',
r'Does the word "thing" mean the same thing in sentences "Things are going well." and "I could n\'t tell what the thing was."?': 'No',
r'Does the word "carrier" mean the same thing in sentences "Refrigerated carriers have revolutionized the grocery business." and "The bonds were transmitted by carrier."?': 'No',
r'Does the word "foul" mean the same thing in sentences "Jones fouled for strike one." and "Foul the water."?': 'No',
r'Does the word "ping" mean the same thing in sentences "My car was pinging until my last oil change." and "I \'ll ping my accountant -- April 15 is nearing."?': 'No',
r'Does the word "receive" mean the same thing in sentences "The lady is receiving Sunday morning." and "Receive letters from the front."?': 'No',
r'Does the word "rule" mean the same thing in sentences "Rule the margins." and "Rule one \'s temper."?': 'No',
r'Does the word "contrast" mean the same thing in sentences "The middle school teacher contrasted her best student \'s work with that of her weakest student." and "The students contrast considerably in their artistic abilities."?': 'No',
r'Does the word "tier" mean the same thing in sentences "A three tier wedding cake." and "Tier upon tier of huge casks."?': 'Yes',
r'Does the word "stone" mean the same thing in sentences "Stone is abundant in New England and there are many quarries." and "Her face was as hard as stone."?': 'No',
r'Does the word "zero" mean the same thing in sentences "The electromagnetic field does not drop all of the way to zero before a reversal." and "The temperature outside is ten degrees below zero."?': 'No',
r'Does the word "consolidation" mean the same thing in sentences "A consolidation of two corporations." and "The defendants asked for a consolidation of the actions against them."?': 'Yes',
r'Does the word "commit" mean the same thing in sentences "She committed herself to the work of God." and "Commit a random act of kindness."?': 'No',
r'Does the word "erase" mean the same thing in sentences "Her painful memories seemingly erased completely." and "Please erase the formula on the blackboard -- it is wrong !"?': 'Yes',
r'Does the word "sawmill" mean the same thing in sentences "The sawmill sells lumber to carpenters and sawdust to gardeners." and "He brought his portable sawmill and turned the old beams into interesting flooring."?': 'Yes',
r'Does the word "embarrassment" mean the same thing in sentences "The outcome of the vote was an embarrassment for the liberals." and "An embarrassment of riches."?': 'No',
r'Does the word "view" mean the same thing in sentences "He viewed the painting and praised the artist for his masterpiece." and "View a problem."?': 'Yes',
r'Does the word "sincerity" mean the same thing in sentences "They demanded some proof of my sincerity." and "His sincerity inspired belief."?': 'Yes',
r'Does the word "hair" mean the same thing in sentences "In the western world, women usually have long hair while men usually have short hair." and "He combed his hair."?': 'Yes',
r'Does the word "issue" mean the same thing in sentences "Politicians never discuss the real issues." and "She found an old issue of the magazine in her dentist \'s waiting room."?': 'No',
r'Does the word "square" mean the same thing in sentences "He squared his shoulders." and "Square the wood with a file."?': 'No',
r'Does the word "concentrate" mean the same thing in sentences "These groups concentrate in the inner cities." and "Population tends to concentrate in cities."?': 'Yes',
r'Does the word "leap" mean the same thing in sentences "The child leapt across the puddle." and "Leap into fame."?': 'No',
r'Does the word "relief" mean the same thing in sentences "The contributions provided some relief for the victims." and "He enjoyed his relief from responsibility."?': 'No',
r'Does the word "balance" mean the same thing in sentences "She has to balance work and her domestic duties." and "Balance the two weights."?': 'Yes',
r'Does the word "oxygenate" mean the same thing in sentences "After we oxygenated the river, the fish returned." and "Oxygenate blood."?': 'Yes',
r'Does the word "flame" mean the same thing in sentences "The person who posted an inflammatory message got flamed." and "The sky seemed to flame in the Hawaiian sunset."?': 'No',
r'Does the word "weather" mean the same thing in sentences "Wooden garden furniture must be well oiled as it is continuously exposed to weather." and "They were hoping for good weather."?': 'Yes',
r'Does the word "extension" mean the same thing in sentences "Most applications provide extensions for the files they create." and "Files with the.txt extension usually contain text."?': 'Yes',
r'Does the word "balance" mean the same thing in sentences "This final payment, or credit, balances the account." and "To balance a set of books."?': 'Yes',
r'Does the word "cut" mean the same thing in sentences "He insisted that we give him the last cut before every deal." and "He played the first cut on the cd."?': 'No',
r'Does the word "disorder" mean the same thing in sentences "The army tried to prevent disorder when claims the elections had been rigged grew stronger." and "The class was thrown into disorder when the teacher left the room."?': 'Yes',
r'Does the word "object" mean the same thing in sentences "It was full of rackets, balls and other objects." and "The sole object of her trip was to see her children."?': 'No',
r'Does the word "part" mean the same thing in sentences "He tried to sing the tenor part." and "The first violin part in this concerto is very challenging."?': 'Yes',
r'Does the word "appeal" mean the same thing in sentences "Their appeal was denied in the superior court." and "An appeal to raise money for starving children."?': 'No',
r'Does the word "draft" mean the same thing in sentences "They served beer on draft." and "I have to revise the first draft of my term paper."?': 'No',
r'Does the word "plant" mean the same thing in sentences "He claimed that the evidence against him was a plant." and "The police used a plant to trick the thieves."?': 'Yes',
r'Does the word "wall" mean the same thing in sentences "His back was to the wall." and "Competition was pushing them to the wall."?': 'Yes',
r'Does the word "space" mean the same thing in sentences "He said the space is the most important character in the alphabet." and "The first major milestone in space exploration was in 1957, when the USSR \'s Sputnik 1 orbited the Earth."?': 'No',
r'Does the word "ice" mean the same thing in sentences "Americans like ice in their drinks." and "Look at the ice on that dame !"?': 'No',
r'Does the word "outburst" mean the same thing in sentences "An outburst of heavy rain." and "The man let out an outburst of invective."?': 'No',
r'Does the word "fix" mean the same thing in sentences "Fix the variables." and "Fix breakfast for the guests, please."?': 'No',
r'Does the word "pizza" mean the same thing in sentences "Should we cook a frozen pizza for dinner ?" and "He ate a whole pizza !"?': 'Yes',
r'Does the word "bask" mean the same thing in sentences "The seals were basking in the sun." and "To bask in someone \'s favour."?': 'No',
r'Does the word "run" mean the same thing in sentences "Run riot." and "Run amok."?': 'Yes',
r'Does the word "cream" mean the same thing in sentences "The cream of a collection of books or pictures." and "The cream of England \'s young men were killed in the Great War."?': 'Yes',
r'Does the word "channel" mean the same thing in sentences "They offer more than one hundred channels." and "A satellite TV channel."?': 'Yes',
r'Does the word "negate" mean the same thing in sentences "A pessimism that always negates." and "The investigation tending to negate any supernatural influences."?': 'No',
r'Does the word "navel" mean the same thing in sentences "Within the navel of this hideous wood, / Immured in cypress shades, a sorcerer dwells. — Milton." and "The Incas believed that Cuzco was the navel of the universe."?': 'Yes',
r'Does the word "resolution" mean the same thing in sentences "He always wrote down his New Year \'s resolutions." and "It was his unshakeable resolution to finish the work."?': 'No',
r'Does the word "guard" mean the same thing in sentences "There would be men guarding the horses." and "Guard my possessions while I \'m away."?': 'No',
r'Does the word "subscription" mean the same thing in sentences "My library subscription is about to expire." and "I have a monthly subscription to The Daily Telegraph newspaper."?': 'Yes',
r'Does the word "stimulate" mean the same thing in sentences "Coffee and tea stimulate me." and "This herbal infusion does n\'t stimulate."?': 'Yes',
r'Does the word "catcher" mean the same thing in sentences "A catcher plays behind the plate." and "A catcher needs a lot of protective equipment."?': 'Yes',
r'Does the word "piracy" mean the same thing in sentences "How shoud the international community respond to Somali piracy ?" and "Air piracy."?': 'Yes',
r'Does the word "chuck" mean the same thing in sentences "Chuck these old notes." and "This food ′s gone off - you′d better chuck it."?': 'Yes',
r'Does the word "sewer" mean the same thing in sentences "All that work went down the sewer." and "A sewer of fine gowns."?': 'No',
r'Does the word "snapshot" mean the same thing in sentences "My snapshots have n\'t been developed yet." and "He carried a snapshot of his daughter."?': 'Yes',
r'Does the word "say" mean the same thing in sentences "Did his face say anything about how he felt ?" and "What does this painting say ?"?': 'Yes',
r'Does the word "see" mean the same thing in sentences "See whether it works." and "The artist must first learn to see."?': 'No',
r'Does the word "face" mean the same thing in sentences "He \'s always stuffing his face with chips." and "Shut your face !"?': 'Yes',
r'Does the word "tune" mean the same thing in sentences "My piano needs to be tuned." and "To tune a piano or a violin."?': 'Yes',
r'Does the word "appreciate" mean the same thing in sentences "To test the power of bees to appreciate color." and "Do you appreciate the full meaning of this letter ?"?': 'Yes',
r'Does the word "reach" mean the same thing in sentences "The water reached the doorstep." and "Can he reach ?"?': 'No',
r'Does the word "memory" mean the same thing in sentences "Memory is a facility common to all animals." and "He can do it from memory."?': 'Yes',
r'Does the word "hit" mean the same thing in sentences "He hit her hard in the face." and "One boy hit the other."?': 'Yes',
r'Does the word "sober" mean the same thing in sentences "Keep him in bed until he sobers up." and "She \'s in no state to go home by herself, so make sure she does n\'t drink anything else, and she \'ll soon sober up."?': 'Yes',
r'Does the word "truncation" mean the same thing in sentences "They were sentenced to a truncation of their limbs." and "It is an obvious truncation of the verse."?': 'Yes',
r'Does the word "analyze" mean the same thing in sentences "The inspector analyzed the building." and "Analyze a sonnet by Shakespeare."?': 'Yes',
r'Does the word "remainder" mean the same thing in sentences "11 divided by 2 is 5 remainder 1." and "17 leaves a remainder of 2 when divided by 3."?': 'Yes',
r'Does the word "experiment" mean the same thing in sentences "The composer experimented with a new style." and "Students experiment sexually."?': 'Yes',
r'Does the word "wrench" mean the same thing in sentences "Wrench one \'s ankle." and "Wrench a window off its hinges."?': 'No',
r'Does the word "tenderness" mean the same thing in sentences "The best results are generally obtained by inserting the needle into the point of maximum tenderness." and "He noted her extreme tenderness when he touched the bruise on her thigh."?': 'Yes',
r'Does the word "honour" mean the same thing in sentences "His honour is at stake." and "She swore on her honour."?': 'Yes',
r'Does the word "lash" mean the same thing in sentences "The whale lashes the sea with its tail." and "And big waves lash the frighted shores. — John Dryden."?': 'Yes',
r'Does the word "origin" mean the same thing in sentences "He comes from good origins." and "Jupiter was the origin of the radiation."?': 'No',
r'Does the word "constitutionalize" mean the same thing in sentences "A woman \'s right to an abortion was constitutionalized in the 1970 \'s." and "A good way of exercising is to constitutionalize."?': 'No',
r'Does the word "run" mean the same thing in sentences "Run a process." and "Run a subject."?': 'Yes',
r'Does the word "start" mean the same thing in sentences "I start at eight in the morning." and "Who will start ?"?': 'No',
r'Does the word "warrant" mean the same thing in sentences "The dealer warrants all the cars he sells." and "I warrant this information."?': 'Yes',
r'Does the word "enjoy" mean the same thing in sentences "Enjoy privileges." and "I plan to go travelling while I still enjoy good health."?': 'Yes',
r'Does the word "dig" mean the same thing in sentences "My seven-year-old son always digs a hole in the middle of his mashed potatoes and fills it with gravy before he starts to eat them." and "Dig a hole."?': 'Yes',
r'Does the word "pop" mean the same thing in sentences "Pop music, despite its commercial focus, includes many artistic gems." and "Listen to the pop of a champagne cork."?': 'No',
r'Does the word "spin" mean the same thing in sentences "He gave the crank a spin." and "He put some spin on the cue ball."?': 'No',
r'Does the word "ribbon" mean the same thing in sentences "Sails torn to ribbons." and "A steel or magnesium ribbon."?': 'Yes',
r'Does the word "tick" mean the same thing in sentences "He counted the ticks of the clock." and "Indicate that you are willing to receive marketing material by putting a tick in the box."?': 'No',
r'Does the word "network" mean the same thing in sentences "To get a job in today \'s economy, it is important to have a strong network." and "A railroad network."?': 'No',
r'Does the word "passing" mean the same thing in sentences "The coach sent in a passing play on third and long." and "The passing of winter."?': 'No',
r'Does the word "draft" mean the same thing in sentences "The final draft of the constitution." and "A preliminary draft."?': 'Yes',
r'Does the word "stater" mean the same thing in sentences "Farm staters." and "Keystone stater."?': 'Yes',
r'Does the word "salvation" mean the same thing in sentences "Tourism was their economic salvation." and "They turned to individualism as their salvation."?': 'Yes',
r'Does the word "generate" mean the same thing in sentences "The hydroelectric plant needs to generate more electricity." and "We ca n\'t generate enough power for the entire city."?': 'Yes',
r'Does the word "crumb" mean the same thing in sentences "There were few crumbs of comfort in the report." and "He gave only a crumb of information about his plans."?': 'Yes',
r'Does the word "start" mean the same thing in sentences "They got an early start." and "It was off to a good start."?': 'No',
r'Does the word "takeaway" mean the same thing in sentences "If you \'re hungry, there \'s a takeaway just around the corner." and "In England they call takeout food ` takeaway \'."?': 'Yes',
r'Does the word "wind" mean the same thing in sentences "The river winds through the hills." and "Wind your watch."?': 'No',
r'Does the word "attack" mean the same thing in sentences "The cancer cells are attacking his liver." and "The visiting team started to attack."?': 'No',
r'Does the word "discourage" mean the same thing in sentences "Do n\'t be discouraged by the amount of work left to do : you \'ll finish it in good time." and "We should discourage this practice among our youth."?': 'No',
r'Does the word "issue" mean the same thing in sentences "The last issue of penicillin was over a month ago." and "He died without issue."?': 'No',
r'Does the word "trip" mean the same thing in sentences "An acid trip." and "He recited the whole poem without a single trip."?': 'No',
r'Does the word "blue" mean the same thing in sentences "She was wearing blue." and "The boys in blue marched to the pipers."?': 'Yes',
r'Does the word "relation" mean the same thing in sentences "He has distant relations back in New Jersey." and "He was the hero according to his own relation."?': 'No',
r'Does the word "work" mean the same thing in sentences "Work the metal." and "Work stone into tools."?': 'Yes',
r'Does the word "check" mean the same thing in sentences "Check your facts." and "Check the spelling of this word."?': 'Yes',
r'Does the word "channel" mean the same thing in sentences "Channel information towards a broad audience." and "We will channel the traffic to the left with these cones."?': 'Yes',
r'Does the word "enclosure" mean the same thing in sentences "The winning horse was first into the unsaddling enclosure." and "He faced punishment for creating the fenced enclosure in a public park."?': 'Yes',
r'Does the word "third" mean the same thing in sentences "The play ended with Jones standing on third." and "He was cut down on a close play at third."?': 'Yes',
r'Does the word "influence" mean the same thing in sentences "The influence of mechanical action." and "Her wishes had a great influence on his thinking."?': 'No',
r'Does the word "hi" mean the same thing in sentences "Hi, how are you ?" and "I just dropped by to say “ hi ”."?': 'Yes',
r'Does the word "infamy" mean the same thing in sentences "Infamy, infamy - they \'ve all got it in for me ! - Kenneth Williams as Julius Caesar in Carry On Cleo." and "A date which will live in infamy - Franklin D. Roosevelt in response to the Japanese attack on Pearl Harbour."?': 'Yes',
r'Does the word "handcart" mean the same thing in sentences "Vendors were selling sunglasses, bagged snacks, and sundry other items from handcarts in the square." and "He used a handcart to carry the rocks away."?': 'Yes',
r'Does the word "fuckup" mean the same thing in sentences "Forgetting our anniversary was a major fuckup." and "You \'ve got to fire that fuckup."?': 'No',
r'Does the word "difference" mean the same thing in sentences "There are many differences between jazz and rock." and "You need to learn to be more tolerant of difference."?': 'Yes',
r'Does the word "problem" mean the same thing in sentences "She and her husband are having problems." and "What \'s the problem ?"?': 'No',
r'Does the word "fantasy" mean the same thing in sentences "She made a lot of money writing romantic fantasies." and "A schoolgirl fantasy."?': 'No',
r'Does the word "mingle" mean the same thing in sentences "Resourcefully he mingled music and dance." and "He was about to mingle in an unpleasant affair."?': 'No',
r'Does the word "luck" mean the same thing in sentences "The raffle is just a matter of luck." and "Sometimes it takes a bit of luck to get success."?': 'Yes',
r'Does the word "stampede" mean the same thing in sentences "The tavern owners stampeded us into overeating." and "Thunderbolts can stampede animals."?': 'No',
r'Does the word "value" mean the same thing in sentences "He has old-fashioned values." and "The value of a crotchet is twice that of a quaver."?': 'No',
r'Does the word "congener" mean the same thing in sentences "Lard was also used, though its congener, butter, was more frequently employed." and "The American shopkeeper differs from his European congener."?': 'Yes',
r'Does the word "sculpt" mean the same thing in sentences "She is sculpting the block of marble into an image of her husband." and "Sculpt a swan out of a block of ice."?': 'No',
r'Does the word "curve" mean the same thing in sentences "The road curved sharply." and "To curve a line."?': 'Yes',
r'Does the word "do" mean the same thing in sentences "She did a little dance." and "All you ever do is surf the Internet."?': 'Yes',
r'Does the word "crook" mean the same thing in sentences "She held the baby in the crook of her arm." and "A crook in the path."?': 'Yes',
r'Does the word "fit" mean the same thing in sentences "He threw a fit when his car broke down." and "He had a laughing fit which lasted more than ten minutes."?': 'Yes',
r'Does the word "tack" mean the same thing in sentences "The boat tacked." and "The sailors decided to tack the boat."?': 'Yes',
r'Does the word "disaster" mean the same thing in sentences "His policies were a disaster." and "The earthquake was a disaster."?': 'No',
r'Does the word "fiddle" mean the same thing in sentences "Zuckerman fiddled that song very nicely." and "The reporter fiddle with the facts."?': 'No',
r'Does the word "witness" mean the same thing in sentences "As a witness to the event, I can tell you that he really said that." and "She can bear witness, since she was there at the time."?': 'No',
r'Does the word "organization" mean the same thing in sentences "His compulsive organization was not an endearing quality." and "Over time, the spontaneous movement had become an organization."?': 'No',
r'Does the word "steam" mean the same thing in sentences "It really steams me to see her treat him like that." and "Steam and clean the upholstered sofa."?': 'No',
r'Does the word "submission" mean the same thing in sentences "Several of his submissions were rejected by publishers." and "What was the date of submission of your proposal ?"?': 'Yes',
r'Does the word "lead" mean the same thing in sentences "The children were playing with lead soldiers." and "The lead was in the dummy."?': 'No',
r'Does the word "elaborate" mean the same thing in sentences "She elaborated on the main ideas in her dissertation." and "What do you mean you did n\'t come home last night ? Would you care to elaborate ?"?': 'No',
r'Does the word "pass" mean the same thing in sentences "The sports car passed all the trucks." and "He attempted the examination, but did not expect to pass."?': 'No',
r'Does the word "bake" mean the same thing in sentences "Bake a cake." and "Potatoes bake to a crisp in the oven."?': 'No',
r'Does the word "foot" mean the same thing in sentences "To foot ( or foot up ) an account." and "Foot the bill."?': 'No',
r'Does the word "sufficiency" mean the same thing in sentences "Her father questioned the young suitor \'s sufficiency." and "He questioned the sufficiency of human intelligence."?': 'No',
r'Does the word "reorganization" mean the same thing in sentences "After the takeover there was a thorough reorganization." and "The reorganization was prescribed by federal bankruptcy laws."?': 'Yes',
r'Does the word "ship" mean the same thing in sentences "To ship the tiller or rudder." and "Ship the cargo in the hold of the vessel."?': 'Yes',
r'Does the word "tone" mean the same thing in sentences "They tested his hearing with pure tones of different frequencies." and "A clergyman improved the tone of the meeting."?': 'No',
r'Does the word "responsibility" mean the same thing in sentences "Why did n\'t you clean the house ? That was your responsibility !" and "Every right implies a responsibility."?': 'Yes',
r'Does the word "career" mean the same thing in sentences "The general had had a distinguished career." and "He had a long career in the law."?': 'Yes',
r'Does the word "increase" mean the same thing in sentences "The increase in unemployment." and "They proposed an increase of 15 percent in the fare."?': 'No',
r'Does the word "law" mean the same thing in sentences "The laws of thermodynamics." and "The law came looking for him."?': 'No',
r'Does the word "here" mean the same thing in sentences "An Alzheimer patient \'s in here may in his mind be anywhere he called home in the time he presently re-lives." and "Where do we go from here ?"?': 'Yes',
r'Does the word "water" mean the same thing in sentences "His eyes were watering." and "The smell of fried onions makes my mouth water."?': 'Yes',
r'Does the word "desire" mean the same thing in sentences "A man of many desires." and "Too much desire can seriously affect one \'s judgment."?': 'No',
r'Does the word "capacity" mean the same thing in sentences "The plant is working at 80 per cent capacity." and "The capacity of a hard disk drive is usually expressed in megabytes."?': 'No',
r'Does the word "care" mean the same thing in sentences "Care had aged him." and "He wrote the manual on car care."?': 'No',
r'Does the word "stretch" mean the same thing in sentences "During the Inquisition, the torturers would stretch their victims on a rack." and "Stretch my patience."?': 'No',
r'Does the word "vegetation" mean the same thing in sentences "Their holiday was spent in sleep and vegetation." and "Pleistocene vegetation."?': 'No',
r'Does the word "hand" mean the same thing in sentences "Give the little lady a great big hand." and "He wanted to try his hand at singing."?': 'No',
r'Does the word "delay" mean the same thing in sentences "He delayed the watering." and "Do n\'t delay your application to graduate school or else it wo n\'t be considered."?': 'Yes',
r'Does the word "crystallize" mean the same thing in sentences "I crystallized the copper sulphate by slowly cooling a saturated solution." and "Crystallize minerals."?': 'Yes',
r'Does the word "dig" mean the same thing in sentences "They dug an eight - foot ditch along the side of the road." and "In the wintertime, heavy truck tires dig into the road, forming potholes."?': 'Yes',
r'Does the word "stage" mean the same thing in sentences "It set the stage for peaceful negotiations." and "We are in a transitional stage in which many former ideas must be revised or rejected."?': 'No',
r'Does the word "spray" mean the same thing in sentences "A spray of bullets." and "The bridesmaid carried a spray of lily-of-the-valley."?': 'No',
r'Does the word "effect" mean the same thing in sentences "The best way to effect change is to work with existing stakeholders." and "Effect a change."?': 'Yes',
r'Does the word "seat" mean the same thing in sentences "This classroom seats two hundred students." and "Seat a concert hall."?': 'Yes',
r'Does the word "cheat" mean the same thing in sentences "My husband cheated on me with his secretary." and "Six years of a happy relationship, and then she had the nerve to cheat on me with a barman !"?': 'Yes',
r'Does the word "extract" mean the same thing in sentences "I extracted a promise from the Dean for two new positions." and "To extract an essential oil from a plant."?': 'No',
r'Does the word "sew" mean the same thing in sentences "Can the seamstress sew me a suit by next week ?" and "To sew money into a bag."?': 'No',
r'Does the word "liquidate" mean the same thing in sentences "The mafia liquidated the informer." and "Liquidate a company."?': 'No',
r'Does the word "relax" mean the same thing in sentences "He relaxed in the hot tub." and "Let \'s all relax after a hard day \'s work."?': 'Yes',
r'Does the word "begin" mean the same thing in sentences "She began Russian at an early age." and "Begin a new chapter in your life."?': 'No',
r'Does the word "corner" mean the same thing in sentences "The car corners." and "Corner the gold market."?': 'No',
r'Does the word "music" mean the same thing in sentences "You have to face the music." and "He fell asleep to the music of the wind chimes."?': 'No',
r'Does the word "appropriation" mean the same thing in sentences "The necessary funds were obtained by the government \'s appropriation of the company \'s operating unit." and "A person \'s appropriation of property belonging to another is dishonest."?': 'Yes',
r'Does the word "pickup" mean the same thing in sentences "She needed the pickup that coffee always gave her." and "Garbage pickup is on Mondays and Thursdays."?': 'No',
r'Does the word "declare" mean the same thing in sentences "His wife declared at once for moving to the West Coast." and "Declare dividends."?': 'No',
r'Does the word "name" mean the same thing in sentences "He named his demands." and "Name your accomplices !"?': 'Yes',
r'Does the word "blaze" mean the same thing in sentences "They chipped off bark to mark the trail with blazes." and "The horse had a blaze between its eyes."?': 'Yes',
r'Does the word "pitch" mean the same thing in sentences "He was employed to see that his paper \'s news pitches were not trespassed upon by rival vendors." and "It is hard to get this pitch off of my hand."?': 'No',
r'Does the word "mash" mean the same thing in sentences "To mash on a bicycle pedal." and "Mash the garlic."?': 'No',
r'Does the word "clip" mean the same thing in sentences "He gave me a clip on the ear." and "Give him a clip round the ear !"?': 'Yes',
r'Does the word "blaze" mean the same thing in sentences "Go to blazes." and "The blaze spread rapidly."?': 'No',
r'Does the word "warn" mean the same thing in sentences "I warned him not to go too far." and "I warn you against false assumptions."?': 'Yes',
r'Does the word "state" mean the same thing in sentences "Though legally single nations, many states comprise several distinct cultural or ethnic groups." and "He was in such a state you just could n\'t reason with him."?': 'No',
r'Does the word "mold" mean the same thing in sentences "A gelatin dessert made in a mold." and "A lobster mold."?': 'Yes',
r'Does the word "drain" mean the same thing in sentences "We drained the oil tank." and "Please drain the sink. It \'s full of dirty water."?': 'Yes',
r'Does the word "yesterday" mean the same thing in sentences "Yesterday \'s technology." and "Yesterday \'s solutions are not good enough."?': 'Yes',
r'Does the word "cook" mean the same thing in sentences "The apothecary cooked the medicinal mixture in a big iron kettle." and "My husband does n\'t cook."?': 'No',
r'Does the word "toast" mean the same thing in sentences "Toast bread." and "Let us toast the birthday girl !"?': 'No',
r'Does the word "writing" mean the same thing in sentences "The doctor \'s writing was illegible." and "He turned the paper over so the writing would n\'t show."?': 'Yes',
r'Does the word "circuit" mean the same thing in sentences "The international tennis circuit." and "On the lecture circuit."?': 'Yes',
r'Does the word "series" mean the same thing in sentences "A comedy series." and "The Masterworks concert series."?': 'Yes',
r'Does the word "deify" mean the same thing in sentences "The people deified their King." and "These young men deify financial success."?': 'No',
r'Does the word "signal" mean the same thing in sentences "He awaited the signal to start." and "The victory was a signal for wild celebration."?': 'Yes',
r'Does the word "automation" mean the same thing in sentences "Automation replaces human workers by machines." and "This factory floor is a showcase for automation and robotic equipment."?': 'No',
r'Does the word "fire" mean the same thing in sentences "The boss fired his secretary today." and "The gun has only been fired two times."?': 'No',
r'Does the word "goal" mean the same thing in sentences "She failed in her goal to become captain of the team." and "The winning goal came with less than a minute left to play."?': 'No',
r'Does the word "register" mean the same thing in sentences "Did you register any change when I pressed the button ?" and "Did this event register in your parents \' minds ?"?': 'No',
r'Does the word "scrap" mean the same thing in sentences "These two fellows are always scrapping over something." and "Scrap the old airplane and sell the parts."?': 'No',
r'Does the word "protocol" mean the same thing in sentences "Safety protocols." and "Academic protocol."?': 'Yes',
r'Does the word "master" mean the same thing in sentences "Her pain completely mastered her." and "The methods can master the problems."?': 'Yes',
r'Does the word "scram" mean the same thing in sentences "Scram !" and "If you do n\'t scram, I \'ll leave instead !"?': 'Yes',
r'Does the word "veil" mean the same thing in sentences "The forest fire was veiled by smoke, but I could hear it clearly." and "Women in Afghanistan veil their faces."?': 'Yes',
r'Does the word "pitcher" mean the same thing in sentences "He has played every position except pitcher." and "Our pitcher has a sore arm."?': 'No',
r'Does the word "whang" mean the same thing in sentences "Whang the ball." and "Whang away at the school reform plan."?': 'No',
r'Does the word "predecease" mean the same thing in sentences "She predeceased her husband." and "Husbands usually predecease their wives."?': 'Yes',
r'Does the word "bare" mean the same thing in sentences "Bare your feelings." and "Bare your breasts."?': 'Yes',
r'Does the word "pickup" mean the same thing in sentences "Put out a pickup on that man." and "His car has a lot of pickup."?': 'No',
r'Does the word "gimmick" mean the same thing in sentences "A cheap promotions gimmick for greedy businessmen." and "It was a great sales gimmick."?': 'Yes',
r'Does the word "restoration" mean the same thing in sentences "The restoration of this medieval church involved undoing all the Victorian modifications." and "The restoration of this painting will take years."?': 'Yes',
r'Does the word "batch" mean the same thing in sentences "A batch of letters." and "We made a batch of cookies to take to the party."?': 'No',
r'Does the word "focus" mean the same thing in sentences "The focus of infection." and "In focus."?': 'No',
r'Does the word "socialize" mean the same thing in sentences "He never socializes with his colleagues." and "The old man hates to socialize."?': 'Yes',
r'Does the word "respect" mean the same thing in sentences "He is an intellectual giant, and I have great respect for him." and "Syngman Rhee kept imprisoned the Dowager Queen Yun Empress Sunjeong of the Korean Empire for fear of the respect the people held for her."?': 'Yes',
r'Does the word "rekindle" mean the same thing in sentences "Rekindle hopes." and "Rekindle her love."?': 'Yes',
r'Does the word "clean" mean the same thing in sentences "Clean the stove !" and "Can you clean the windows today ?"?': 'Yes',
r'Does the word "electricity" mean the same thing in sentences "The stage crackled with electricity whenever she was on it." and "They built a car that runs on electricity."?': 'No',
r'Does the word "season" mean the same thing in sentences "Season the chicken breast after roasting it." and "This trip will season even the hardiest traveller."?': 'No',
r'Does the word "brand" mean the same thing in sentences "Whenever Sally drinks her favorite beer, Coors \' brand image floods her senses with scenes of being on a hot beach drinking something refreshing." and "There \'s a new brand of hero in the movies now."?': 'No',
r'Does the word "plague" mean the same thing in sentences "Those children are a damn plague." and "A plague of grasshoppers."?': 'No',
r'Does the word "draw" mean the same thing in sentences "She drew water from the barrel." and "Draw water from a well."?': 'Yes',
r'Does the word "hearth" mean the same thing in sentences "The hearth was black with the charcoal of many fires." and "They sat on the hearth and warmed themselves before the fire."?': 'No',
r'Does the word "makeup" mean the same thing in sentences "The two teams played a makeup one week later." and "She is wearing a lot of makeup."?': 'No',
r'Does the word "guard" mean the same thing in sentences "We let our guard down." and "The prison guard unlocked the door of the cell."?': 'No',
r'Does the word "ring" mean the same thing in sentences "She had rings on every finger." and "I \'ll give you a ring when the plane lands."?': 'No',
r'Does the word "home" mean the same thing in sentences "A home for the elderly." and "A home for outcasts."?': 'Yes',
r'Does the word "string" mean the same thing in sentences "String pearls on a string." and "The child drew glass beads on a string."?': 'Yes',
r'Does the word "investment" mean the same thing in sentences "He made an emotional investment in the work." and "This job calls for the investment of some hard thinking."?': 'Yes',
r'Does the word "racket" mean the same thing in sentences "Power tools work quickly, but they sure make a racket." and "What \'s all this racket ?"?': 'Yes',
r'Does the word "susurrus" mean the same thing in sentences "The soft susurrus and sighs of the branches. — Longfellow." and "A soft susurrus of conversation."?': 'Yes',
r'Does the word "process" mean the same thing in sentences "Process cheese." and "Process hair."?': 'Yes',
r'Does the word "edge" mean the same thing in sentences "He rounded the edges of the box." and "His voice had an edge to it."?': 'No',
r'Does the word "glory" mean the same thing in sentences "He valued glory above life itself." and "The glory of the sunrise."?': 'No',
r'Does the word "direction" mean the same thing in sentences "He looked the other direction." and "They proposed a new direction for the firm."?': 'No',
r'Does the word "truss" mean the same thing in sentences "Trussed bridges." and "Truss the roofs."?': 'Yes',
r'Does the word "refer" mean the same thing in sentences "This plant can be referred to a known species." and "Refer a patient to a specialist."?': 'No',
r'Does the word "bitch" mean the same thing in sentences "This problem is a real bitch." and "Karma \'s a bitch."?': 'Yes',
r'Does the word "loss" mean the same thing in sentences "The car was a total loss." and "Loss of livestock left the rancher bankrupt."?': 'Yes',
r'Does the word "conflict" mean the same thing in sentences "This behavior conflicts with our rules." and "The two proposals conflict !"?': 'No',
r'Does the word "wax" mean the same thing in sentences "The moon is waxing." and "Wax the car."?': 'No',
r'Does the word "kilter" mean the same thing in sentences "Out of kilter." and "In good kilter."?': 'Yes',
r'Does the word "pop" mean the same thing in sentences "This popcorn pops quickly in the microwave oven." and "Pop the clutch."?': 'No',
r'Does the word "stimulate" mean the same thing in sentences "The book stimulated her imagination." and "Stimulate my appetite."?': 'No',
r'Does the word "corrupt" mean the same thing in sentences "Socrates was accused of corrupting young men." and "Corrupt the morals."?': 'Yes',
r'Does the word "program" mean the same thing in sentences "You ca n\'t tell the players without a program." and "Working mothers rely on the day care program."?': 'No',
r'Does the word "vamp" mean the same thing in sentences "Vamp up an old speech." and "Vamp up an excuse for not attending the meeting."?': 'No',
r'Does the word "come" mean the same thing in sentences "This kind of accident comes when you are careless." and "Nothing good will come of this."?': 'Yes',
r'Does the word "crush" mean the same thing in sentences "Ye shall not offer unto the Lord that which is bruised, crushed, broken or cut. --Lev. xxii." and "Crush an aluminum can."?': 'Yes',
r'Does the word "agony" mean the same thing in sentences "Being in an agony he prayed more earnestly. —Luke xxii. 44." and "An agony of doubt."?': 'Yes',
r'Does the word "extent" mean the same thing in sentences "The extent of his knowledge of the language is a few scattered words." and "The full extent of the law."?': 'No',
r'Does the word "room" mean the same thing in sentences "The rooms were very small but they had a nice view." and "Room for improvement."?': 'No',
r'Does the word "expression" mean the same thing in sentences "A sad expression." and "The expression of milk from her breast."?': 'No',
r'Does the word "ticker" mean the same thing in sentences "To my surprise, the ticker showed that the deal had already gone through." and "I checked the prices on the ticker one last time before placing the trade."?': 'Yes',
r'Does the word "half" mean the same thing in sentences "You do n\'t know the half of it." and "Half an hour."?': 'Yes',
r'Does the word "consideration" mean the same thing in sentences "After much consideration, I have decided to stay." and "He showed no consideration for her feelings."?': 'No',
r'Does the word "walk" mean the same thing in sentences "I walk the dog every morning." and "I \'ll walk you to your car."?': 'Yes',
r'Does the word "height" mean the same thing in sentences "At the height of her career." and "She \'s at the height of her career."?': 'Yes',
r'Does the word "obstruct" mean the same thing in sentences "The trees obstruct my view of the mountains." and "Obstruct the path."?': 'No',
r'Does the word "disposition" mean the same thing in sentences "He has such a foul disposition." and "She has a sunny disposition."?': 'Yes',
r'Does the word "suit" mean the same thing in sentences "To deal and shuffle, to divide and sort Her mingled suits and sequences. —." and "What suit is trumps ?"?': 'Yes',
r'Does the word "laughter" mean the same thing in sentences "Their loud laughter betrayed their presence." and "He enjoyed the laughter of the crowd."?': 'No',
r'Does the word "manhood" mean the same thing in sentences "A strapping youth poised on the brink of manhood." and "He feared the speedy decline of all manhood."?': 'No',
r'Does the word "fertility" mean the same thing in sentences "In America the fertility rate of recent immigrants tends to be higher than that of native population." and "In most industrialized countries the fertility rate has dropped well below replacement rate."?': 'Yes',
r'Does the word "stun" mean the same thing in sentences "The news stunned her." and "Stun fish."?': 'No',
r'Does the word "joke" mean the same thing in sentences "He told a very funny joke." and "I regarded his campaign for mayor as a joke."?': 'No',
r'Does the word "function" mean the same thing in sentences "A seemingly endless round of social functions." and "It was a black-tie function."?': 'No',
r'Does the word "stem" mean the same thing in sentences "To stem a tide." and "Stem the tide."?': 'Yes',
r'Does the word "agency" mean the same thing in sentences "Agency for Toxic Substances and Disease Registry." and "The agency of providence."?': 'No',
r'Does the word "club" mean the same thing in sentences "He carried a club in self defense." and "He felt as if he had been hit with a club."?': 'Yes',
r'Does the word "half" mean the same thing in sentences "A century and one half." and "Half a loaf."?': 'Yes',
r'Does the word "batter" mean the same thing in sentences "To the dismay of his mother, the boy put his finger into the cake batter." and "The first batter hit the ball into the corner for a double."?': 'No',
r'Does the word "revival" mean the same thing in sentences "The revival of trade." and "A revival of a neglected play by Moliere."?': 'Yes',
r'Does the word "recognition" mean the same thing in sentences "He was unable to make his motion because he could n\'t get recognition by the chairman." and "He looked at her for ten full minutes before recognition dawned."?': 'No',
r'Does the word "splash" mean the same thing in sentences "The painter put a splash of blue on the wall to make it more colorful." and "I heard a splash when the rock landed in the pond."?': 'No',
r'Does the word "prefer" mean the same thing in sentences "Some people prefer camping to staying in hotels." and "We prefer sleeping outside."?': 'Yes',
r'Does the word "expense" mean the same thing in sentences "He kept a careful record of his expenses at the meeting." and "At the expense of."?': 'No',
r'Does the word "alphabet" mean the same thing in sentences "The Greek alphabet has only twenty-four letters." and "In the first year of school, pupils are taught to recite the alphabet."?': 'Yes',
r'Does the word "guy" mean the same thing in sentences "The guy \'s only doing it for some doll." and "A nice guy."?': 'Yes',
r'Does the word "categorize" mean the same thing in sentences "First, categorize incoming messages according to the needed actions." and "Children learn early on to categorize."?': 'Yes',
r'Does the word "step" mean the same thing in sentences "It \'s only a step to the drugstore." and "It is but a step."?': 'Yes',
r'Does the word "fulfil" mean the same thing in sentences "My lady is positively fulfilled of grace." and "You made a promise, son, and now you must fulfil it."?': 'No',
r'Does the word "reshuffle" mean the same thing in sentences "The new Prime Minister reshuffled his cabinet." and "So as to prevent cheating, he was asked to reshuffle the cards."?': 'No',
r'Does the word "vent" mean the same thing in sentences "She vented her anger." and "He vented her frustration when he got home."?': 'Yes',
r'Does the word "girlfriend" mean the same thing in sentences "Marc went to the park with his girlfriend and watched the sunset with her." and "His girlfriend kicked him out."?': 'Yes',
r'Does the word "aroma" mean the same thing in sentences "The toilet was emitting a pungent, foul aroma." and "I love the aroma of cinnamon."?': 'Yes',
r'Does the word "inclination" mean the same thing in sentences "He walked with a heavy inclination to the right." and "Artillery must take account of a weapon \'s precise inclination."?': 'No',
r'Does the word "organization" mean the same thing in sentences "His organization of the work force was very efficient." and "This painting shows little organization at first glance, but little by little the structure becomes clear."?': 'No',
r'Does the word "record" mean the same thing in sentences "The lawyer has a good record." and "The track record shows that he will be a good president."?': 'Yes',
r'Does the word "immunity" mean the same thing in sentences "Immunity to criticism." and "Some people have better immunity to diseases than others."?': 'No',
r'Does the word "shrink" mean the same thing in sentences "Can you shrink this image ?" and "Hot water will shrink the sweater."?': 'Yes',
r'Does the word "harvest" mean the same thing in sentences "Harvest the grapes." and "The Chinese are said to harvest organs from executed criminals."?': 'No',
r'Does the word "washout" mean the same thing in sentences "As an actor, he was a complete washout, so he went back to accounting." and "It was several days after the storm before they could repair the washout and open the road."?': 'No',
r'Does the word "land" mean the same thing in sentences "Good or bad land for growing potatoes." and "The land had never been plowed."?': 'Yes',
r'Does the word "day" mean the same thing in sentences "He deserves his day in court." and "Every dog has his day."?': 'Yes',
r'Does the word "salvation" mean the same thing in sentences "The salvation of his party was the president \'s major concern." and "In Islam, salvation is achieved by praying in the mosque to Allah and following the sunnah of Muhammad."?': 'Yes',
r'Does the word "service" mean the same thing in sentences "He accepted service of the subpoena." and "The service happened yesterday."?': 'Yes',
r'Does the word "throw" mean the same thing in sentences "She threw the blanket around the child." and "Throw accusations at someone."?': 'No',
r'Does the word "clap" mean the same thing in sentences "The wings of the birds clapped loudly." and "Clap two boards together."?': 'No',
r'Does the word "denier" mean the same thing in sentences "Holocaust denier ( see : Wikipedia : Holocaust denial )." and "Global warming denier ( see : Wikipedia : Global warming denial )."?': 'Yes',
r'Does the word "carry" mean the same thing in sentences "This crime carries a penalty of five years in prison." and "To carry the war from Greece into Asia."?': 'No',
r'Does the word "separatism" mean the same thing in sentences "Separatism is a serious problem in Quebec." and "Demands for some form of separatism on grounds of religion have been perceived as a threat to mainstream education."?': 'Yes',
r'Does the word "good" mean the same thing in sentences "Among the highest goods of all are happiness and self-realization." and "Weigh the good against the bad."?': 'Yes',
r'Does the word "bail" mean the same thing in sentences "The judge set bail at $ 10,000." and "He is out on bail."?': 'No',
r'Does the word "circuit" mean the same thing in sentences "We took a quick circuit of the park." and "A ten-day coach circuit of the island."?': 'Yes',
r'Does the word "spreader" mean the same thing in sentences "He used his knife as a spreader." and "Ellen used a spreader to butter her bagel."?': 'Yes',
r'Does the word "setup" mean the same thing in sentences "The classroom setup was simple and efficient." and "It takes time to learn the setup around here."?': 'Yes',
r'Does the word "product" mean the same thing in sentences "A product of lime and nitric acid." and "This is a product of lime and nitric acid."?': 'Yes',
r'Does the word "trail" mean the same thing in sentences "The bride \'s veiled trailed along the ground." and "You \'ll get your coat all muddy if you trail it around like that."?': 'Yes',
r'Does the word "walk" mean the same thing in sentences "The pitcher now has two walks in this inning alone." and "He had a funny walk."?': 'No',
r'Does the word "gang" mean the same thing in sentences "A gang of wires." and "Do a drop for the telephone gang through the ceiling of the wiring closet."?': 'Yes',
r'Does the word "claimant" mean the same thing in sentences "Claimants of unemployment compensation." and "He was a claimant to the throne."?': 'Yes',
r'Does the word "avenue" mean the same thing in sentences "It promises to open new avenues to understanding." and "They explored every avenue they could think of."?': 'Yes',
r'Does the word "mellow" mean the same thing in sentences "The sun mellowed the fruit." and "These apples need to mellow a bit more."?': 'Yes',
r'Does the word "draw" mean the same thing in sentences "John only had four cards of the same suit ; he had a drawing hand." and "He played only draw and stud."?': 'Yes',
r'Does the word "control" mean the same thing in sentences "They instituted controls over drinking on campus." and "Measures for the control of disease."?': 'Yes',
r'Does the word "variation" mean the same thing in sentences "Any variation in his routine was immediately reported." and "An emery wheel is the modern variation of a grindstone."?': 'No',
r'Does the word "amount" mean the same thing in sentences "The dogs need different amounts of food." and "The amount he had in cash was insufficient."?': 'Yes',
r'Does the word "man" mean the same thing in sentences "Students were manning the booths." and "We can not man all the desks."?': 'Yes',
r'Does the word "residence" mean the same thing in sentences "A person can have several residences." and "He refused to live in the governor \'s residence."?': 'No',
r'Does the word "compel" mean the same thing in sentences "Logic compels the wise, while fools feel compelled by emotions." and "We compel all students to fill out this form."?': 'Yes',
r'Does the word "eclat" mean the same thing in sentences "They gave him more eclat than he really deserved." and "The eclat of a great achievement."?': 'No',
r'Does the word "slave" mean the same thing in sentences "A slave to cocaine." and "A slave to fashion."?': 'Yes',
r'Does the word "overreact" mean the same thing in sentences "Do n\'t overreact to the bad news -- take it easy." and "Please try not to overreact if she drives badly when she is first learning."?': 'Yes',
r'Does the word "zinger" mean the same thing in sentences "He always greeted me with a new zinger." and "She tried to think of some killer of an argument, a real zinger that would disarm all opposition."?': 'Yes',
r'Does the word "tread" mean the same thing in sentences "For fools rush in where angels fear to tread." and "To tread land when too light."?': 'Yes',
r'Does the word "lay" mean the same thing in sentences "Lay the books on the table." and "To lay a body in the grave."?': 'Yes',
r'Does the word "softening" mean the same thing in sentences "He observed the softening of iron by heat." and "Refrigeration delayed the softening of the fruit."?': 'Yes',
r'Does the word "drink" mean the same thing in sentences "Cynthia drank the wine." and "You can lead a horse to water but you ca n\'t make him drink."?': 'Yes',
r'Does the word "squeamishness" mean the same thing in sentences "He refused to allow squeamishness to deter him from his duty." and "The program was withdrawn because of the squeamishness of some viewers."?': 'Yes',
r'Does the word "date" mean the same thing in sentences "They hoped to get together at an early date." and "You may need that at a later date."?': 'Yes',
r'Does the word "oat" mean the same thing in sentences "The oat stalks made good straw." and "The main forms of oat are meal and bran."?': 'Yes',
r'Does the word "community" mean the same thing in sentences "A community is infinitely more brutalised by the habitual employment of punishment than it is by the occasional occurrence of crime ( Oscar Wilde )." and "They shared a community of possessions."?': 'No',
r'Does the word "period" mean the same thing in sentences "Death put a period to his endeavors." and "A change soon put a period to my tranquility."?': 'Yes',
r'Does the word "unlock" mean the same thing in sentences "I unlocked the door and walked in." and "Unlock the door."?': 'Yes',
r'Does the word "fit" mean the same thing in sentences "I wanted to borrow my little sister \'s jeans, but they did n\'t fit." and "This piece wo n\'t fit into the puzzle."?': 'Yes',
r'Does the word "bed" mean the same thing in sentences "He sat on the edge of the bed." and "The room had only a bed and chair."?': 'Yes',
r'Does the word "have" mean the same thing in sentences "She has $ 1,000 in the bank." and "Have a postdoc."?': 'No',
r'Does the word "resolve" mean the same thing in sentences "Can this image be resolved ?" and "I \'ll have to resolve the equation with the new values."?': 'No',
r'Does the word "reading" mean the same thing in sentences "The teacher assigned new readings." and "He bought some reading material at the airport."?': 'Yes',
r'Does the word "hope" mean the same thing in sentences "He was their best hope for a victory." and "In spite of his troubles he never gave up hope."?': 'No',
r'Does the word "digress" mean the same thing in sentences "She always digresses when telling a story." and "Do n\'t digress when you give a lecture."?': 'Yes',
r'Does the word "audience" mean the same thing in sentences "We joined the audience just as the lights went down." and "Someone in the audience began to cough."?': 'Yes',
r'Does the word "care" mean the same thing in sentences "The nurse was caring for the wounded." and "Do you care to try this dish ?"?': 'No',
r'Does the word "head" mean the same thing in sentences "The salmon are first headed and then scaled." and "Head the fish."?': 'Yes',
r'Does the word "tease" mean the same thing in sentences "She has a way of teasing men with her flirtatious behavior." and "Tease tissue for microscopic examinations."?': 'No',
r'Does the word "file" mean the same thing in sentences "File these bills, please." and "File one \'s fingernails."?': 'No',
r'Does the word "smoking" mean the same thing in sentences "Smoking is permitted." and "Smoking stinks."?': 'Yes',
r'Does the word "entrance" mean the same thing in sentences "She made a grand entrance." and "Her entrance attracted no attention whatsoever."?': 'Yes',
r'Does the word "game" mean the same thing in sentences "The game lasted two hours." and "He is serving for the game."?': 'No',
r'Does the word "gravity" mean the same thing in sentences "Gravity is weaker than electromagnetism, but affects all matter and energy." and "The more remote the body the less the gravity."?': 'Yes',
r'Does the word "scoop" mean the same thing in sentences "He used a scoop to serve the ice cream." and "He got a scoop on the bribery of city officials."?': 'No',
r'Does the word "determination" mean the same thing in sentences "His determination showed in his every movement." and "The determination of molecular structures."?': 'No',
r'Does the word "stay" mean the same thing in sentences "Stay a bit longer -- the day is still young." and "You can stay with me while you are in town."?': 'Yes',
r'Does the word "sustain" mean the same thing in sentences "She alone sustained her family." and "The money will sustain our good cause."?': 'Yes',
r'Does the word "contour" mean the same thing in sentences "The contours of the melody." and "It defines a major contour of this administration."?': 'Yes',
r'Does the word "child" mean the same thing in sentences "They had three children." and "My youngest child is forty-three."?': 'Yes',
r'Does the word "elaborate" mean the same thing in sentences "Could you elaborate on the plot for your novel for me ?" and "Elaborate a plan."?': 'Yes',
r'Does the word "cradle" mean the same thing in sentences "He cradled the infant in his arms." and "Cradle gold."?': 'No',
r'Does the word "endorsement" mean the same thing in sentences "A star athlete can make a lot of money from endorsements." and "Companies sometimes pay millions for product endorsement by celebrities."?': 'Yes',
r'Does the word "vascularity" mean the same thing in sentences "Reduced bodyfat brought out the vascularity in his arms." and "A prominent vascularity."?': 'Yes',
r'Does the word "grey" mean the same thing in sentences "The Confederate army was a vast grey." and "He was dressed in grey."?': 'No',
r'Does the word "interest" mean the same thing in sentences "How much interest do you pay on your mortgage ?" and "He has a lot of interest in vintage cars."?': 'No',
r'Does the word "inhale" mean the same thing in sentences "Inhale deeply." and "Inhale the fresh mountain air."?': 'Yes',
r'Does the word "symbol" mean the same thing in sentences "Chinese people use word symbols for writing." and "The eagle is a symbol of the United States."?': 'No',
r'Does the word "mound" mean the same thing in sentences "They built small mounds to hide behind." and "They have a southpaw on the mound."?': 'No',
r'Does the word "genre" mean the same thing in sentences "The computer game Half-Life redefined the first-person shooter genre." and "The still-life has been a popular genre in painting since the 17th century."?': 'Yes',
r'Does the word "train" mean the same thing in sentences "Express trains do n\'t stop at Princeton Junction." and "Train of mourners."?': 'No',
r'Does the word "focus" mean the same thing in sentences "I believe I can bring the high degree of focus required for this important job." and "The controversy brought clearly into focus an important difference of opinion."?': 'No',
r'Does the word "procurement" mean the same thing in sentences "I have a lot of experience in the procurement of construction materials and sub-contracts." and "He was responsible for the procurement of materials and supplies."?': 'Yes',
r'Does the word "mediate" mean the same thing in sentences "He mediated a settlement." and "Mediate between the old and the new."?': 'No',
r'Does the word "bring" mean the same thing in sentences "Bring water to the boiling point." and "Bring comments."?': 'No',
r'Does the word "stage" mean the same thing in sentences "The stage pulled into town carrying the payroll for the mill and three ladies." and "All the world \'s a stage -- Shakespeare."?': 'No',
r'Does the word "account" mean the same thing in sentences "I ca n\'t account for the missing money." and "Passing grades account for half of the grades given in this exam."?': 'No',
r'Does the word "pluck" mean the same thing in sentences "He plucked the strings of his mandolin." and "To pluck at somebody \'s sleeve."?': 'Yes',
r'Does the word "lighten" mean the same thing in sentences "She lightened the load on the tired donkey." and "Lighten the burden of caring for her elderly parents."?': 'No',
r'Does the word "chip" mean the same thing in sentences "Potato chip, tortilla chip." and "This cup has a chip in it."?': 'No',
r'Does the word "evidence" mean the same thing in sentences "There is no evidence that anyone was here earlier." and "His trembling was evidence of his fear."?': 'No',
r'Does the word "yogi" mean the same thing in sentences "It \'s a yogi trick of some sort." and "Gurdjieff connects this type of breathing with yogi breathing."?': 'Yes',
r'Does the word "wonder" mean the same thing in sentences "She wondered whether it would snow tonight." and "I wonder whether this was the right thing to do."?': 'Yes',
r'Does the word "colony" mean the same thing in sentences "An artists \' colony." and "The American colony in Paris."?': 'No',
r'Does the word "grain" mean the same thing in sentences "Wheat is a grain that is grown in Kansas." and "A grain of sugar."?': 'No',
r'Does the word "rudeness" mean the same thing in sentences "The rudeness of the cabin gave it a rustic charm but little comfort." and "His rudeness was inexcusable."?': 'Yes',
r'Does the word "stick" mean the same thing in sentences "Cigarettes are taxed at one dollar per stick." and "The policy so far is all stick and no carrot."?': 'No',
r'Does the word "bench" mean the same thing in sentences "She placed the workpiece on the bench, inspected it closely, and opened the cover." and "Injuries have shortened the bench."?': 'No',
r'Does the word "cap" mean the same thing in sentences "They established a cap for prices." and "We should put a cap on the salaries, to keep them under control."?': 'Yes',
r'Does the word "umbrella" mean the same thing in sentences "An air umbrella over England." and "Quick, grab that umbrella before you get rained on !"?': 'No',
r'Does the word "seal" mean the same thing in sentences "Close the lid tightly to get a good seal." and "A coat of seal."?': 'No',
r'Does the word "stir" mean the same thing in sentences "Stir the soil." and "Stir the soup."?': 'Yes',
r'Does the word "issue" mean the same thing in sentences "Water issued from the hole in the wall." and "Issue a new uniform to the children."?': 'No',
r'Does the word "subject" mean the same thing in sentences "The sergeant subjected the new recruits to many drills." and "He did not want to subject himself to the judgments of his superiors."?': 'No',
r'Does the word "repression" mean the same thing in sentences "The long repression of Christian sects." and "His goal was the repression of insolence."?': 'No',
r'Does the word "property" mean the same thing in sentences "The president was concerned about the property across from the White House." and "Self-confidence is not an endearing property."?': 'No',
r'Does the word "flip" mean the same thing in sentences "Flip me the beachball." and "I do n\'t know what to do -- I may as well flip a coin !"?': 'No',
r'Does the word "support" mean the same thing in sentences "The policy found little public support." and "The foundation provided support for the experiment."?': 'No',
r'Does the word "charge" mean the same thing in sentences "That \'s a slanderous charge of abuse of trust." and "He was arrested on a charge of larceny."?': 'Yes',
r'Does the word "absorption" mean the same thing in sentences "The absorption of photons by atoms or molecules." and "The absorption of light, heat, electricity, etc."?': 'Yes',
r'Does the word "puddle" mean the same thing in sentences "There were puddles of muddy water in the road after the rain." and "His chair sat in a puddle of books and magazines."?': 'No',
r'Does the word "push" mean the same thing in sentences "Let \'s give one last push on our advertising campaign." and "Give the door a hard push if it sticks."?': 'No',
r'Does the word "ply" mean the same thing in sentences "He plied his ax with bloody results." and "Ships ply the waters near the coast."?': 'No',
r'Does the word "slot" mean the same thing in sentences "The TV program has a new time slot." and "He followed the deer \'s slot over the soft turf to the edge of the trees."?': 'No',
r'Does the word "recapitulate" mean the same thing in sentences "Ontogeny recapitulates phylogeny." and "Let \'s recapitulate the main ideas."?': 'No',
r'Does the word "slope" mean the same thing in sentences "The slope of a parabola increases linearly with x." and "He climbed the steep slope."?': 'No',
r'Does the word "tap" mean the same thing in sentences "He was tapping his fingers on the table impatiently." and "Tap beer in a bar."?': 'No',
r'Does the word "simmer" mean the same thing in sentences "Simmering water." and "Simmer the sauce."?': 'Yes',
r'Does the word "past" mean the same thing in sentences "Reporters dug into the candidate \'s past." and "Past tense."?': 'No',
r'Does the word "mark" mean the same thing in sentences "Mark my words." and "Mark homework."?': 'No',
r'Does the word "hide" mean the same thing in sentences "Muslim women hide their faces." and "Hide the money."?': 'Yes',
r'Does the word "suggestion" mean the same thing in sentences "Suggestion often works better than explicit demand." and "It was a suggestion we could n\'t refuse."?': 'No',
r'Does the word "consolidate" mean the same thing in sentences "Consolidate one \'s gains." and "Consolidate one \'s hold on first place."?': 'Yes',
r'Does the word "remember" mean the same thing in sentences "Some remember better than others." and "After the shelling, many people lost the ability to remember."?': 'Yes',
r'Does the word "instill" mean the same thing in sentences "Her presence instilled faith into the children." and "Instill medication into my eye."?': 'No',
r'Does the word "curl" mean the same thing in sentences "She curled farther down under the covers." and "Curl my hair, please."?': 'No',
r'Does the word "holder" mean the same thing in sentences "A towel holder." and "Put your umbrella in the umbrella holder."?': 'No',
r'Does the word "news" mean the same thing in sentences "Did you listen to the news tonight ?" and "We watch the 7 o\'clock news every night."?': 'Yes',
r'Does the word "acknowledge" mean the same thing in sentences "She acknowledged his complement with a smile." and "In all thy ways acknowledge Him. - Proverbs 3:6."?': 'No',
r'Does the word "confidence" mean the same thing in sentences "Confidence is always borrowed, never owned." and "He took me into his confidence."?': 'No',
r'Does the word "unload" mean the same thing in sentences "Unload the truck." and "To unload a ship."?': 'Yes',
r'Does the word "gentrify" mean the same thing in sentences "Gentrify a row of old houses." and "Gentrify the old center of town."?': 'Yes',
r'Does the word "pastime" mean the same thing in sentences "His main pastime is gambling." and "Sailing is her favorite pastime."?': 'Yes',
r'Does the word "loop" mean the same thing in sentences "The bicycle looped around the tree." and "Loop a rope."?': 'No',
r'Does the word "grind" mean the same thing in sentences "This bag contains espresso grind." and "A coarse grind of coffee."?': 'Yes',
r'Does the word "blow" mean the same thing in sentences "He blew a lot of money on his new home theater." and "Certain insects are said to blow."?': 'No',
r'Does the word "function" mean the same thing in sentences "The function of an auger is to bore holes." and "The function of a teacher."?': 'No',
r'Does the word "shape" mean the same thing in sentences "The vet checked to see what kind of shape the animal was in." and "He cut a square shape out of the cake."?': 'No',
r'Does the word "overcome" mean the same thing in sentences "He overcame his infirmity." and "You must overcome all difficulties."?': 'Yes',
r'Does the word "prime" mean the same thing in sentences "Prime a cannon." and "I need to prime these handrails before we can apply the finish coat."?': 'No',
r'Does the word "figure" mean the same thing in sentences "I ca n\'t figure if he \'s telling the truth or lying." and "He did n\'t figure her."?': 'Yes',
r'Does the word "fall" mean the same thing in sentences "A fall from virtue." and "The rise and fall of the tides."?': 'No',
r'Does the word "intimidate" mean the same thing in sentences "Her boss intimidates her." and "He \'s trying to intimidate you. If you ignore him, hopefully he \'ll stop."?': 'Yes',
r'Does the word "chairperson" mean the same thing in sentences "Address your remarks to the chairperson." and "She was the chairperson of the board and she presided over the meeting."?': 'Yes',
r'Does the word "pour" mean the same thing in sentences "Water poured onto the plants." and "Pour water over the floor."?': 'Yes',
r'Does the word "play" mean the same thing in sentences "She played nervously with her wedding ring." and "Play a hooked fish."?': 'No',
r'Does the word "bootleg" mean the same thing in sentences "They were bootlegging whiskey." and "Bootleg tapes of the diva \'s singing."?': 'No',
r'Does the word "drip" mean the same thing in sentences "The constant sound of dripping irritated him." and "I put a drip of vanilla extract in my hot cocoa."?': 'Yes',
r'Does the word "plan" mean the same thing in sentences "The rebels had planned turmoil and confusion." and "Plan the new wing of the museum."?': 'No',
r'Does the word "knead" mean the same thing in sentences "I kneaded the dough into a loaf." and "Knead dough."?': 'Yes',
r'Does the word "commitment" mean the same thing in sentences "His business commitments took him to London." and "A man of energy and commitment."?': 'No',
r'Does the word "stop" mean the same thing in sentences "The third baseman made some remarkable stops." and "It came to a stop at the bottom of the hill."?': 'No',
r'Does the word "conduct" mean the same thing in sentences "He conducted us to the palace." and "Conduct an orchestra."?': 'No',
r'Does the word "cord" mean the same thing in sentences "The burglar tied up the victim with a cord." and "The bundle was tied with a cord."?': 'Yes',
r'Does the word "complement" mean the same thing in sentences "A full complement of workers." and "The complement of 01100101_2 is 10011011_2."?': 'Yes',
r'Does the word "pig" mean the same thing in sentences "The farmer kept a pen with two pigs that he fed from table scraps and field waste." and "He was a pig and his apartment a pigpen ; take-away containers and pizza boxes in a long, moldy stream lined his counter tops."?': 'No',
r'Does the word "complaint" mean the same thing in sentences "I have no complaints about the quality of his work, but I do n\'t enjoy his company." and "Do n\'t come too close, I \'ve got this nasty complaint."?': 'No',
r'Does the word "house" mean the same thing in sentences "They had a large carriage house." and "The house gets a percentage of every bet."?': 'No',
r'Does the word "trial" mean the same thing in sentences "Most of these complaints are settled before they go to trial." and "He had a fair trial and the jury found him guilty."?': 'Yes',
r'Does the word "pull" mean the same thing in sentences "She took a pull on her cigarette." and "The pull up the hill had him breathing harder."?': 'No',
r'Does the word "finger" mean the same thing in sentences "Her fingers were long and thin." and "Human hands have five fingers : the thumb, the forefinger ( or index finger ), the middle finger, the ring finger and the little finger."?': 'Yes',
r'Does the word "record" mean the same thing in sentences "Coffee production last year broke all previous records." and "Chicago set the homicide record."?': 'Yes',
r'Does the word "moderate" mean the same thing in sentences "Moderate your speed." and "Moderate your alcohol intake."?': 'No',
r'Does the word "curve" mean the same thing in sentences "You should slow down when approaching a curve." and "She scribbled a curve on the paper."?': 'No',
r'Does the word "potentiate" mean the same thing in sentences "Potentiate the drug." and "These anti-depressants potentiate the analgesic action of opiates."?': 'Yes',
r'Does the word "circus" mean the same thing in sentences "The children always love to go to the circus." and "They used the elephants to help put up the circus."?': 'No',
r'Does the word "matter" mean the same thing in sentences "It matters that they left." and "It does not matter much."?': 'Yes',
r'Does the word "remount" mean the same thing in sentences "He remounted his horse." and "Remount a regiment."?': 'No',
r'Does the word "existence" mean the same thing in sentences "Laws in existence for centuries." and "The biggest tree in existence."?': 'No',
r'Does the word "offspring" mean the same thing in sentences "Industrialism prepared the way for acceptance of the French Revolution \'s various socialistic offspring." and "She was the mother of many offspring."?': 'No',
r'Does the word "do" mean the same thing in sentences "Do nothing." and "Do research."?': 'Yes',
r'Does the word "crispness" mean the same thing in sentences "Crispness of new dollar bills." and "The crispness of his reply."?': 'No',
r'Does the word "eligibility" mean the same thing in sentences "Eligibility of a candidate for office." and "Eligibility for a loan."?': 'Yes',
r'Does the word "possession" mean the same thing in sentences "Back then, people with psychiatric disorders were sometimes thought to be victims of demonic possession." and "The scoreboard shows a little football symbol next to the name of the team that has possession."?': 'No',
r'Does the word "asthmatic" mean the same thing in sentences "She is a chronic asthmatic." and "The painful gasps of a dying asthmatic."?': 'Yes',
r'Does the word "talk" mean the same thing in sentences "Susan talked with Rachel." and "Actions talk louder than words."?': 'Yes',
r'Does the word "trust" mean the same thing in sentences "He put more trust in his own two legs than in the gun." and "The experience destroyed his trust and personal dignity."?': 'No',
r'Does the word "refinement" mean the same thing in sentences "A man of intellectual refinement." and "He described a refinement of this technique."?': 'No',
r'Does the word "bang" mean the same thing in sentences "Tiffany has long hair and bangs." and "The party went with a bang."?': 'No',
r'Does the word "look" mean the same thing in sentences "Look at my new car !" and "Do n\'t look in the closet."?': 'Yes',
r'Does the word "appeal" mean the same thing in sentences "An appeal to the public to keep calm." and "An appeal for help."?': 'Yes',
r'Does the word "faddist" mean the same thing in sentences "The faddist magazines have been hyping that lately." and "He is such a fashion faddist."?': 'Yes',
r'Does the word "feeling" mean the same thing in sentences "I disliked him and the feeling was mutual." and "I had a feeling that she was lying."?': 'No',
r'Does the word "growth" mean the same thing in sentences "Growth was dampened by a softening of the global economy in 2001, but picked up in the subsequent years due to strong growth in China." and "The growth of population."?': 'Yes',
r'Does the word "bother" mean the same thing in sentences "A bit of a bother." and "A spot of bother."?': 'No',
r'Does the word "proportion" mean the same thing in sentences "A building of vast proportions." and "An inordinate proportion of the book is given over to quotations."?': 'No',
r'Does the word "swing" mean the same thing in sentences "The party went with a swing." and "It took time to get into the swing of things."?': 'Yes',
r'Does the word "insufficiency" mean the same thing in sentences "Pointed out the insufficiencies in my report." and "The troops went hungry because of the insufficiency of their supplies."?': 'No',
r'Does the word "zap" mean the same thing in sentences "If it \'s not warm in the middle, zap it some more." and "Zap the enemy."?': 'No',
r'Does the word "processing" mean the same thing in sentences "The processing of newly arrived immigrants." and "The processing of ore to obtain minerals."?': 'Yes',
r'Does the word "mind" mean the same thing in sentences "I do n\'t mind your behavior." and "I would n\'t mind an ice cream right now."?': 'Yes',
r'Does the word "resemble" mean the same thing in sentences "This paper resembles my own work." and "The twins resemble each other."?': 'Yes',
r'Does the word "post" mean the same thing in sentences "In England they call mail ` the post \'." and "A soldier manned the entrance post."?': 'No',
r'Does the word "couple" mean the same thing in sentences "The engaged couple." and "He \'s coming for a couple of days."?': 'No',
r'Does the word "narrow" mean the same thing in sentences "The road narrowed." and "I can not narrow down the rules for this game."?': 'No',
r'Does the word "tread" mean the same thing in sentences "She trod with care as the ground was slippery." and "To tread a path."?': 'Yes',
r'Does the word "divest" mean the same thing in sentences "The board of trustees divested $ 20 million in real estate property." and "The company decided to divest."?': 'Yes',
r'Does the word "cultivate" mean the same thing in sentences "Cultivate your musical taste." and "Cultivate the land."?': 'No',
r'Does the word "communicate" mean the same thing in sentences "The disease was mainly communicated via rats and other vermin." and "Please communicate this message to all employees."?': 'No',
r'Does the word "lash" mean the same thing in sentences "To lash something to a spar." and "Lash the horse."?': 'Yes',
r'Does the word "carpet" mean the same thing in sentences "A carpet of flowers." and "The larvae of some moths spin a web that resembles a carpet."?': 'Yes',
r'Does the word "gravitate" mean the same thing in sentences "The stars gravitate towards each other." and "Boys gravitate towards girls at that age."?': 'No',
r'Does the word "knot" mean the same thing in sentences "Their muscles stood out in knots." and "The bird had a knot of feathers forming a crest."?': 'No',
r'Does the word "wipe" mean the same thing in sentences "Brian wiped at the counter." and "Wipe the blackboard."?': 'Yes',
r'Does the word "street" mean the same thing in sentences "They walked the streets of the small town." and "He worked both sides of the street."?': 'No',
r'Does the word "look" mean the same thing in sentences "If looks could kill..." and "He gave me a dirty look."?': 'Yes',
r'Does the word "presidency" mean the same thing in sentences "Andrew Jackson expanded the power of the presidency beyond what was customary before his time." and "After many years as a party leader, she finally ascended to the presidency."?': 'Yes',
r'Does the word "den" mean the same thing in sentences "An opium den." and "A den of vice."?': 'Yes',
r'Does the word "vision" mean the same thing in sentences "He had a vision of his own death." and "He had a vision of the Virgin Mary."?': 'No',
r'Does the word "expedite" mean the same thing in sentences "He expedited the search by alphabetizing the papers." and "This should expedite the process."?': 'Yes',
r'Does the word "form" mean the same thing in sentences "The water formed little beads." and "When icicles start to form on the eaves you know the roads will be icy."?': 'Yes',
r'Does the word "freeze" mean the same thing in sentences "It \'s freezing in here !" and "Do n\'t go outside wearing just a t-shirt ; you \'ll freeze !"?': 'Yes',
r'Does the word "circulate" mean the same thing in sentences "Blood circulates in my veins." and "The air here does not circulate."?': 'Yes',
r'Does the word "revamp" mean the same thing in sentences "They revamped their old house before selling it." and "They plan to revamp the historical theater in the old downtown."?': 'Yes',
r'Does the word "toke" mean the same thing in sentences "The boys took a few tokes on a joint." and "The artist took a thoughtful toke off the joint, then passed it along."?': 'Yes',
r'Does the word "agree" mean the same thing in sentences "They agreed about what should be done." and "I ca n\'t agree with you !"?': 'Yes',
r'Does the word "device" mean the same thing in sentences "He would stoop to any device to win a point." and "The device is small enough to wear on your wrist."?': 'No',
r'Does the word "consensus" mean the same thing in sentences "Those rights and obligations are based on an unstated consensus." and "The lack of consensus reflected differences in theoretical positions."?': 'Yes',
r'Does the word "aggravate" mean the same thing in sentences "The defense made by the prisoner \'s counsel did rather aggravate than extenuate his crime. —Addison." and "To aggravate my woes. —Alexander Pope."?': 'Yes',
r'Does the word "knitting" mean the same thing in sentences "I find knitting very relaxing." and "She put down her knitting and went to answer the phone."?': 'No',
r'Does the word "adhere" mean the same thing in sentences "The residents of this village adhered to Catholicism." and "Will this wallpaper adhere to the wall ?"?': 'No',
r'Does the word "retain" mean the same thing in sentences "The dam retains the water." and "I retain this drug for a long time."?': 'Yes',
r'Does the word "cry" mean the same thing in sentences "I heard a cry from afar." and "A cry to arms."?': 'Yes',
r'Does the word "round" mean the same thing in sentences "Round the bend." and "Round the edges."?': 'No',
r'Does the word "spring" mean the same thing in sentences "The spring of a bow." and "The spring was broken."?': 'No',
r'Does the word "sprinkling" mean the same thing in sentences "Baptized with a sprinkling of holy water." and "A sprinkling of grey at his temples."?': 'No',
r'Does the word "generation" mean the same thing in sentences "Dams were built for the generation of electricity." and "They had to wait a generation for that prejudice to fade."?': 'No',
r'Does the word "model" mean the same thing in sentences "Model the latest fashion." and "Model an airplane."?': 'No',
r'Does the word "pride" mean the same thing in sentences "He took pride in his work." and "He had pride of ownership in his department."?': 'Yes',
r'Does the word "occupy" mean the same thing in sentences "He occupies the position of manager." and "The young prince will soon occupy the throne."?': 'Yes',
r'Does the word "view" mean the same thing in sentences "He took the computer with a view to pawning it." and "If there are any rabbits in this park, they keep carefully out of our view."?': 'No',
r'Does the word "baseline" mean the same thing in sentences "The established a baseline for the budget." and "The ref missed the call. The ball hit the baseline."?': 'No',
r'Does the word "program" mean the same thing in sentences "The program lasted more than two hours." and "He proposed an elaborate program of public works."?': 'No',
r'Does the word "manner" mean the same thing in sentences "His rapid manner of talking." and "All manner of persons participate."?': 'No',
r'Does the word "crack" mean the same thing in sentences "The crack of a whip." and "The crack of the bat hitting the ball."?': 'Yes',
r'Does the word "sweep" mean the same thing in sentences "Her new show dog swept all championships." and "Please sweep the floor."?': 'No',
r'Does the word "overdrive" mean the same thing in sentences "Melissa \'s brain was in overdrive." and "Troops are ready to go into overdrive as soon as the signal is given."?': 'Yes',
r'Does the word "moon" mean the same thing in sentences "He made a moon lamp that he used as a night light." and "The clock had a moon that showed various phases."?': 'Yes',
r'Does the word "zero" mean the same thing in sentences "Write 0.0 to indicate a floating point number rather than the integer zero." and "In unary and k - adic notation in general, zero is the empty string."?': 'Yes',
r'Does the word "deceleration" mean the same thing in sentences "He initiated deceleration by braking." and "The rocket is now in deceleration."?': 'Yes',
r'Does the word "concoction" mean the same thing in sentences "It suddenly spewed out a thick green concoction." and "He volunteered to taste her latest concoction."?': 'No',
r'Does the word "lifeline" mean the same thing in sentences "The airlift provided a lifeline for Berlin." and "She offered me a lifeline in my time of grief."?': 'Yes',
r'Does the word "gospel" mean the same thing in sentences "Newton \'s writings were gospel for those who followed." and "His word was gospel."?': 'No',
r'Does the word "failure" mean the same thing in sentences "He resented my failure to return his call." and "The mechanic \'s failure to check the brakes."?': 'Yes',
r'Does the word "course" mean the same thing in sentences "We offer seafood as the first course." and "The course of the river."?': 'No',
r'Does the word "note" mean the same thing in sentences "I had to co-sign his note at the bank." and "The singer held the note too long."?': 'No',
r'Does the word "prepare" mean the same thing in sentences "Prepare a report." and "Prepare a speech."?': 'Yes',
r'Does the word "array" mean the same thing in sentences "Drawn up in battle array." and "An array of troops in battle order."?': 'Yes',
r'Does the word "buster" mean the same thing in sentences "Crime buster." and "Dam buster."?': 'Yes',
r'Does the word "sell" mean the same thing in sentences "He sold his house in January." and "I \'ll sell you all three for a hundred dollars."?': 'Yes',
r'Does the word "point" mean the same thing in sentences "The point of the arrow was due north." and "A point is defined by its coordinates."?': 'No',
r'Does the word "nick" mean the same thing in sentences "I nicked myself while I was shaving." and "Nick horses."?': 'No',
r'Does the word "butter" mean the same thing in sentences "Lora buttered the toast." and "Lora buttered the toast with unsalted butter."?': 'Yes',
r'Does the word "print" mean the same thing in sentences "Paw prints were everywhere." and "Using a crayon, the girl made a print of the leaf under the page."?': 'Yes',
r'Does the word "post" mean the same thing in sentences "He set a row of posts in the ground and strung barbwire between them." and "Two of the receivers ran post patterns."?': 'Yes',
r'Does the word "crutch" mean the same thing in sentences "He walked on crutches for a month until the cast was removed from his leg." and "He uses drugs as a psychological crutch."?': 'No',
r'Does the word "skip" mean the same thing in sentences "He skipped a row in the text and so the sentence was incomprehensible." and "Skip town."?': 'No',
r'Does the word "sweat" mean the same thing in sentences "Sweat poured off his brow." and "He \'s in a sweat about exams."?': 'No',
r'Does the word "lock" mean the same thing in sentences "He was locked in a laughing fit." and "Lock the bike to the fence."?': 'No',
r'Does the word "blade" mean the same thing in sentences "Gay young blades bragged of their amorous adventures." and "A blade of lint on his suit."?': 'No',
r'Does the word "grasp" mean the same thing in sentences "A terrible power had her in its grasp." and "He has a good grasp of accounting practices."?': 'No',
r'Does the word "trade" mean the same thing in sentences "The skilled trades were the first to organize modern labor unions." and "As they say in the trade."?': 'Yes',
r'Does the word "acquire" mean the same thing in sentences "I acquired a strong aversion to television." and "Children acquire language at an amazing rate."?': 'No',
r'Does the word "segregate" mean the same thing in sentences "This neighborhood is segregated." and "We do n\'t segregate in this county."?': 'Yes',
r'Does the word "dean" mean the same thing in sentences "Dean of the diplomatic corps - a country \'s most senior ambassador." and "He is the dean of foreign correspondents."?': 'Yes',
r'Does the word "series" mean the same thing in sentences "The harmonic series has been much studied." and "The third series of Friends aired from 1996 to 1997."?': 'No',
r'Does the word "flee" mean the same thing in sentences "He threw down his gun and fled." and "The prisoner tried to flee, but was caught by the guards."?': 'Yes',
r'Does the word "race" mean the same thing in sentences "Some biologists doubt that there are important genetic differences between races of human beings." and "The race is to the swift."?': 'No',
r'Does the word "crisscross" mean the same thing in sentences "Wrinkles crisscrossed her face." and "Crisscross the sheet of paper."?': 'No',
r'Does the word "web" mean the same thing in sentences "Some of that content is now only available on the Web." and "The trees cast a delicate web of shadows over the lawn."?': 'No',
r'Does the word "steep" mean the same thing in sentences "Steep the fruit in alcohol." and "Steep the blossoms in oil."?': 'Yes',
r'Does the word "stalk" mean the same thing in sentences "I stalked the woods for game." and "Stalk the woods for deer."?': 'Yes',
r'Does the word "retraction" mean the same thing in sentences "Retraction of the foreskin." and "The retraction of the landing gear."?': 'Yes',
r'Does the word "bandy" mean the same thing in sentences "We bandied around these difficult questions." and "To bandy words ( with somebody )."?': 'No',
r'Does the word "transportation" mean the same thing in sentences "The sentence was one of transportation for life." and "Mulligan \'s sentence was commuted from death to transportation."?': 'Yes',
r'Does the word "center" mean the same thing in sentences "They were raising money to build a new center for research." and "It is in the center of town."?': 'No',
r'Does the word "access" mean the same thing in sentences "He gained access to the building." and "He took a wrong turn on the access to the bridge."?': 'No',
r'Does the word "tack" mean the same thing in sentences "He tacked together some verses." and "Tack the notice on the board."?': 'No',
r'Does the word "cybernate" mean the same thing in sentences "We live in a cybernated age." and "Cybernate a factory."?': 'Yes',
r'Does the word "redefinition" mean the same thing in sentences "She provided a redefinition of his duties." and "Words like ` conservative \' require periodic redefinition."?': 'Yes',
r'Does the word "vice" mean the same thing in sentences "Smoking is a vice, not a virtue." and "Vice offends the moral standards of the community."?': 'Yes',
r'Does the word "land" mean the same thing in sentences "He made it the law of the land." and "Maybe that \'s how it works in TV - land, bu not in the real world."?': 'No',
r'Does the word "bar" mean the same thing in sentences "The window was protected by steel bars." and "There was no bar against leaving."?': 'No',
r'Does the word "real" mean the same thing in sentences "A real number can be regarded abstractly as an equivalence class of Cauchy sequences of real numbers." and "Every integer is a real number, but not vice versa."?': 'Yes',
r'Does the word "solve" mean the same thing in sentences "Solve an equation." and "Solve for x."?': 'Yes',
r'Does the word "superlative" mean the same thing in sentences "` most surely \' is the superlative of the adverb ` surely \'." and "The superlative of " big " is " biggest "."?': 'Yes',
r'Does the word "exposure" mean the same thing in sentences "The exposure of his anger was shocking." and "The studio had a northern exposure."?': 'No',
r'Does the word "kind" mean the same thing in sentences "What kinds of desserts are there ?" and "This is a strange kind of tobacco."?': 'Yes',
r'Does the word "vengeance" mean the same thing in sentences "Vengeance is mine." and "He swore vengeance on the man who betrayed him."?': 'Yes',
r'Does the word "hack" mean the same thing in sentences "He hacked his way through the forest." and "I ca n\'t hack it anymore."?': 'No',
r'Does the word "file" mean the same thing in sentences "File a complaint." and "File for divorce."?': 'Yes',
r'Does the word "justify" mean the same thing in sentences "The end justifies the means." and "Justify the margins."?': 'No',
r'Does the word "suspect" mean the same thing in sentences "The U.S. suspected Bin Laden as the mastermind behind the terrorist attacks." and "I suspect him of being the thief."?': 'Yes',
r'Does the word "claim" mean the same thing in sentences "His claim asked for damages." and "They struck in support of their claim for a shorter work day."?': 'No',
r'Does the word "arrangement" mean the same thing in sentences "The arrangement of the furniture." and "He changed the arrangement of the topics."?': 'No',
r'Does the word "action" mean the same thing in sentences "He is out of action." and "A man of action."?': 'No',
r'Does the word "contact" mean the same thing in sentences "They claimed that they had been in contact with extraterrestrial beings." and "They kept in daily contact."?': 'Yes',
r'Does the word "capture" mean the same thing in sentences "Capture an idea." and "Capture the essence of Spring."?': 'Yes',
r'Does the word "count" mean the same thing in sentences "Count your change." and "Your views do n\'t count here."?': 'No',
r'Does the word "sensitivity" mean the same thing in sentences "A galvanometer of extreme sensitivity." and "Sensitivity to pain."?': 'No',
r'Does the word "intrude" mean the same thing in sentences "They intruded on our dinner party." and "The colors do n\'t intrude on the viewer."?': 'No',
r'Does the word "ball" mean the same thing in sentences "She kicked him in the balls and got away." and "The children were playing ball on the beach."?': 'No',
r'Does the word "principal" mean the same thing in sentences "A portion of your mortgage payment goes to reduce the principal, and the rest covers interest." and "She sent unruly pupils to see the principal."?': 'No',
r'Does the word "row" mean the same thing in sentences "They rowed the canals of Venice." and "Row the boat across the lake."?': 'Yes',
r'Does the word "strain" mean the same thing in sentences "He created a new strain of sheep." and "His responsibilities were a constant strain."?': 'No',
r'Does the word "sleeper" mean the same thing in sentences "The British call a railroad tie a sleeper." and "The winner was a true sleeper -- no one expected him to get it."?': 'No',
r'Does the word "correct" mean the same thing in sentences "The stock market corrected." and "Correct the alignment of the front wheels."?': 'No',
r'Does the word "excogitate" mean the same thing in sentences "Excogitate a way to measure the speed of light." and "The scientist must stop to observe and start to excogitate."?': 'No',
r'Does the word "glimpse" mean the same thing in sentences "We glimpsed the Queen as she got into her limousine." and "I have only begun to glimpse the magnitude of the problem."?': 'Yes',
r'Does the word "explode" mean the same thing in sentences "The enemy exploded the bridge." and "They sought to explode the myth."?': 'Yes',
r'Does the word "interest" mean the same thing in sentences "Primary colors can add interest to a room." and "They said nothing of great interest."?': 'Yes',
r'Does the word "matter" mean the same thing in sentences "He always took some reading matter with him on the plane." and "Is anything the matter ?"?': 'No',
r'Does the word "observe" mean the same thing in sentences "She observed that his presentation took up too much time." and "Please observe the reaction of these two chemicals."?': 'No',
r'Does the word "heart" mean the same thing in sentences "The wood at the heart of a tree is the oldest." and "Buddhists believe that suffering is right at the heart of all life."?': 'Yes',
r'Does the word "extend" mean the same thing in sentences "Extend your backyard." and "Extend a hand."?': 'No',
r'Does the word "rawness" mean the same thing in sentences "After taking a cold, rawness of the larynx and trachea come on." and "Their poor behavior was due to the rawness of the troops."?': 'No',
r'Does the word "inspire" mean the same thing in sentences "The patient has trouble inspiring." and "The lung cancer patient can not inspire air very well."?': 'Yes',
r'Does the word "favor" mean the same thing in sentences "That style is in favor this season." and "The outcome was in his favor."?': 'No',
r'Does the word "coarsen" mean the same thing in sentences "Because the wool is poor quality it will coarsen the fabric." and "Coarsen the surface."?': 'Yes',
r'Does the word "reek" mean the same thing in sentences "Your fridge reeks of egg." and "You reek of perfume."?': 'Yes',
r'Does the word "second" mean the same thing in sentences "The treasure is 2 minutes and 45 seconds south of here." and "He came in a close second."?': 'No',
r'Does the word "disappearance" mean the same thing in sentences "He regretted the disappearance of Greek from school curricula." and "What was responsible for the disappearance of the rainforest ?"?': 'Yes',
r'Does the word "shock" mean the same thing in sentences "Whole fields of wheat in shock." and "He was numb with shock."?': 'No',
r'Does the word "basis" mean the same thing in sentences "He worked on an interim basis." and "The whole argument rested on a basis of conjecture."?': 'No',
r'Does the word "quarter" mean the same thing in sentences "The Latin Quarter." and "A quarter of a pound."?': 'No',
r'Does the word "sickness" mean the same thing in sentences "A great sickness of his judgment." and "Drugs have become a sickness they can not cure."?': 'Yes',
r'Does the word "importance" mean the same thing in sentences "The importance of a well-balanced diet." and "A person of importance."?': 'No',
r'Does the word "convert" mean the same thing in sentences "Convert centimeters into inches." and "Could you convert my dollars into pounds ?"?': 'Yes',
r'Does the word "line" mean the same thing in sentences "` let me show you my etchings \' is a rather worn line." and "That salesman must have practiced his fast line of talk."?': 'Yes',
r'Does the word "provide" mean the same thing in sentences "The hostess provided lunch for all the guests." and "We aim to provide the local community with more green spaces."?': 'Yes',
r'Does the word "score" mean the same thing in sentences "He scored a 200." and "Score the clay before firing it."?': 'No',
r'Does the word "reach" mean the same thing in sentences "This car can reach a speed of 140 miles per hour." and "To reach one a book."?': 'No',
r'Does the word "balance" mean the same thing in sentences "The balance of the agreement remains in effect." and "He took what he wanted and I got the balance."?': 'Yes',
r'Does the word "gasp" mean the same thing in sentences "I \'m popping out for a gasp." and "She gave a gasp and fainted."?': 'Yes',
r'Does the word "void" mean the same thing in sentences "The huge desert voids." and "Nobody has crossed the void since one man died trying three hundred years ago ; it \'s high time we had another go."?': 'Yes',
r'Does the word "high" mean the same thing in sentences "Summer temperatures reached an all-time high." and "I \'m on a permanent high these days."?': 'No',
r'Does the word "campaign" mean the same thing in sentences "An election campaign." and "I managed his campaign for governor."?': 'No',
r'Does the word "notice" mean the same thing in sentences "He gave notice two months before he moved." and "His hard work soon attracted the teacher \'s notice."?': 'No',
r'Does the word "flux" mean the same thing in sentences "The newness and flux of the computer industry." and "That high a neutron flux would be lethal in seconds."?': 'No',
r'Does the word "love" mean the same thing in sentences "She was his first love." and "Their love left them indifferent to their surroundings."?': 'Yes',
r'Does the word "treatment" mean the same thing in sentences "The treatment of water sewage." and "The right to equal treatment in the criminal justice system."?': 'Yes',
r'Does the word "addition" mean the same thing in sentences "The addition of five more items to the agenda will make the meeting unbearably long." and "The addition of flowers created a pleasing effect."?': 'Yes',
r'Does the word "harvest" mean the same thing in sentences "They gathered a harvest of examples." and "A harvest of love."?': 'Yes',
r'Does the word "divaricate" mean the same thing in sentences "The road divaricates here." and "Divaricate one \'s fingers."?': 'No',
r'Does the word "resign" mean the same thing in sentences "He resigned the crown to follow his heart." and "Resign a claim to the throne."?': 'No',
r'Does the word "know" mean the same thing in sentences "This student knows her irregular verbs." and "I know Latin."?': 'Yes',
r'Does the word "grace" mean the same thing in sentences "It has become less common to say grace before having dinner." and "Their youngest son said grace."?': 'Yes',
r'Does the word "read" mean the same thing in sentences "Have you read this book ?" and "The King will read the proclamation at noon."?': 'Yes',
r'Does the word "cross" mean the same thing in sentences "The trains crossed." and "Cross a horse and a donkey."?': 'No',
r'Does the word "eyeful" mean the same thing in sentences "She was a statuesque redheaded eyeful." and "They wanted to see violence and they got an eyeful."?': 'No',
r'Does the word "superlative" mean the same thing in sentences "` fastest \' is the superlative of the adjective ` fast \'." and "` least famous \' is the superlative degree of the adjective ` famous \'."?': 'Yes',
r'Does the word "edit" mean the same thing in sentences "The same family has been editing the influential newspaper for almost 100 years." and "Edit film."?': 'No',
r'Does the word "represent" mean the same thing in sentences "The flower represents a young girl." and "I represent the silent majority."?': 'No',
r'Does the word "endorsement" mean the same thing in sentences "The association announced its endorsement of the policy." and "A Democrat usually gets the union \'s endorsement."?': 'No',
r'Does the word "vacate" mean the same thing in sentences "Vacate a death sentence." and "You must vacate your office by tonight."?': 'No',
r'Does the word "chokehold" mean the same thing in sentences "The president applied a chokehold to labor disputes that inconvenienced the public." and "He grabbed the woman in a chokehold, demanded her cash and jewelry, and then fled."?': 'No',
r'Does the word "rent" mean the same thing in sentences "We rented our apartment to friends while we were abroad." and "Let \'s rent a car."?': 'No',
r'Does the word "sell" mean the same thing in sentences "The new idea sold well in certain circles." and "The French try to sell us their image as great lovers."?': 'No',
r'Does the word "express" mean the same thing in sentences "Italians express coffee rather than filter it." and "Can you express this distance in kilometers ?"?': 'No',
r'Does the word "juice" mean the same thing in sentences "Digestive juices." and "Squeeze the orange and some juice will come out."?': 'No',
r'Does the word "cast" mean the same thing in sentences "Cast a ballot." and "Cast a warm light."?': 'No',
r'Does the word "matter" mean the same thing in sentences "Several matters to attend to." and "What \'s the matter ?"?': 'Yes',
r'Does the word "content" mean the same thing in sentences "They could read to their heart \'s content." and "The two groups were similar in content."?': 'No',
r'Does the word "rusticate" mean the same thing in sentences "He was rusticated for his bad behavior." and "Rusticate the house in the country."?': 'No',
r'Does the word "converge" mean the same thing in sentences "Social forces converged to bring the Fascists back to power." and "The lines converge at this point."?': 'No',
r'Does the word "bang" mean the same thing in sentences "My bangs keep getting in my eyes when I swim." and "When he struck it with a hammer, there was a loud bang."?': 'No',
r'Does the word "carry" mean the same thing in sentences "She carried herself well." and "He can not carry a tune."?': 'No',
r'Does the word "wind" mean the same thing in sentences "As they accelerated onto the motorway, the wind tore the plywood off the car \'s roof-rack." and "That \'s a lot of wind."?': 'No',
r'Does the word "bury" mean the same thing in sentences "They buried the stolen goods." and "Bury a bone."?': 'Yes',
r'Does the word "edition" mean the same thing in sentences "The first edition appeared in 1920." and "It was too late for the morning edition."?': 'Yes',
r'Does the word "rank" mean the same thing in sentences "The fancy hotel was of the first rank." and "Based on your test scores, you have a rank of 23."?': 'Yes',
r'Does the word "train" mean the same thing in sentences "It led to a train of disasters." and "A train of thought."?': 'No',
r'Does the word "trough" mean the same thing in sentences "The buoy bobbed between the crests and troughs of the waves moving across the bay." and "The neurologist pointed to a troubling trough in the pattern of his brain-waves."?': 'Yes',
r'Does the word "level" mean the same thing in sentences "The hunter levels the gun before taking a shot." and "Level the ground."?': 'No',
r'Does the word "day" mean the same thing in sentences "Two days later they left." and "Every dog has its day."?': 'Yes',
r'Does the word "stoop" mean the same thing in sentences "He stooped to tie his shoe-laces." and "I wo n\'t stoop to reading other people \'s mail."?': 'No',
r'Does the word "feeling" mean the same thing in sentences "He had a queasy feeling." and "I had a strange feeling in my leg."?': 'Yes',
r'Does the word "custody" mean the same thing in sentences "The mother was awarded custody of the children." and "Your guests are now in my custody."?': 'Yes',
r'Does the word "erupt" mean the same thing in sentences "Unrest erupted in the country." and "Erupt in anger."?': 'No',
r'Does the word "differ" mean the same thing in sentences "The winter schedule differed from the spring schedule." and "I beg to differ !"?': 'No',
r'Does the word "fear" mean the same thing in sentences "We should not fear the Communists !" and "I fear the winters in Moscow."?': 'Yes',
r'Does the word "ligate" mean the same thing in sentences "The enzyme ligated." and "Ligate the artery."?': 'No',
r'Does the word "sign" mean the same thing in sentences "It was a sign from God." and "Do n\'t forget the minus sign."?': 'No',
r'Does the word "presidency" mean the same thing in sentences "In France, a presidency lasts for five years." and "Ernest was a historian specializing in the presidency of Herbert Hoover."?': 'Yes',
r'Does the word "shuffle" mean the same thing in sentences "The data packets are shuffled before transmission." and "Do n\'t forget to shuffle the cards."?': 'Yes',
r'Does the word "aim" mean the same thing in sentences "He took aim and fired." and "It was created with the conscious aim of answering immediate needs."?': 'No',
r'Does the word "wounded" mean the same thing in sentences "They had to leave the wounded where they fell." and "The wounded lay on stretchers waiting for surgery."?': 'Yes',
r'Does the word "boil" mean the same thing in sentences "Boil some water in a pan." and "Boil this liquid until it evaporates."?': 'Yes',
r'Does the word "victory" mean the same thing in sentences "The general always gets credit for his army \'s victory." and "Clinched a victory."?': 'Yes',
r'Does the word "understanding" mean the same thing in sentences "There was an understanding between management and the workers." and "He has virtually no understanding of social cause and effect."?': 'No',
r'Does the word "quarry" mean the same thing in sentences "Michelangelo personally quarried marble from the world-famous quarry at Carrara." and "A British term for ` quarry \' is ` stone pit \'."?': 'Yes',
r'Does the word "game" mean the same thing in sentences "For him, life is all fun and games." and "For actors, memorizing lines is no game."?': 'Yes',
r'Does the word "holder" mean the same thing in sentences "In 2012, there were 28 living holders of the Victoria Cross or the George Cross." and "He \'s been an account holder with us since 2004."?': 'Yes',
r'Does the word "chapter" mean the same thing in sentences "He joined the Atlanta chapter." and "He read a chapter every night before falling asleep."?': 'No',
r'Does the word "put" mean the same thing in sentences "Put your things here." and "We put the time of arrival at 8 P.M."?': 'No',
r'Does the word "deflate" mean the same thing in sentences "The balloons deflated." and "Deflate the air mattress."?': 'No',
r'Does the word "love" mean the same thing in sentences "Children need a lot of love." and "A mother \'s love is not easily shaken."?': 'Yes',
r'Does the word "feed" mean the same thing in sentences "Her success feeds her vanity." and "Feed carrots into a food processor."?': 'No',
r'Does the word "age" mean the same thing in sentences "She aged gracefully." and "We age every day -- what a depressing thought !"?': 'Yes',
r'Does the word "counsel" mean the same thing in sentences "The lawyer counselled his client to remain silent." and "Psychiatrists, psychologists, social workers and other mental health professionals counsel clients."?': 'Yes',
r'Does the word "leak" mean the same thing in sentences "I have to take a leak." and "The leak gained on the ship \'s pumps."?': 'No',
r'Does the word "repair" mean the same thing in sentences "To repair a shattered fortune." and "To repair a house, a road, a shoe, or a ship."?': 'Yes',
r'Does the word "wall" mean the same thing in sentences "The wall followed the road." and "He ducked behind the garden wall and waited."?': 'Yes',
r'Does the word "agree" mean the same thing in sentences "They agreed about what to do." and "All parties agree in the expediency of the law."?': 'Yes',
r'Does the word "place" mean the same thing in sentences "Place emphasis on a certain point." and "Place a phone call."?': 'No',
r'Does the word "nursing" mean the same thing in sentences "Study nursing." and "She went into nursing as a career."?': 'Yes',
r'Does the word "naught" mean the same thing in sentences "All my efforts led to naught." and "It was all for naught."?': 'No',
r'Does the word "onrush" mean the same thing in sentences "The explosion interrupted the wild onrush of her thoughts." and "From the bow she stared at the mesmerising onrush of the sea where it split and foamed."?': 'Yes',
r'Does the word "correction" mean the same thing in sentences "The correction of his vision with eye glasses." and "Market runups are invariably followed by a correction."?': 'No',
r'Does the word "fumble" mean the same thing in sentences "He fumbled the key into the lock." and "Fumble a grounder."?': 'No',
r'Does the word "extent" mean the same thing in sentences "The vast extent of the desert." and "An orchard of considerable extent."?': 'Yes',
r'Does the word "stress" mean the same thing in sentences "He put the stress on the wrong syllable." and "Stress is a vasoconstrictor."?': 'No',
r'Does the word "build" mean the same thing in sentences "Build a million-dollar business." and "Build a modern nation."?': 'Yes',
r'Does the word "exchange" mean the same thing in sentences "Exchange prisoners." and "Exchange employees between branches of the company."?': 'Yes',
}

In [5]:
reverse=True

In [7]:
!model
if not reverse:
  questions = list(human_readable_questions_full.keys())
else:
  questions = list(human_readable_reversed.keys())
for i in range(len(questions)):
    prompt = f'Answer the question with Yes or No!\n{questions[i]}'
    print(prompt)  # or process this with gemma

/bin/bash: line 1: model: command not found
Answer the question with Yes or No!
Does the word "defeat" mean the same thing in sentences "The army \'s only defeat." and "It was a narrow defeat."?
Answer the question with Yes or No!
Does the word "groom" mean the same thing in sentences "Sheila groomed the horse." and "Groom the dogs."?
Answer the question with Yes or No!
Does the word "penetration" mean the same thing in sentences "Any penetration, however slight, is sufficient to complete the offense." and "The penetration of upper management by women."?
Answer the question with Yes or No!
Does the word "hit" mean the same thing in sentences "An interesting idea hit her." and "We hit Detroit at one in the morning but kept driving through the night."?
Answer the question with Yes or No!
Does the word "deliberation" mean the same thing in sentences "A little deliberation would have deterred them." and "He was a man of judicial deliberation."?
Answer the question with Yes or No!
Does the 